# EDA

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid", palette="muted")
df = pd.read_csv("MiNDAT.csv")
print("Shape:", df.shape)
print("\nInfo:")
print(df.info())
print("\nSummary Stats:")
print(df.describe().T.head())
missing = df.isnull().sum().sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=missing[:20].values, y=missing[:20].index, palette="viridis")
plt.title("Missing Values per Feature (Top 20)")
plt.show()

plt.figure(figsize=(8,5))
sns.histplot(df['CORRUCYSTIC_DENSITY'].dropna(), bins=40, kde=True, color="royalblue")
plt.title("Distribution of CORRUCYSTIC_DENSITY")
plt.show()

corr = df.corr(numeric_only=True)
plt.figure(figsize=(12,8))
sns.heatmap(corr[['CORRUCYSTIC_DENSITY']].sort_values(by='CORRUCYSTIC_DENSITY', ascending=False),
            annot=True, cmap='coolwarm')
plt.title("Correlation of Features with Target")
plt.show()

categorical_cols = df.select_dtypes(include=['object']).columns
print("\nCategorical columns:", categorical_cols.tolist())

for col in categorical_cols:
    plt.figure(figsize=(10,5))
    sns.boxplot(x=col, y="CORRUCYSTIC_DENSITY", data=df)
    plt.xticks(rotation=45)
    plt.title(f"Boxplot of {col} vs CORRUCYSTIC_DENSITY")
    plt.show()

top_corr_features = corr['CORRUCYSTIC_DENSITY'].abs().sort_values(ascending=False).head(6).index
print("\nTop correlated features with target:", top_corr_features.tolist())

sns.pairplot(df[top_corr_features].dropna())
plt.show()

# Initial Approach

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings("ignore")


def load_data():
    train = pd.read_csv(r"/kaggle/input/recruitment-task-for-gdsc-ml/MiNDAT.csv")
    test = pd.read_csv(r"/kaggle/input/recruitment-task-for-gdsc-ml/MiNDAT_UNK.csv")
    return train, test


def preprocess(train, test, target="CORRUCYSTIC_DENSITY", id_col="LOCAL_IDENTIFIER"):
    # drop rows with missing target
    train = train.dropna(subset=[target])

    features = [c for c in train.columns if c not in [target, id_col]]
    X_train, y_train = train[features], train[target]
    X_test = test[features]

    # encode categoricals
    cat_cols = X_train.select_dtypes(include="object").columns
    encoders = {}
    for col in cat_cols:
        le = LabelEncoder()
        data = pd.concat([X_train[col], X_test[col]]).fillna("missing")
        le.fit(data)
        X_train[col] = le.transform(X_train[col].fillna("missing"))
        X_test[col] = le.transform(X_test[col].fillna("missing"))
        encoders[col] = le

    # impute + scale numerics
    num_cols = X_train.select_dtypes(include=np.number).columns
    imputer = SimpleImputer(strategy="median")
    scaler = StandardScaler()

    X_train[num_cols] = imputer.fit_transform(X_train[num_cols])
    X_test[num_cols] = imputer.transform(X_test[num_cols])

    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=features)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=features)

    return X_train, X_test, y_train


def train_models(X, y):
    models = {
        "RF": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
        "GB": GradientBoostingRegressor(n_estimators=100, random_state=42),
        "Ridge": Ridge(alpha=1.0),
        "Lasso": Lasso(alpha=1.0),
    }

    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    scores, best_rmse, best_model = {}, float("inf"), None
    for name, model in models.items():
        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)
        rmse = mean_squared_error(y_val, preds, squared=False)
        scores[name] = {
            "RMSE": rmse,
            "MAE": mean_absolute_error(y_val, preds),
            "R2": r2_score(y_val, preds),
        }
        if rmse < best_rmse:
            best_rmse, best_model = rmse, model

    # retrain best on full data
    best_model.fit(X, y)
    return best_model, scores


def tune_model(model, X, y):
    grids = {
        "RandomForestRegressor": {
            "n_estimators": [100, 200],
            "max_depth": [10, 20, None],
        },
        "GradientBoostingRegressor": {
            "n_estimators": [100, 200],
            "learning_rate": [0.05, 0.1],
            "max_depth": [3, 5],
        },
        "Ridge": {"alpha": [0.1, 1, 10]},
        "Lasso": {"alpha": [0.01, 0.1, 1]},
    }

    name = model.__class__.__name__
    if name not in grids:
        return model

    grid = GridSearchCV(model, grids[name], cv=3, scoring="neg_mean_squared_error", n_jobs=-1)
    grid.fit(X, y)
    return grid.best_estimator_

train, test = load_data()
X_train, X_test, y_train = preprocess(train, test)

best_model, scores = train_models(X_train, y_train)
tuned = tune_model(best_model, X_train, y_train)

preds = tuned.predict(X_test)
sub = pd.DataFrame({
    "LOCAL_IDENTIFIER": test["LOCAL_IDENTIFIER"],
    "CORRUCYSTIC_DENSITY": preds
})
sub.to_csv("submission.csv", index=False)

print("Scores:", scores)
print("Saved submission:", sub.shape)

model, submission = tuned, sub

# Version - 2 (Used better feature engineering, increased number of models...)

In [ ]:
import pandas as pd, numpy as np, warnings
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import xgboost as xgb, lightgbm as lgb
warnings.filterwarnings("ignore")

# Load Data
train = pd.read_csv(r"/kaggle/input/recruitment-task-for-gdsc-ml/MiNDAT.csv")
test = pd.read_csv(r"/kaggle/input/recruitment-task-for-gdsc-ml/MiNDAT_UNK.csv")

# Target / ID
target, id_col = "CORRUCYSTIC_DENSITY", "LOCAL_IDENTIFIER"
train = train.dropna(subset=[target])

# Outlier Removal
Q1, Q3 = train[target].quantile([0.25, 0.75])
IQR = Q3 - Q1
lb, ub = Q1 - 1.5*IQR, Q3 + 1.5*IQR
train = train[(train[target] >= lb) & (train[target] <= ub)]

features = [c for c in train.columns if c not in [target, id_col]]
X, y = train[features], train[target]
X_test = test[features]

# Encode categoricals
cat_cols = X.select_dtypes(include="object").columns
for col in cat_cols:
    le = LabelEncoder()
    data = pd.concat([X[col], X_test[col]]).fillna("missing")
    le.fit(data)
    X[col] = le.transform(X[col].fillna("missing"))
    X_test[col] = le.transform(X_test[col].fillna("missing"))

# Handle numericals
num_cols = X.select_dtypes(include=np.number).columns
for col in num_cols:
    X[col] = X[col].replace([np.inf, -np.inf], np.nan)
    X_test[col] = X_test[col].replace([np.inf, -np.inf], np.nan)

imputer = KNNImputer(n_neighbors=5)
X[num_cols] = imputer.fit_transform(X[num_cols])
X_test[num_cols] = imputer.transform(X_test[num_cols])

# Feature Engineering
imp_feats = num_cols[:10]
for i, f1 in enumerate(imp_feats):
    for f2 in imp_feats[i+1:i+3]:
        X[f'{f1}_x_{f2}'] = X[f1]*X[f2]
        X_test[f'{f1}_x_{f2}'] = X_test[f1]*X_test[f2]
        X[f'{f1}_div_{f2}'] = X[f1]/(X[f2]+1e-8)
        X_test[f'{f1}_div_{f2}'] = X_test[f1]/(X_test[f2]+1e-8)

for f in num_cols[:5]:
    X[f'{f}_squared'] = X[f]**2
    X_test[f'{f}_squared'] = X_test[f]**2
    X[f'{f}_sqrt'] = np.sqrt(np.abs(X[f]))
    X_test[f'{f}_sqrt'] = np.sqrt(np.abs(X_test[f]))

# col name fix for XGBoost / LightGBM
import re

# Clean feature names for LightGBM / XGBoost compatibility
def clean_column(name):
    # Replace any non-alphanumeric character with underscore
    return re.sub(r'[^A-Za-z0-9_]', '_', str(name))

X.columns = [clean_column(c) for c in X.columns]
X_test.columns = [clean_column(c) for c in X_test.columns]
# Scaling
scaler = RobustScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Models
models = {
    "RF": RandomForestRegressor(n_estimators=300, max_depth=15, random_state=42, n_jobs=-1),
    "ET": ExtraTreesRegressor(n_estimators=300, max_depth=15, random_state=42, n_jobs=-1),
    "GB": GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    "XGB": xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42),
    "LGB": lgb.LGBMRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42, verbose=-1),
    "Ridge": Ridge(alpha=10.0),
    "Lasso": Lasso(alpha=1.0),
    "ElasticNet": ElasticNet(alpha=1.0, l1_ratio=0.5),
}

# CV Training
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores, trained_models = {}, {}
for name, model in models.items():
    rmses, folds = [], []
    for tr, val in kf.split(X):
        m = type(model)(**model.get_params())
        m.fit(X.iloc[tr], y.iloc[tr])
        p = m.predict(X.iloc[val])
        rmses.append(mean_squared_error(y.iloc[val], p, squared=False))
        folds.append(m)
    scores[name] = {"CV_RMSE": np.mean(rmses), "CV_STD": np.std(rmses)}
    trained_models[name] = folds
    print(f"{name}: {np.mean(rmses):.4f} (+/- {np.std(rmses):.4f})")

best_model_name = min(scores, key=lambda x: scores[x]["CV_RMSE"])
print(f"\nBest model: {best_model_name}")

# hyperparameter Tuning (grid only for best model)
grids = {
    "RF": {"n_estimators":[200,300,400],"max_depth":[10,15,20]},
    "ET": {"n_estimators":[200,300,400],"max_depth":[10,15,20]},
    "GB": {"n_estimators":[150,200,250],"learning_rate":[0.03,0.05,0.07]},
    "XGB": {"n_estimators":[200,300,400],"learning_rate":[0.03,0.05,0.07]},
    "LGB": {"n_estimators":[200,300,400],"learning_rate":[0.03,0.05,0.07]},
    "Ridge": {"alpha":[0.1,1,10,100]},
    "Lasso": {"alpha":[0.01,0.1,1,10]},
    "ElasticNet": {"alpha":[0.1,1,10],"l1_ratio":[0.1,0.5,0.9]}
}
if best_model_name in grids:
    base = models[best_model_name]
    grid = GridSearchCV(base, grids[best_model_name], cv=3,
                        scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)
    grid.fit(X, y)
    tuned_model = grid.best_estimator_
    print(f"Best tuned params: {grid.best_params_}")
else:
    tuned_model = None

# Ensemble (top 3 weighted by RMSE)
top3 = sorted(scores.items(), key=lambda x: x[1]["CV_RMSE"])[:3]
ens, total_w = None, 0
for name, score in top3:
    w = 1.0/score["CV_RMSE"]
    total_w += w
    fold_preds = np.mean([m.predict(X_test) for m in trained_models[name]], axis=0)
    ens = w*fold_preds if ens is None else ens + w*fold_preds
ensemble_preds = ens/total_w

# Final Predictions
if tuned_model is not None:
    tuned_preds = tuned_model.predict(X_test)
    final_preds = ensemble_preds
else:
    final_preds = ensemble_preds

sub = pd.DataFrame({id_col: test[id_col], target: final_preds})
sub.to_csv("submission.csv", index=False)

print("\n--- FINAL RESULTS ---")
for n, s in sorted(scores.items(), key=lambda x: x[1]["CV_RMSE"]):
    print(f"{n:10} - CV RMSE: {s['CV_RMSE']:.4f} (+/- {s['CV_STD']:.4f})")
print(f"\nSubmission shape: {sub.shape}")
print(f"Pred stats - mean: {final_preds.mean():.2f}, std: {final_preds.std():.2f}, "
      f"min: {final_preds.min():.2f}, max: {final_preds.max():.2f}")


# After this point I started running the models locally since online on Kaggle or Colab was slow and prone to runtime disconnections

## Approach 3

In [ ]:
import pandas as pd, numpy as np, warnings
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.preprocessing import LabelEncoder, QuantileTransformer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import xgboost as xgb, lightgbm as lgb
from catboost import CatBoostRegressor
import re
warnings.filterwarnings("ignore")

# Load Data - CORRUCYST Fragment Analysis
train = pd.read_csv("MiNDAT.csv")
test = pd.read_csv("MiNDAT_UNK.csv")

# Target / ID - CORRUCYSTIC_DENSITY represents fragment data integrity
target, id_col = "CORRUCYSTIC_DENSITY", "LOCAL_IDENTIFIER"
train = train.dropna(subset=[target])

print(f"Initial dataset: {train.shape[0]} fragments")
print(f"Target distribution - Mean: {train[target].mean():.4f}, Std: {train[target].std():.4f}")
print(f"Target range: [{train[target].min():.4f}, {train[target].max():.4f}]")

def detect_outliers_iqr(data, multiplier=1.5):
    Q1, Q3 = data.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    return (data < Q1 - multiplier*IQR) | (data > Q3 + multiplier*IQR)

def detect_outliers_zscore(data, threshold=3):
    z_scores = np.abs((data - data.mean()) / data.std())
    return z_scores > threshold

outliers_iqr = detect_outliers_iqr(train[target], multiplier=2.0)
outliers_zscore = detect_outliers_zscore(train[target], threshold=3.5)
extreme_outliers = outliers_iqr & outliers_zscore

print(f"Removing {extreme_outliers.sum()} extreme outliers ({extreme_outliers.sum()/len(train)*100:.2f}%)")
train = train[~extreme_outliers]

features = [c for c in train.columns if c not in [target, id_col]]
X, y = train[features], train[target]
X_test = test[features]

print(f"Feature space: {len(features)} dimensions")
print(f"Training fragments: {X.shape[0]}")
print(f"Test fragments: {X_test.shape[0]}")

cat_cols = X.select_dtypes(include="object").columns
print(f"Categorical features (alien classifications): {len(cat_cols)}")

# Check for high cardinality categories that might need special handling
for col in cat_cols:
    unique_train = X[col].nunique()
    unique_total = pd.concat([X[col], X_test[col]]).nunique()
    print(f"  {col}: {unique_train} train classes, {unique_total} total classes")

# Advanced categorical encoding
for col in cat_cols:
    combined_data = pd.concat([X[col], X_test[col]]).fillna("CORRUPTED_DATA")
    if combined_data.nunique() > 20:
        target_means = train.groupby(col)[target].mean()
        global_mean = train[target].mean()
        counts = train.groupby(col).size()
        smoothing_factor = 10
        smoothed_means = (target_means * counts + global_mean * smoothing_factor) / (counts + smoothing_factor)
        
        X[col] = X[col].map(smoothed_means).fillna(global_mean)
        X_test[col] = X_test[col].map(smoothed_means).fillna(global_mean)
    else:
        le = LabelEncoder()
        le.fit(combined_data)
        X[col] = le.transform(X[col].fillna("CORRUPTED_DATA"))
        X_test[col] = le.transform(X_test[col].fillna("CORRUPTED_DATA"))

num_cols = X.select_dtypes(include=np.number).columns
print(f"Numerical features (sensor readings): {len(num_cols)}")

for col in num_cols:
    # Replace infinities with NaN for proper imputation
    X[col] = X[col].replace([np.inf, -np.inf], np.nan)
    X_test[col] = X_test[col].replace([np.inf, -np.inf], np.nan)
    
    if X[col].notna().sum() > 10:
        mean_val = X[col].mean()
        std_val = X[col].std()
        if std_val > 0:
            lower_bound = mean_val - 5 * std_val
            upper_bound = mean_val + 5 * std_val
            X[col] = X[col].clip(lower_bound, upper_bound)
            X_test[col] = X_test[col].clip(lower_bound, upper_bound)

# Advanced Imputation Strategy
print("Applying advanced imputation for missing alien data...")

imputer = IterativeImputer(random_state=42, max_iter=10)
X_imputed = pd.DataFrame(
    imputer.fit_transform(X[num_cols]),
    columns=num_cols,
    index=X.index
)
X_test_imputed = pd.DataFrame(
    imputer.transform(X_test[num_cols]),
    columns=num_cols,
    index=X_test.index
)

X[num_cols] = X_imputed
X_test[num_cols] = X_test_imputed

print("Engineering features for CORRUCYST fragment analysis...")

feature_importance_selector = SelectKBest(f_regression, k=min(15, len(num_cols)))
feature_importance_selector.fit(X[num_cols], y)
important_features = num_cols[feature_importance_selector.get_support()]

print(f"Selected {len(important_features)} most important features for engineering")

for i, f1 in enumerate(important_features[:8]):  
    for j, f2 in enumerate(important_features[i+1:i+4]):  
        X[f'{f1}_x_{f2}'] = X[f1] * X[f2]
        X_test[f'{f1}_x_{f2}'] = X_test[f1] * X_test[f2]
        
        X[f'{f1}_div_{f2}'] = X[f1] / (np.abs(X[f2]) + 1e-8)
        X_test[f'{f1}_div_{f2}'] = X_test[f1] / (np.abs(X_test[f2]) + 1e-8)

for f in important_features[:6]:
    X[f'{f}_squared'] = X[f] ** 2
    X_test[f'{f}_squared'] = X_test[f] ** 2
    
    X[f'{f}_cbrt'] = np.sign(X[f]) * np.abs(X[f]) ** (1/3)
    X_test[f'{f}_cbrt'] = np.sign(X_test[f]) * np.abs(X_test[f]) ** (1/3)
    
    X[f'{f}_log'] = np.log1p(np.abs(X[f]))
    X_test[f'{f}_log'] = np.log1p(np.abs(X_test[f]))

if len(important_features) > 5:
    X['important_mean'] = X[important_features].mean(axis=1)
    X['important_std'] = X[important_features].std(axis=1)
    X['important_skew'] = X[important_features].skew(axis=1)
    X['important_max'] = X[important_features].max(axis=1)
    X['important_min'] = X[important_features].min(axis=1)
    
    X_test['important_mean'] = X_test[important_features].mean(axis=1)
    X_test['important_std'] = X_test[important_features].std(axis=1)
    X_test['important_skew'] = X_test[important_features].skew(axis=1)
    X_test['important_max'] = X_test[important_features].max(axis=1)
    X_test['important_min'] = X_test[important_features].min(axis=1)

energy_features = [f for f in X.columns if any(keyword in f.lower() for keyword in ['energy', 'power', 'volt', 'amp'])]
if len(energy_features) >= 2:
    X['energy_coherence'] = X[energy_features].sum(axis=1) / (X[energy_features].std(axis=1) + 1e-8)
    X_test['energy_coherence'] = X_test[energy_features].sum(axis=1) / (X_test[energy_features].std(axis=1) + 1e-8)

print(f"Feature engineering complete. New feature count: {X.shape[1]}")

def clean_column(name):
    return re.sub(r'[^A-Za-z0-9_]', '_', str(name))

X.columns = [clean_column(c) for c in X.columns]
X_test.columns = [clean_column(c) for c in X_test.columns]

print("Applying robust scaling for alien biocomputer measurements...")

scaler = QuantileTransformer(output_distribution='normal', random_state=42)
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# Keep original data for tree-based models and scaled for linear models
X_original = X.copy()
X_test_original = X_test.copy()
X = X_scaled
X_test = X_test_scaled

print(f"Final feature matrix: {X.shape} for training, {X_test.shape} for prediction")

models = {
    "RF": RandomForestRegressor(n_estimators=500, max_depth=20, random_state=42, n_jobs=-1, 
                               min_samples_split=3, min_samples_leaf=2),
    "ET": ExtraTreesRegressor(n_estimators=500, max_depth=20, random_state=42, n_jobs=-1,
                             min_samples_split=3, min_samples_leaf=2),
    "GB": GradientBoostingRegressor(n_estimators=300, learning_rate=0.03, max_depth=8, 
                                   random_state=42, subsample=0.8),
    "XGB": xgb.XGBRegressor(n_estimators=500, learning_rate=0.03, max_depth=8, 
                           random_state=42, subsample=0.8, colsample_bytree=0.8),
    "LGB": lgb.LGBMRegressor(n_estimators=500, learning_rate=0.03, max_depth=8, 
                            random_state=42, subsample=0.8, colsample_bytree=0.8, verbose=-1),
    "CatBoost": CatBoostRegressor(iterations=500, learning_rate=0.03, depth=8, 
                                 random_seed=42, verbose=False),
    "Ridge": Ridge(alpha=10.0),
    "Lasso": Lasso(alpha=1.0, max_iter=2000),
    "ElasticNet": ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=2000),
    "BayesianRidge": BayesianRidge(),
    "Huber": HuberRegressor(epsilon=1.35, max_iter=500),
}

# Tree-based models use original data, linear models use scaled data
tree_models = ["RF", "ET", "GB", "XGB", "LGB", "CatBoost"]
linear_models = ["Ridge", "Lasso", "ElasticNet", "BayesianRidge", "Huber"]

# Enhanced Cross-Validation with Multiple Metrics
kf = KFold(n_splits=7, shuffle=True, random_state=42)
scores, trained_models = {}, {}

print("\n=== CORRUCYST Fragment Analysis Results ===")
for name, model in models.items():
    print(f"Training {name} for alien biocomputer analysis...")

    X_train = X_original if name in tree_models else X
    X_test_model = X_test_original if name in tree_models else X_test
    
    rmses, maes, r2s, folds = [], [], [], []
    for tr, val in kf.split(X_train):
        m = type(model)(**model.get_params())
        m.fit(X_train.iloc[tr], y.iloc[tr])
        p = m.predict(X_train.iloc[val])
        
        rmses.append(mean_squared_error(y.iloc[val], p, squared=False))
        maes.append(mean_absolute_error(y.iloc[val], p))
        r2s.append(r2_score(y.iloc[val], p))
        folds.append(m)
    
    scores[name] = {
        "CV_RMSE": np.mean(rmses), "CV_RMSE_STD": np.std(rmses),
        "CV_MAE": np.mean(maes), "CV_MAE_STD": np.std(maes),
        "CV_R2": np.mean(r2s), "CV_R2_STD": np.std(r2s)
    }
    trained_models[name] = folds
    
    print(f"{name:12} - RMSE: {np.mean(rmses):.4f} (±{np.std(rmses):.4f}) | "
          f"MAE: {np.mean(maes):.4f} (±{np.std(maes):.4f}) | "
          f"R²: {np.mean(r2s):.4f} (±{np.std(r2s):.4f})")

best_model_name = min(scores, key=lambda x: scores[x]["CV_RMSE"])
print(f"\nBest performing model: {best_model_name}")

print(f"\nPerforming advanced hyperparameter tuning for {best_model_name}...")

hyperparameter_grids = {
    "RF": {
        "n_estimators": [300, 500, 700],
        "max_depth": [15, 20, 25],
        "min_samples_split": [2, 3, 5],
        "min_samples_leaf": [1, 2, 3]
    },
    "ET": {
        "n_estimators": [300, 500, 700],
        "max_depth": [15, 20, 25],
        "min_samples_split": [2, 3, 5],
        "min_samples_leaf": [1, 2, 3]
    },
    "GB": {
        "n_estimators": [200, 300, 400],
        "learning_rate": [0.02, 0.03, 0.05],
        "max_depth": [6, 8, 10],
        "subsample": [0.7, 0.8, 0.9]
    },
    "XGB": {
        "n_estimators": [300, 500, 700],
        "learning_rate": [0.02, 0.03, 0.05],
        "max_depth": [6, 8, 10],
        "subsample": [0.7, 0.8, 0.9]
    },
    "LGB": {
        "n_estimators": [300, 500, 700],
        "learning_rate": [0.02, 0.03, 0.05],
        "max_depth": [6, 8, 10],
        "subsample": [0.7, 0.8, 0.9]
    },
    "CatBoost": {
        "iterations": [300, 500, 700],
        "learning_rate": [0.02, 0.03, 0.05],
        "depth": [6, 8, 10]
    },
    "Ridge": {"alpha": [0.1, 1, 10, 100, 1000]},
    "Lasso": {"alpha": [0.001, 0.01, 0.1, 1, 10]},
    "ElasticNet": {"alpha": [0.1, 1, 10], "l1_ratio": [0.1, 0.5, 0.7, 0.9]},
    "BayesianRidge": {"alpha_1": [1e-6, 1e-5, 1e-4], "alpha_2": [1e-6, 1e-5, 1e-4]},
    "Huber": {"epsilon": [1.1, 1.35, 1.5, 2.0], "alpha": [0.0001, 0.001, 0.01]}
}

tuned_model = None
if best_model_name in hyperparameter_grids:
    base_model = models[best_model_name]
    X_tune = X_original if best_model_name in tree_models else X
    
    grid = GridSearchCV(
        base_model, 
        hyperparameter_grids[best_model_name], 
        cv=5,
        scoring="neg_mean_squared_error", 
        n_jobs=-1, 
        verbose=1
    )
    grid.fit(X_tune, y)
    tuned_model = grid.best_estimator_
    print(f"Best hyperparameters: {grid.best_params_}")
    print(f"Tuned model CV score: {-grid.best_score_:.4f}")


# Top performing models weighted ensemble
top_models = sorted(scores.items(), key=lambda x: x[1]["CV_RMSE"])[:5]
print("Top 5 models for ensemble:")
for name, score in top_models:
    print(f"  {name}: RMSE {score['CV_RMSE']:.4f}")

# Weight by inverse RMSE with exponential scaling
ensemble_weights = {}
total_weight = 0
for name, score in top_models:
    weight = np.exp(-score["CV_RMSE"] * 2)
    ensemble_weights[name] = weight
    total_weight += weight

# Normalize weights
for name in ensemble_weights:
    ensemble_weights[name] /= total_weight
    print(f"  {name} weight: {ensemble_weights[name]:.3f}")

# Create ensemble predictions
ensemble_preds = np.zeros(X_test.shape[0])
for name, weight in ensemble_weights.items():
    X_pred = X_test_original if name in tree_models else X_test
    fold_preds = np.mean([m.predict(X_pred) for m in trained_models[name]], axis=0)
    ensemble_preds += weight * fold_preds

# 2. Stacking ensemble (meta-learner)
print("\nCreating stacking ensemble...")
meta_features = np.zeros((X.shape[0], len(top_models)))
meta_test_features = np.zeros((X_test.shape[0], len(top_models)))

# Generate meta-features using cross-validation
for fold_idx, (tr, val) in enumerate(kf.split(X)):
    for model_idx, (name, _) in enumerate(top_models):
        X_fold = X_original if name in tree_models else X
        X_test_fold = X_test_original if name in tree_models else X_test
        
        # Train on fold and predict validation
        model = trained_models[name][fold_idx]
        meta_features[val, model_idx] = model.predict(X_fold.iloc[val])

# Generate test meta-features
for model_idx, (name, _) in enumerate(top_models):
    X_pred = X_test_original if name in tree_models else X_test
    meta_test_features[:, model_idx] = np.mean(
        [m.predict(X_pred) for m in trained_models[name]], axis=0
    )

# Train meta-learner (simple Ridge regression)
meta_learner = Ridge(alpha=1.0)
meta_learner.fit(meta_features, y)
stacking_preds = meta_learner.predict(meta_test_features)

# Final predictions
if tuned_model is not None:
    X_final = X_original if best_model_name in tree_models else X_test
    tuned_preds = tuned_model.predict(X_final)
    final_preds = 0.4 * ensemble_preds + 0.3 * stacking_preds + 0.3 * tuned_preds
else:
    final_preds = 0.6 * ensemble_preds + 0.4 * stacking_preds

# Generate submission
submission = pd.DataFrame({
    id_col: test[id_col], 
    target: final_preds
})
submission.to_csv("enhanced_corrucyst_submission.csv", index=False)

# Approach 4

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")


def load_data():
    train = pd.read_csv("MiNDAT.csv")
    test = pd.read_csv("MiNDAT_UNK.csv")
    return train, test


def preprocess(train, test, target="CORRUCYSTIC_DENSITY", id_col="LOCAL_IDENTIFIER"):
    # drop rows with missing target
    train = train.dropna(subset=[target])
    
    # Remove outliers using IQR method for better model performance
    Q1 = train[target].quantile(0.25)
    Q3 = train[target].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    train = train[(train[target] >= lower_bound) & (train[target] <= upper_bound)]
    
    print(f"After outlier removal: {train.shape[0]} samples")

    features = [c for c in train.columns if c not in [target, id_col]]
    X_train, y_train = train[features], train[target]
    X_test = test[features]

    # Advanced categorical encoding
    cat_cols = X_train.select_dtypes(include="object").columns
    encoders = {}
    for col in cat_cols:
        le = LabelEncoder()
        data = pd.concat([X_train[col], X_test[col]]).fillna("missing")
        le.fit(data)
        X_train[col] = le.transform(X_train[col].fillna("missing"))
        X_test[col] = le.transform(X_test[col].fillna("missing"))
        encoders[col] = le

    # Handle numerical features
    num_cols = X_train.select_dtypes(include=np.number).columns
    
    # Replace infinite values
    for col in num_cols:
        X_train[col] = X_train[col].replace([np.inf, -np.inf], np.nan)
        X_test[col] = X_test[col].replace([np.inf, -np.inf], np.nan)
    
    # Advanced imputation using KNN
    imputer = KNNImputer(n_neighbors=5)
    X_train[num_cols] = imputer.fit_transform(X_train[num_cols])
    X_test[num_cols] = imputer.transform(X_test[num_cols])
    
    # Feature engineering - create interaction features
    important_features = num_cols[:10]  # Top numerical features
    for i, feat1 in enumerate(important_features):
        for feat2 in important_features[i+1:i+3]:  # Limit combinations to avoid explosion
            # Multiplication
            X_train[f'{feat1}_x_{feat2}'] = X_train[feat1] * X_train[feat2]
            X_test[f'{feat1}_x_{feat2}'] = X_test[feat1] * X_test[feat2]
            
            # Ratio (avoid division by zero)
            X_train[f'{feat1}_div_{feat2}'] = X_train[feat1] / (X_train[feat2] + 1e-8)
            X_test[f'{feat1}_div_{feat2}'] = X_test[feat1] / (X_test[feat2] + 1e-8)
    
    # Statistical features
    for i, feat in enumerate(num_cols[:5]):  # Limit to prevent overfitting
        X_train[f'{feat}_squared'] = X_train[feat] ** 2
        X_test[f'{feat}_squared'] = X_test[feat] ** 2
        
        X_train[f'{feat}_sqrt'] = np.sqrt(np.abs(X_train[feat]))
        X_test[f'{feat}_sqrt'] = np.sqrt(np.abs(X_test[feat]))
    
    # Update feature list
    features = X_train.columns.tolist()
    
    # Advanced scaling using RobustScaler (less sensitive to outliers)
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train = pd.DataFrame(X_train_scaled, columns=features)
    X_test = pd.DataFrame(X_test_scaled, columns=features)

    return X_train, X_test, y_train


def train_models(X, y):
    models = {
        "RF": RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1, max_depth=15),
        "ET": ExtraTreesRegressor(n_estimators=300, random_state=42, n_jobs=-1, max_depth=15),
        "GB": GradientBoostingRegressor(n_estimators=200, random_state=42, learning_rate=0.05),
        "XGB": xgb.XGBRegressor(n_estimators=300, random_state=42, learning_rate=0.05, max_depth=6),
        "LGB": lgb.LGBMRegressor(n_estimators=300, random_state=42, learning_rate=0.05, max_depth=6, verbose=-1),
        "Ridge": Ridge(alpha=10.0),
        "Lasso": Lasso(alpha=1.0),
        "ElasticNet": ElasticNet(alpha=1.0, l1_ratio=0.5),
    }

    # Use more robust validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    scores, cv_scores, trained_models = {}, {}, {}
    
    print("Training models with 5-fold cross-validation...")
    
    for name, model in models.items():
        print(f"Training {name}...")
        
        # Cross-validation scores
        cv_rmse_scores = []
        fold_models = []
        
        for train_idx, val_idx in kf.split(X):
            X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            model_copy = type(model)(**model.get_params())
            model_copy.fit(X_tr, y_tr)
            preds = model_copy.predict(X_val)
            rmse = mean_squared_error(y_val, preds, squared=False)
            cv_rmse_scores.append(rmse)
            fold_models.append(model_copy)
        
        avg_rmse = np.mean(cv_rmse_scores)
        std_rmse = np.std(cv_rmse_scores)
        
        scores[name] = {
            "CV_RMSE": avg_rmse,
            "CV_STD": std_rmse,
        }
        
        cv_scores[name] = cv_rmse_scores
        trained_models[name] = fold_models
        
        print(f"{name} - CV RMSE: {avg_rmse:.4f} (+/- {std_rmse:.4f})")

    # Find best model
    best_model_name = min(scores.keys(), key=lambda x: scores[x]["CV_RMSE"])
    print(f"\nBest model: {best_model_name} with CV RMSE: {scores[best_model_name]['CV_RMSE']:.4f}")
    
    return trained_models, scores, best_model_name


def tune_model(best_model_name, X, y):
    print(f"\nPerforming hyperparameter tuning for {best_model_name}...")
    
    grids = {
        "RF": {
            "n_estimators": [200, 300, 400],
            "max_depth": [10, 15, 20],
            "min_samples_split": [2, 5],
            "min_samples_leaf": [1, 2],
        },
        "ET": {
            "n_estimators": [200, 300, 400],
            "max_depth": [10, 15, 20],
            "min_samples_split": [2, 5],
        },
        "GB": {
            "n_estimators": [150, 200, 250],
            "learning_rate": [0.03, 0.05, 0.07],
            "max_depth": [4, 6, 8],
        },
        "XGB": {
            "n_estimators": [200, 300, 400],
            "learning_rate": [0.03, 0.05, 0.07],
            "max_depth": [4, 6, 8],
            "subsample": [0.8, 0.9],
        },
        "LGB": {
            "n_estimators": [200, 300, 400],
            "learning_rate": [0.03, 0.05, 0.07],
            "max_depth": [4, 6, 8],
            "subsample": [0.8, 0.9],
        },
        "Ridge": {"alpha": [0.1, 1, 10, 100]},
        "Lasso": {"alpha": [0.01, 0.1, 1, 10]},
        "ElasticNet": {"alpha": [0.1, 1, 10], "l1_ratio": [0.1, 0.5, 0.9]},
    }

    if best_model_name not in grids:
        print("No hyperparameter tuning defined for this model.")
        return None

    # Create base model
    base_models = {
        "RF": RandomForestRegressor(random_state=42, n_jobs=-1),
        "ET": ExtraTreesRegressor(random_state=42, n_jobs=-1),
        "GB": GradientBoostingRegressor(random_state=42),
        "XGB": xgb.XGBRegressor(random_state=42),
        "LGB": lgb.LGBMRegressor(random_state=42, verbose=-1),
        "Ridge": Ridge(),
        "Lasso": Lasso(),
        "ElasticNet": ElasticNet(),
    }
    
    base_model = base_models[best_model_name]
    
    grid = GridSearchCV(
        base_model, 
        grids[best_model_name], 
        cv=3, 
        scoring="neg_mean_squared_error", 
        n_jobs=-1,
        verbose=1
    )
    grid.fit(X, y)
    
    print(f"Best parameters: {grid.best_params_}")
    print(f"Best CV RMSE: {np.sqrt(-grid.best_score_):.4f}")
    
    return grid.best_estimator_

def create_ensemble_predictions(trained_models, scores, X_test):
    """Create ensemble predictions using weighted average based on CV performance"""
    print("\nCreating ensemble predictions...")
    
    # Get top 3 models based on CV RMSE
    sorted_models = sorted(scores.items(), key=lambda x: x[1]["CV_RMSE"])[:3]
    print(f"Top 3 models for ensemble: {[name for name, _ in sorted_models]}")
    
    ensemble_preds = None
    total_weight = 0
    
    for model_name, score_info in sorted_models:
        # Weight inversely proportional to RMSE (lower RMSE = higher weight)
        weight = 1.0 / score_info["CV_RMSE"]
        total_weight += weight
        
        # Average predictions from all folds
        fold_preds = []
        for fold_model in trained_models[model_name]:
            fold_preds.append(fold_model.predict(X_test))
        
        model_pred = np.mean(fold_preds, axis=0)
        
        if ensemble_preds is None:
            ensemble_preds = weight * model_pred
        else:
            ensemble_preds += weight * model_pred
        
        print(f"{model_name}: weight = {weight:.4f}")
    
    # Normalize weights
    ensemble_preds /= total_weight
    
    return ensemble_preds

train, test = load_data()
X_train, X_test, y_train = preprocess(train, test)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

# Train models with cross-validation
trained_models, scores, best_model_name = train_models(X_train, y_train)

# Tune best individual model
tuned_model = tune_model(best_model_name, X_train, y_train)

# Create ensemble predictions
ensemble_preds = create_ensemble_predictions(trained_models, scores, X_test)

# Also get tuned model predictions
if tuned_model is not None:
    tuned_preds = tuned_model.predict(X_test)
    print(f"\nTuned {best_model_name} predictions vs Ensemble:")
    print(f"Tuned model prediction range: {tuned_preds.min():.2f} to {tuned_preds.max():.2f}")
    print(f"Ensemble prediction range: {ensemble_preds.min():.2f} to {ensemble_preds.max():.2f}")
    
    # Use ensemble predictions (typically better)
    final_preds = ensemble_preds
else:
    final_preds = ensemble_preds

# Create submission
sub = pd.DataFrame({
    "LOCAL_IDENTIFIER": test["LOCAL_IDENTIFIER"],
    "CORRUCYSTIC_DENSITY": final_preds
})
sub.to_csv("submission.csv", index=False)

print("\n" + "="*50)
print("FINAL RESULTS:")
print("="*50)
for name, score_info in sorted(scores.items(), key=lambda x: x[1]["CV_RMSE"]):
    print(f"{name:12} - CV RMSE: {score_info['CV_RMSE']:.4f} (+/- {score_info['CV_STD']:.4f})")

print(f"\nSubmission saved: {sub.shape}")
print(f"Final prediction stats:")
print(f"  Mean: {final_preds.mean():.2f}")
print(f"  Std:  {final_preds.std():.2f}")
print(f"  Min:  {final_preds.min():.2f}")
print(f"  Max:  {final_preds.max():.2f}")

model, submission = tuned_model, sub

# Best Approach

In [ ]:
import os
import re
import warnings
from typing import Dict, List, Tuple, Optional
import numpy as np
import pandas as pd
from scipy import stats

# SKlearn Models
from sklearn.model_selection import StratifiedKFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


warnings.filterwarnings("ignore")

# Declaring Parameters to be used multiple times to avoid writing multiple times
SEED = 42
NFOLDS = 7
NREPEATS = 2
TARGET = "CORRUCYSTIC_DENSITY"
ID_COL = "LOCAL_IDENTIFIER"
AUTO_LOG_TARGET = True
SMOOTHING_M = 25.0
USE_FEATURE_SELECTION = True
SELECT_TOP_K_FEATURES = 100
SAVE_FEATURE_IMPORTANCES = True

OUTLIER_METHOD = "isolation_forest"
OUTLIER_CONTAMINATION = 0.05

def seed_everything(seed: int = SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def read_first_existing(paths: List[str]) -> Optional[str]:
    for p in paths:
        if os.path.exists(p):
            return p
    return None

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"[^A-Za-z0-9_]", "_", str(c)).strip("_") for c in df.columns]
    df.columns = [re.sub(r"_+", "_", c) for c in df.columns]
    return df

def advanced_downcast(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.select_dtypes(include=[np.number]).columns:
        col = df[c]
        c_min, c_max = col.min(), col.max()
        
        if pd.api.types.is_integer_dtype(col):
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[c] = col.astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[c] = col.astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[c] = col.astype(np.int32)
        else:
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[c] = col.astype(np.float32)
    return df

def detect_outliers_advanced(df: pd.DataFrame, target_col: str, method: str = "isolation_forest", contamination: float = 0.05) -> pd.Series:
    from sklearn.ensemble import IsolationForest
    from sklearn.neighbors import LocalOutlierFactor
    
    mask = pd.Series(True, index=df.index)
    
    if method == "iqr":
        Q1, Q3 = df[target_col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lb, ub = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        mask = (df[target_col] >= lb) & (df[target_col] <= ub)
    
    elif method == "zscore":
        z_scores = np.abs(stats.zscore(df[target_col]))
        mask = z_scores < 3
    
    elif method == "isolation_forest":
        iso_forest = IsolationForest(contamination=contamination, random_state=SEED)
        outliers = iso_forest.fit_predict(df[[target_col]])
        mask = outliers == 1
    
    elif method == "local_outlier":
        lof = LocalOutlierFactor(contamination=contamination)
        outliers = lof.fit_predict(df[[target_col]])
        mask = outliers == 1
    
    return mask

def make_target_bins_improved(y: pd.Series, n_bins: int = 10) -> pd.Series:
    try:
        bins = pd.qcut(y, q=n_bins, labels=False, duplicates="drop")
        if bins.nunique() < n_bins // 2:  # If too few unique bins
            raise ValueError("Too few unique bins")
        return bins.astype(int)
    except:
        bins = pd.cut(y, bins=n_bins, labels=False, include_lowest=True)
        return bins.astype(int)

def enhanced_target_encode(
    X: pd.DataFrame,
    y: pd.Series,
    X_test: pd.DataFrame,
    cat_cols: List[str],
    n_splits: int = NFOLDS,
    n_repeats: int = NREPEATS,
    smoothing_m: float = SMOOTHING_M,
    seed: int = SEED,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    #Enhanced target encoding with repeated CV and noise addition for robustness.
    X = X.copy()
    X_test = X_test.copy()
    global_mean = y.mean()
    
    # Use repeated K-fold for more stable encoding
    rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=seed)
    
    for col in cat_cols:
        oof_list = []
        test_encoded_list = []
        
        for tr_idx, val_idx in rkf.split(X):
            tr_y = y.iloc[tr_idx]
            tr_col = X.iloc[tr_idx][col].astype("object").fillna("__MISSING__")
            val_col = X.iloc[val_idx][col].astype("object").fillna("__MISSING__")
            
            # Calculate statistics with smoothing
            stats_df = tr_y.groupby(tr_col).agg(['sum', 'count', 'mean', 'std']).reset_index()
            stats_df.columns = [col, 'sum', 'count', 'mean', 'std']
            stats_df['enc'] = (stats_df['sum'] + smoothing_m * global_mean) / (stats_df['count'] + smoothing_m)
            
            # Add uncertainty-based noise
            stats_df['uncertainty'] = 1 / np.sqrt(stats_df['count'] + 1)
            noise_scale = stats_df['uncertainty'] * stats_df['std'].fillna(y.std())
            stats_df['enc'] += np.random.normal(0, noise_scale * 0.1, len(stats_df))
            
            # Create mapping dict
            encode_map = dict(zip(stats_df[col], stats_df['enc']))
            
            # Encode validation set
            oof_encoded = val_col.map(encode_map).fillna(global_mean)
            oof_list.append(pd.Series(oof_encoded.values, index=val_col.index))
            
            # Encode test set
            test_col = X_test[col].astype("object").fillna("__MISSING__")
            test_encoded = test_col.map(encode_map).fillna(global_mean)
            test_encoded_list.append(test_encoded.values)
        
        # Combine OOF predictions
        oof_combined = pd.concat(oof_list).groupby(level=0).mean()
        X[col + "__te"] = oof_combined.reindex(X.index).astype(np.float32)
        
        # Average test predictions
        X_test[col + "__te"] = np.mean(test_encoded_list, axis=0).astype(np.float32)
    
    return X, X_test

def create_advanced_features(df: pd.DataFrame, num_cols: List[str]) -> pd.DataFrame:
    """Create domain-specific features for the CORRUCYSTIC data."""
    df = df.copy()
    
    if len(num_cols) == 0:
        return df
    
    # Ensure we have numeric data
    num_data = df[num_cols].select_dtypes(include=[np.number])
    if num_data.empty:
        return df
    
    # Basic statistical features
    df["corr_density_mean"] = num_data.mean(axis=1)
    df["corr_density_std"] = num_data.std(axis=1)
    df["corr_density_skew"] = num_data.skew(axis=1)
    df["corr_density_kurt"] = num_data.kurtosis(axis=1)
    df["corr_density_range"] = num_data.max(axis=1) - num_data.min(axis=1)
    
    # Percentile features
    df["corr_density_q25"] = num_data.quantile(0.25, axis=1)
    df["corr_density_q75"] = num_data.quantile(0.75, axis=1)
    df["corr_density_iqr"] = df["corr_density_q75"] - df["corr_density_q25"]
    
    # Count-based features
    df["corr_positive_count"] = (num_data > 0).sum(axis=1)
    df["corr_negative_count"] = (num_data < 0).sum(axis=1)
    df["corr_zero_count"] = (num_data == 0).sum(axis=1)
    df["corr_missing_count"] = num_data.isnull().sum(axis=1)
    
    # Advanced statistical measures
    df["corr_cv"] = df["corr_density_std"] / (np.abs(df["corr_density_mean"]) + 1e-8)
    df["corr_mad"] = num_data.apply(manual_mad, axis=1)  # Using manual MAD implementation
    
    # Energy and signal processing inspired features (domain-specific)
    df["corr_energy"] = (num_data ** 2).sum(axis=1)
    df["corr_rms"] = np.sqrt(df["corr_energy"] / len(num_cols))
    
    # Handle infinite and NaN values
    inf_cols = df.select_dtypes(include=[np.number]).columns
    df[inf_cols] = df[inf_cols].replace([np.inf, -np.inf], np.nan)
    
    return df

def manual_mad(series):
    median = series.median()
    return np.median(np.abs(series - median))

def advanced_feature_selection(X: pd.DataFrame, y: pd.Series, k: int = 100) -> List[str]:
    if len(X.columns) <= k:
        return X.columns.tolist()
    
    valid_features = []
    for col in X.columns:
        if X[col].isnull().sum() / len(X) < 0.8 and X[col].nunique() > 1:
            valid_features.append(col)
    
    X_valid = X[valid_features].fillna(X[valid_features].median())
    
    # Combine multiple selection methods
    scores = {}
    
    # F-regression scores
    try:
        f_selector = SelectKBest(score_func=f_regression, k='all')
        f_selector.fit(X_valid, y)
        f_scores = dict(zip(valid_features, f_selector.scores_))
        scores['f_regression'] = f_scores
    except:
        pass
    
    # Mutual information scores
    try:
        mi_scores = mutual_info_regression(X_valid, y, random_state=SEED)
        mi_scores_dict = dict(zip(valid_features, mi_scores))
        scores['mutual_info'] = mi_scores_dict
    except:
        pass
    
    # Combine scores using rank aggregation
    if scores:
        feature_ranks = {}
        for feature in valid_features:
            ranks = []
            for method, score_dict in scores.items():
                if feature in score_dict:
                    # Convert to rank (lower rank = better)
                    sorted_features = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
                    rank = next(i for i, (f, _) in enumerate(sorted_features) if f == feature)
                    ranks.append(rank)
            feature_ranks[feature] = np.mean(ranks) if ranks else len(valid_features)
        
        # Select top k features
        selected = sorted(feature_ranks.items(), key=lambda x: x[1])[:k]
        return [f[0] for f in selected]
    
    return valid_features[:k]

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

# -------------------- Load -------------------- #
seed_everything(SEED)

train_path = "./MiNDAT.csv"
test_path = "./MiNDAT_UNK.csv"

train = pd.read_csv(train_path)
train = clean_columns(train)

test = pd.read_csv(test_path)
test = clean_columns(test)

# Basic hygiene
if TARGET not in train.columns:
    raise KeyError(f"Target column '{TARGET}' not in training data.")

# Drop rows with missing target
train = train.dropna(subset=[TARGET]).reset_index(drop=True)

# Remove outliers via IQR (robust)
Q1, Q3 = train[TARGET].quantile([0.25, 0.75])
IQR = Q3 - Q1
lb, ub = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
train = train[(train[TARGET] >= lb) & (train[TARGET] <= ub)].reset_index(drop=True)

# Identify columns (keep features simple: no extra row stats or poly features)
features = [c for c in train.columns if c not in [TARGET, ID_COL]]
X_full = train[features].copy()
y_full = train[TARGET].copy()

if test is not None:
    X_test_full = test[features].copy()
else:
    X_test_full = None

# Replace infs
for df in [X_full] + ([X_test_full] if X_test_full is not None else []):
    if df is None:
        continue
    numc = df.select_dtypes(include=[np.number]).columns
    df[numc] = df[numc].replace([np.inf, -np.inf], np.nan)

# Advanced memory optimization
X_full = advanced_downcast(X_full)
if X_test_full is not None:
    X_test_full = advanced_downcast(X_test_full)

# Split types
cat_cols = X_full.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X_full.select_dtypes(include=[np.number]).columns.tolist()

# Simple imputers (per column)
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

X_full[num_cols] = num_imputer.fit_transform(X_full[num_cols])
if X_test_full is not None:
    X_test_full[num_cols] = num_imputer.transform(X_test_full[num_cols])

X_full[cat_cols] = cat_imputer.fit_transform(X_full[cat_cols])
if X_test_full is not None:
    X_test_full[cat_cols] = cat_imputer.transform(X_test_full[cat_cols])

# Cast categoricals to 'category' dtype (memory);
for c in cat_cols:
    X_full[c] = X_full[c].astype("category")
    if X_test_full is not None:
        X_test_full[c] = X_test_full[c].astype("category")

if len(cat_cols) > 0:
    print("Performing enhanced target encoding...")
    X_te, X_test_te = enhanced_target_encode(
        X_full,
        y_full,
        X_test_full if X_test_full is not None else X_full.iloc[:0],
        cat_cols,
    )
else:
    X_te, X_test_te = X_full.copy(), (X_test_full.copy() if X_test_full is not None else None)

X_te = X_te.drop(columns=cat_cols, errors="ignore")
if X_test_te is not None:
    X_test_te = X_test_te.drop(columns=cat_cols, errors="ignore")

# Feature selection
if USE_FEATURE_SELECTION and len(X_te.columns) > SELECT_TOP_K_FEATURES:
    print(f"Selecting top {SELECT_TOP_K_FEATURES} features...")
    selected_features = advanced_feature_selection(X_te, y_full, SELECT_TOP_K_FEATURES)
    X_te = X_te[selected_features]
    if X_test_te is not None:
        X_test_te = X_test_te[selected_features]
    print(f"Selected {len(selected_features)} features")

# Enhanced scaling for linear models
scaler = QuantileTransformer(output_distribution='normal', random_state=SEED)
X_lin = X_te.copy()
lin_num_cols = X_lin.select_dtypes(include=[np.number]).columns
X_lin[lin_num_cols] = scaler.fit_transform(X_lin[lin_num_cols])

if X_test_te is not None:
    X_test_lin = X_test_te.copy()
    lin_test_num = X_test_lin.select_dtypes(include=[np.number]).columns
    X_test_lin[lin_test_num] = scaler.transform(X_test_lin[lin_test_num])
else:
    X_test_lin = None

use_log = False
if AUTO_LOG_TARGET:
    skew_val = pd.Series(y_full).skew()
    if skew_val > 1.0:
        use_log = True
        y_work = np.log1p(y_full)
    else:
        y_work = y_full.copy()
else:
    y_work = y_full.copy()

# Stratified folds by target bins (for stability)
strat_bins = make_target_bins_improved(y_full, n_bins=8)
skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
base_models: Dict[str, Tuple[object, str]] = {}

base_models["LGBM"] = (
    LGBMRegressor(
        n_estimators=2000,
        learning_rate=0.05,
        max_depth=8,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        min_child_samples=20,
        random_state=SEED,
        n_jobs=1,
        verbose=-1,
        objective='regression',  
        boosting_type='gbdt',  
    ),
    "tree",
)

base_models["XGB"] = (
    XGBRegressor(
        n_estimators=1200,
        learning_rate=0.03,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.0,
        reg_lambda=1.0,
        random_state=SEED,
        n_jobs=-1,
        tree_method="hist",
    ),
    "tree",
)

base_models["CAT"] = (
    CatBoostRegressor(
        depth=8,
        learning_rate=0.03,
        n_estimators=1500,
        subsample=0.8,
        random_state=SEED,
        loss_function="RMSE",
        verbose=False,
    ),
    "tree",
)

# Strong classical ensembles
base_models["ET"] = (
    ExtraTreesRegressor(
        n_estimators=1000, 
        max_depth=None,
        max_features="sqrt",
        random_state=SEED,
        n_jobs=-1,
    ),
    "tree",
)

base_models["RF"] = (
    RandomForestRegressor(
        n_estimators=1000, 
        max_depth=None,
        max_features="sqrt",
        random_state=SEED,
        n_jobs=-1,
    ),
    "tree",
)

base_models["GBDT"] = (
    GradientBoostingRegressor(
        n_estimators=800,
        learning_rate=0.03,
        max_depth=3,
        random_state=SEED,
    ),
    "tree",
)

base_models["Ridge"] = (
    Ridge(alpha=5.0, random_state=SEED),
    "linear",
)
base_models["Lasso"] = (
    Lasso(alpha=0.0005, random_state=SEED, max_iter=20000),
    "linear",
)

# -------------- OOF Training / Prediction -------------- #
models_oof = {}
models_test_pred = {}
models_fold_objs = {}
fold_scores = []

# Feature matrices: tree models use TE (numeric-only); linear models use scaled copy
X_tree = X_te
X_tree_test = X_test_te

X_linear = X_lin
X_linear_test = X_test_lin

for name, (model, mtype) in base_models.items():
    print(f"\n[Model={name}] training with {NFOLDS}-fold CV ...")
    oof = np.zeros(len(X_tree))
    test_preds_folds = []
    fold_importances = []

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_tree, strat_bins)):
        if mtype == "tree":
            Xtr, Xva = X_tree.iloc[tr_idx], X_tree.iloc[va_idx]
        else:
            Xtr, Xva = X_linear.iloc[tr_idx], X_linear.iloc[va_idx]

        ytr, yva = y_work.iloc[tr_idx], y_work.iloc[va_idx]

        # Fresh instance per fold
        m = type(model)(**model.get_params())

        # Special handling for different model types
        if name == "LGBM":
            # LightGBM specific handling with clean feature names and reset index
            Xtr_clean = Xtr.copy().reset_index(drop=True)
            Xva_clean = Xva.copy().reset_index(drop=True)
            ytr_clean = ytr.reset_index(drop=True)
            yva_clean = yva.reset_index(drop=True)
            
            # Ensure all data is numeric and clean feature names
            Xtr_clean = Xtr_clean.select_dtypes(include=[np.number])
            Xva_clean = Xva_clean.select_dtypes(include=[np.number])
            
            # Create simple sequential feature names to avoid any naming conflicts
            n_features = len(Xtr_clean.columns)
            simple_names = [f'feature_{i}' for i in range(n_features)]
            
            Xtr_clean.columns = simple_names
            Xva_clean.columns = simple_names
            
            try:
                # Use minimal parameters to avoid conflicts
                lgb_params = {
                    'n_estimators': 500,
                    'learning_rate': 0.1,
                    'max_depth': 6,
                    'num_leaves': 31,
                    'subsample': 0.8,
                    'colsample_bytree': 0.8,
                    'random_state': SEED,
                    'verbose': -1,
                    'n_jobs': 1
                }
                m = LGBMRegressor(**lgb_params)
                
                # Fit without eval_set to avoid feature name conflicts
                m.fit(Xtr_clean, ytr_clean)
                pred_va = m.predict(Xva_clean)
                
            except Exception as e:
                print(f"    Warning: LGBM fold {fold} failed: {e}")
                pred_va = np.full(len(yva_clean), ytr_clean.mean())  # Fallback prediction
                
        elif name == "CAT":
            m.fit(Xtr.astype(np.float32), ytr, eval_set=(Xva.astype(np.float32), yva), verbose=False)
            pred_va = m.predict(Xva.astype(np.float32))
        else:
            m.fit(Xtr, ytr)
            pred_va = m.predict(Xva)

        oof[va_idx] = pred_va

        if X_tree_test is not None:
            Xte = X_tree_test if mtype == "tree" else X_linear_test
            if name == "LGBM":
                Xte_clean = Xte.copy().reset_index(drop=True)
                Xte_clean = Xte_clean.select_dtypes(include=[np.number])
                
                n_features = len(Xte_clean.columns)
                simple_names = [f'feature_{i}' for i in range(n_features)]
                Xte_clean.columns = simple_names
                
                try:
                    test_preds_folds.append(m.predict(Xte_clean))
                except Exception as e:
                    print(f"    Warning: LGBM test prediction failed: {e}")
                    test_preds_folds.append(np.full(len(Xte_clean), oof[oof != 0].mean() if len(oof[oof != 0]) > 0 else 0))
            elif name == "CAT":
                test_preds_folds.append(m.predict(Xte.astype(np.float32)))
            else:
                test_preds_folds.append(m.predict(Xte))

        # Save feature importances when available
        if SAVE_FEATURE_IMPORTANCES and hasattr(m, "feature_importances_"):
            fi = pd.DataFrame({
                "feature": Xtr.columns,
                "importance": m.feature_importances_,
                "fold": fold,
                "model": name,
            })
            fold_importances.append(fi)

        models_fold_objs.setdefault(name, []).append(m)

    # Back-transform if log was used
    oof_final = np.expm1(oof) if use_log else oof
    rmse_score = rmse(y_full, oof_final)

    models_oof[name] = oof_final
    if test_preds_folds:
        pred_test_mean = np.mean(test_preds_folds, axis=0)
        pred_test_final = np.expm1(pred_test_mean) if use_log else pred_test_mean
        models_test_pred[name] = pred_test_final

    fold_scores.append({"model": name, "cv_rmse": rmse_score})
    print(f"[Model={name}] CV RMSE: {rmse_score:.5f}")

    if SAVE_FEATURE_IMPORTANCES and len(fold_importances) > 0:
        imp_df = pd.concat(fold_importances, ignore_index=True)
        imp_df.groupby("feature")["importance"].mean().sort_values(ascending=False).head(50).to_csv(
            f"feature_importances_{name}.csv", index=True
        )

# -------------------- Stacking (meta-model) -------------------- #
print("\n[Stacking] Training meta-model (GradientBoostingRegressor) on OOF predictions ...")
oof_df = pd.DataFrame(models_oof)

meta_model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=SEED,
)
meta_model.fit(oof_df, y_full)
meta_oof = meta_model.predict(oof_df)
stack_rmse = rmse(y_full, meta_oof)
print(f"[Stacking] Meta CV RMSE: {stack_rmse:.5f}")

# Weighted Blend of the best individual models (by CV)
sorted_models = sorted(fold_scores, key=lambda d: d["cv_rmse"])[: min(3, len(fold_scores))]
weights = []
for d in sorted_models:
    w = 1.0 / max(d["cv_rmse"], 1e-6)
    weights.append(w)
weights = np.array(weights) / np.sum(weights)

print("\n[Blending] Top models & weights:")
for (d, w) in zip(sorted_models, weights):
    print(f"  - {d['model']}: weight={w:.3f}, cv_rmse={d['cv_rmse']:.5f}")

pd.DataFrame({"id": train[ID_COL], "y": y_full, **models_oof}).to_csv("oof_predictions.csv", index=False)
pd.DataFrame(fold_scores).to_csv("fold_scores.csv", index=False)

if X_test_full is not None and len(models_test_pred) > 0:
    test_stack_mat = pd.DataFrame(models_test_pred)
    stack_pred = meta_model.predict(test_stack_mat)

    blend_pred = np.zeros(len(test_stack_mat))
    for (d, w) in zip(sorted_models, weights):
        blend_pred += w * test_stack_mat[d["model"]].values

    final_pred = 0.5 * stack_pred + 0.5 * blend_pred

    sub = pd.DataFrame({ID_COL: test[ID_COL], TARGET: final_pred})
    sub.to_csv("submission.csv", index=False)
    print("\nSaved submission.csv")


# Approach 5

In [ ]:
import os
import re
import warnings
from typing import Dict, List, Tuple, Optional
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import StratifiedKFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import QuantileTransformer, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.cluster import KMeans

try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

try:
    from lightgbm import LGBMRegressor
    HAS_LGB = True
except Exception:
    HAS_LGB = False

try:
    from catboost import CatBoostRegressor
    HAS_CAT = True
except Exception:
    HAS_CAT = False

warnings.filterwarnings("ignore")

SEED = 42
NFOLDS = 7
NREPEATS = 2
TARGET = "CORRUCYSTIC_DENSITY"
ID_COL = "LOCAL_IDENTIFIER"
AUTO_LOG_TARGET = True
SMOOTHING_M = 25.0
USE_FEATURE_SELECTION = True
SELECT_TOP_K_FEATURES = 120
SAVE_FEATURE_IMPORTANCES = True

# Enhanced outlier detection
OUTLIER_METHOD = "iqr"  
OUTLIER_CONTAMINATION = 0.05  

# Feature engineering toggles
USE_ADVANCED_FEATURES = True
USE_INTERACTIONS = True
USE_POLYNOMIAL = True
USE_CLUSTERING = True

def seed_everything(seed: int = SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def read_first_existing(paths: List[str]) -> Optional[str]:
    for p in paths:
        if os.path.exists(p):
            return p
    return None

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"[^A-Za-z0-9_]", "_", str(c)).strip("_") for c in df.columns]
    df.columns = [re.sub(r"_+", "_", c) for c in df.columns]
    return df

def advanced_downcast(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.select_dtypes(include=[np.number]).columns:
        col = df[c]
        c_min, c_max = col.min(), col.max()
        
        if pd.api.types.is_integer_dtype(col):
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[c] = col.astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[c] = col.astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[c] = col.astype(np.int32)
        else:
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[c] = col.astype(np.float32)
    return df

def detect_outliers_advanced(df: pd.DataFrame, target_col: str, method: str = "isolation_forest", contamination: float = 0.05) -> pd.Series:
    from sklearn.ensemble import IsolationForest
    from sklearn.neighbors import LocalOutlierFactor
    
    mask = pd.Series(True, index=df.index)
    
    if method == "iqr":
        Q1, Q3 = df[target_col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lb, ub = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        mask = (df[target_col] >= lb) & (df[target_col] <= ub)
    
    elif method == "zscore":
        z_scores = np.abs(stats.zscore(df[target_col]))
        mask = z_scores < 3
    
    elif method == "isolation_forest":
        iso_forest = IsolationForest(contamination=contamination, random_state=SEED)
        outliers = iso_forest.fit_predict(df[[target_col]])
        mask = outliers == 1
    
    elif method == "local_outlier":
        lof = LocalOutlierFactor(contamination=contamination)
        outliers = lof.fit_predict(df[[target_col]])
        mask = outliers == 1
    
    return mask

def make_target_bins_improved(y: pd.Series, n_bins: int = 10) -> pd.Series:
    # Try quantile-based binning first
    try:
        bins = pd.qcut(y, q=n_bins, labels=False, duplicates="drop")
        if bins.nunique() < n_bins // 2:
            raise ValueError("Too few unique bins")
        return bins.astype(int)
    except:
        bins = pd.cut(y, bins=n_bins, labels=False, include_lowest=True)
        return bins.astype(int)

def enhanced_target_encode(
    X: pd.DataFrame,
    y: pd.Series,
    X_test: pd.DataFrame,
    cat_cols: List[str],
    n_splits: int = NFOLDS,
    n_repeats: int = NREPEATS,
    smoothing_m: float = SMOOTHING_M,
    seed: int = SEED,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    X = X.copy()
    X_test = X_test.copy()
    global_mean = y.mean()
    
    # Use repeated K-fold for more stable encoding
    rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=seed)
    
    for col in cat_cols:
        oof_list = []
        test_encoded_list = []
        
        for tr_idx, val_idx in rkf.split(X):
            tr_y = y.iloc[tr_idx]
            tr_col = X.iloc[tr_idx][col].astype("object").fillna("__MISSING__")
            val_col = X.iloc[val_idx][col].astype("object").fillna("__MISSING__")
            
            # Calculate statistics with smoothing
            stats_df = tr_y.groupby(tr_col).agg(['sum', 'count', 'mean', 'std']).reset_index()
            stats_df.columns = [col, 'sum', 'count', 'mean', 'std']
            stats_df['enc'] = (stats_df['sum'] + smoothing_m * global_mean) / (stats_df['count'] + smoothing_m)
            
            # Add uncertainty-based noise
            stats_df['uncertainty'] = 1 / np.sqrt(stats_df['count'] + 1)
            noise_scale = stats_df['uncertainty'] * stats_df['std'].fillna(y.std())
            stats_df['enc'] += np.random.normal(0, noise_scale * 0.1, len(stats_df))
            
            # Create mapping dict
            encode_map = dict(zip(stats_df[col], stats_df['enc']))
            
            # Encode validation set
            oof_encoded = val_col.map(encode_map).fillna(global_mean)
            oof_list.append(pd.Series(oof_encoded.values, index=val_col.index))
            
            # Encode test set
            test_col = X_test[col].astype("object").fillna("__MISSING__")
            test_encoded = test_col.map(encode_map).fillna(global_mean)
            test_encoded_list.append(test_encoded.values)
        
        # Combine OOF predictions
        oof_combined = pd.concat(oof_list).groupby(level=0).mean()
        X[col + "__te"] = oof_combined.reindex(X.index).astype(np.float32)
        
        # Average test predictions
        X_test[col + "__te"] = np.mean(test_encoded_list, axis=0).astype(np.float32)
    
    return X, X_test

def create_advanced_features(df: pd.DataFrame, num_cols: List[str]) -> pd.DataFrame:
    
    if len(num_cols) == 0:
        return df
    
    # Ensure we have numeric data
    num_data = df[num_cols].select_dtypes(include=[np.number])
    if num_data.empty:
        return df
    
    # Basic statistical features
    df["corr_density_mean"] = num_data.mean(axis=1)
    df["corr_density_std"] = num_data.std(axis=1)
    df["corr_density_skew"] = num_data.skew(axis=1)
    df["corr_density_kurt"] = num_data.kurtosis(axis=1)
    df["corr_density_range"] = num_data.max(axis=1) - num_data.min(axis=1)
    
    # Percentile features
    df["corr_density_q25"] = num_data.quantile(0.25, axis=1)
    df["corr_density_q75"] = num_data.quantile(0.75, axis=1)
    df["corr_density_iqr"] = df["corr_density_q75"] - df["corr_density_q25"]
    
    # Count-based features
    df["corr_positive_count"] = (num_data > 0).sum(axis=1)
    df["corr_negative_count"] = (num_data < 0).sum(axis=1)
    df["corr_zero_count"] = (num_data == 0).sum(axis=1)
    df["corr_missing_count"] = num_data.isnull().sum(axis=1)
    
    # Advanced statistical measures
    df["corr_cv"] = df["corr_density_std"] / (np.abs(df["corr_density_mean"]) + 1e-8)
    df["corr_mad"] = num_data.apply(manual_mad, axis=1)  # Using manual MAD implementation
    
    # Energy and signal processing inspired features (domain-specific)
    df["corr_energy"] = (num_data ** 2).sum(axis=1)
    df["corr_rms"] = np.sqrt(df["corr_energy"] / len(num_cols))
    
    # Data integrity and stability measures (alien biocomputer context)
    df["corr_stability"] = 1 / (1 + df["corr_cv"])  # Inverse CV as stability measure
    df["corr_coherence"] = df["corr_positive_count"] / (df["corr_positive_count"] + df["corr_negative_count"] + 1e-8)
    df["corr_completeness"] = 1 - (df["corr_missing_count"] / len(num_cols))
    df["corr_balance"] = 1 - np.abs(df["corr_positive_count"] - df["corr_negative_count"]) / len(num_cols)
    
    # Memory/Neural network inspired features
    df["corr_activation"] = num_data.apply(lambda x: (x > x.mean()).sum() / len(x), axis=1)  # Fraction above mean
    df["corr_inhibition"] = num_data.apply(lambda x: (x < x.mean()).sum() / len(x), axis=1)  # Fraction below mean
    df["corr_signal_noise"] = np.abs(df["corr_density_mean"]) / (df["corr_density_std"] + 1e-8)
    
    # Frequency domain features (alien signal analysis)
    df["corr_peak_to_avg"] = num_data.max(axis=1) / (np.abs(df["corr_density_mean"]) + 1e-8)
    df["corr_trough_to_avg"] = num_data.min(axis=1) / (np.abs(df["corr_density_mean"]) + 1e-8)
    
    # Handle infinite and NaN values
    inf_cols = df.select_dtypes(include=[np.number]).columns
    df[inf_cols] = df[inf_cols].replace([np.inf, -np.inf], np.nan)
    
    return df

def create_biocomputer_interactions(df: pd.DataFrame, df_test: Optional[pd.DataFrame], important_cols: List[str]) -> Tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    df = df.copy()
    df_test = df_test.copy() if df_test is not None else None
    
    # Limit to available columns
    available_cols = [col for col in important_cols if col in df.columns][:5]  # Top 5 to avoid explosion
    
    if len(available_cols) < 2:
        return df, df_test
    
    # Create pairwise interactions
    for i, col1 in enumerate(available_cols):
        for j, col2 in enumerate(available_cols[i+1:], i+1):
            # Multiplicative interactions (signal amplification)
            interaction_name = f"interaction_{col1}_x_{col2}"
            df[interaction_name] = df[col1] * df[col2]
            if df_test is not None:
                df_test[interaction_name] = df_test[col1] * df_test[col2]
            
            # Ratio interactions (relative signal strength)
            ratio_name = f"ratio_{col1}_div_{col2}"
            df[ratio_name] = df[col1] / (np.abs(df[col2]) + 1e-8)
            if df_test is not None:
                df_test[ratio_name] = df_test[col1] / (np.abs(df_test[col2]) + 1e-8)
    
    # Clean up infinite values
    for dataset in [df] + ([df_test] if df_test is not None else []):
        if dataset is None:
            continue
        numeric_cols = dataset.select_dtypes(include=[np.number]).columns
        dataset[numeric_cols] = dataset[numeric_cols].replace([np.inf, -np.inf], np.nan)
    
    return df, df_test

def create_memory_sequence_features(df: pd.DataFrame, df_test: Optional[pd.DataFrame], num_cols: List[str]) -> Tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    df = df.copy()
    df_test = df_test.copy() if df_test is not None else None
    
    if len(num_cols) == 0:
        return df, df_test
    
    # Get numeric data
    for dataset, name in [(df, "train"), (df_test, "test")]:
        if dataset is None:
            continue
            
        num_data = dataset[num_cols].select_dtypes(include=[np.number])
        if num_data.empty:
            continue
        
        # Memory sequence patterns (treating columns as temporal sequence)
        dataset["memory_trend"] = num_data.apply(lambda x: np.corrcoef(x.values, np.arange(len(x)))[0,1] if len(x) > 1 else 0, axis=1)
        dataset["memory_volatility"] = num_data.rolling(window=3, axis=1).std().mean(axis=1)
        dataset["memory_persistence"] = num_data.apply(lambda x: np.sum(np.diff(x) > 0) / (len(x) - 1) if len(x) > 1 else 0, axis=1)
        
        # Alien signal patterns
        dataset["signal_periodicity"] = num_data.apply(
            lambda x: np.abs(np.fft.fft(x.fillna(0).values)).mean() if len(x) > 2 else 0, axis=1
        )
        dataset["signal_complexity"] = num_data.apply(
            lambda x: -np.sum(x / x.sum() * np.log(x / x.sum() + 1e-8)) if x.sum() != 0 else 0, axis=1
        )
        
        # Memory fragmentation patterns
        dataset["memory_fragmentation"] = (num_data.diff(axis=1) != 0).sum(axis=1) / len(num_cols)
        dataset["memory_coherence_seq"] = num_data.apply(
            lambda x: np.corrcoef(x[:-1].values, x[1:].values)[0,1] if len(x) > 2 else 0, axis=1
        )
    
    # Clean up infinite and NaN values
    for dataset in [df] + ([df_test] if df_test is not None else []):
        if dataset is None:
            continue
        numeric_cols = dataset.select_dtypes(include=[np.number]).columns
        dataset[numeric_cols] = dataset[numeric_cols].replace([np.inf, -np.inf], np.nan)
        dataset[numeric_cols] = dataset[numeric_cols].fillna(0)
    
    return df, df_test

def create_network_topology_features(df: pd.DataFrame, df_test: Optional[pd.DataFrame], num_cols: List[str]) -> Tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    df = df.copy()
    df_test = df_test.copy() if df_test is not None else None
    
    if len(num_cols) == 0:
        return df, df_test
    
    # Create network-inspired features
    for dataset, name in [(df, "train"), (df_test, "test")]:
        if dataset is None:
            continue
            
        num_data = dataset[num_cols].select_dtypes(include=[np.number])
        if num_data.empty:
            continue
        
        # Network connectivity patterns
        dataset["network_density"] = (num_data != 0).sum(axis=1) / len(num_cols)
        # Simplified clustering coefficient using autocorrelation
        dataset["network_clustering"] = num_data.apply(
            lambda x: np.corrcoef(x.values[:-1], x.values[1:])[0,1] if len(x) > 2 and x.std() > 0 else 0, axis=1
        )
        
        # Information flow patterns
        dataset["info_entropy"] = num_data.apply(
            lambda x: -np.sum((x / (x.sum() + 1e-8)) * np.log(x / (x.sum() + 1e-8) + 1e-8)), axis=1
        )
        dataset["info_concentration"] = num_data.apply(
            lambda x: np.max(np.abs(x)) / (np.sum(np.abs(x)) + 1e-8), axis=1
        )
        
        # Distributed processing patterns
        dataset["processing_load"] = np.sqrt((num_data ** 2).sum(axis=1))
        dataset["processing_efficiency"] = num_data.apply(
            lambda x: np.sum(x > x.mean()) / len(x), axis=1
        )
        
        # Network resilience features
        dataset["network_robustness"] = 1 - (num_data == 0).sum(axis=1) / len(num_cols)
        dataset["network_redundancy"] = num_data.apply(
            lambda x: len(x[x > x.quantile(0.5)]) / len(x), axis=1
        )
    
    # Clean up infinite and NaN values
    for dataset in [df] + ([df_test] if df_test is not None else []):
        if dataset is None:
            continue
        numeric_cols = dataset.select_dtypes(include=[np.number]).columns
        dataset[numeric_cols] = dataset[numeric_cols].replace([np.inf, -np.inf], np.nan)
        dataset[numeric_cols] = dataset[numeric_cols].fillna(0)
    
    return df, df_test

def manual_mad(series):
    median = series.median()
    return np.median(np.abs(series - median))

def advanced_feature_selection(X: pd.DataFrame, y: pd.Series, k: int = 100) -> List[str]:
    if len(X.columns) <= k:
        return X.columns.tolist()
    
    # Remove features with too many missing values or zero variance
    valid_features = []
    for col in X.columns:
        if X[col].isnull().sum() / len(X) < 0.8 and X[col].nunique() > 1:
            valid_features.append(col)
    
    X_valid = X[valid_features].fillna(X[valid_features].median())
    
    # Combine multiple selection methods
    scores = {}
    
    # F-regression scores
    try:
        f_selector = SelectKBest(score_func=f_regression, k='all')
        f_selector.fit(X_valid, y)
        f_scores = dict(zip(valid_features, f_selector.scores_))
        scores['f_regression'] = f_scores
    except:
        pass
    
    # Mutual information scores
    try:
        mi_scores = mutual_info_regression(X_valid, y, random_state=SEED)
        mi_scores_dict = dict(zip(valid_features, mi_scores))
        scores['mutual_info'] = mi_scores_dict
    except:
        pass
    
    # Combine scores using rank aggregation
    if scores:
        feature_ranks = {}
        for feature in valid_features:
            ranks = []
            for method, score_dict in scores.items():
                if feature in score_dict:
                    # Convert to rank (lower rank = better)
                    sorted_features = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
                    rank = next(i for i, (f, _) in enumerate(sorted_features) if f == feature)
                    ranks.append(rank)
            feature_ranks[feature] = np.mean(ranks) if ranks else len(valid_features)
        
        # Select top k features
        selected = sorted(feature_ranks.items(), key=lambda x: x[1])[:k]
        return [f[0] for f in selected]
    
    return valid_features[:k]

def enhanced_feature_selection(X: pd.DataFrame, y: pd.Series, k: int = 100) -> List[str]:
    if len(X.columns) <= k:
        return X.columns.tolist()
    
    # Enhanced validation with stability check
    valid_features = []
    for col in X.columns:
        missing_rate = X[col].isnull().sum() / len(X)
        unique_count = X[col].nunique()
        
        if missing_rate < 0.8 and unique_count > 1:
            if unique_count > 2:
                try:
                    col_std = X[col].std()
                    col_mean = X[col].mean()
                    if col_std > 0 and not np.isnan(col_std) and not np.isinf(col_std):
                        cv = np.abs(col_std / (col_mean + 1e-8))
                        if cv < 100:
                            valid_features.append(col)
                except:
                    valid_features.append(col)
            else:
                valid_features.append(col)
    
    if len(valid_features) <= k:
        return valid_features
    
    X_valid = X[valid_features].copy()
    
    # Enhanced imputation for selection
    for col in X_valid.columns:
        if X_valid[col].dtype in [np.number]:
            X_valid[col] = X_valid[col].fillna(X_valid[col].median())
        else:
            X_valid[col] = X_valid[col].fillna(X_valid[col].mode().iloc[0] if len(X_valid[col].mode()) > 0 else 0)
    
    scores = {}
    try:
        f_selector = SelectKBest(score_func=f_regression, k='all')
        f_selector.fit(X_valid, y)
        f_scores = dict(zip(valid_features, f_selector.scores_))
        # Normalize scores
        max_score = max(f_scores.values())
        f_scores = {k: v/max_score for k, v in f_scores.items()}
        scores['f_regression'] = f_scores
    except Exception as e:
        print(f"F-regression failed: {e}")
    
    # Mutual information scores (non-linear relationships)
    try:
        mi_scores = mutual_info_regression(X_valid, y, random_state=SEED, n_neighbors=5)
        mi_scores_dict = dict(zip(valid_features, mi_scores))

        max_score = max(mi_scores_dict.values()) if max(mi_scores_dict.values()) > 0 else 1
        mi_scores_dict = {k: v/max_score for k, v in mi_scores_dict.items()}
        scores['mutual_info'] = mi_scores_dict
    except Exception as e:
        print(f"Mutual information failed: {e}")
    
    try:
        corr_scores = {}
        for col in valid_features:
            corr = np.abs(np.corrcoef(X_valid[col], y)[0, 1])
            corr_scores[col] = corr if not np.isnan(corr) else 0
        # Normalize scores
        max_score = max(corr_scores.values()) if max(corr_scores.values()) > 0 else 1
        corr_scores = {k: v/max_score for k, v in corr_scores.items()}
        scores['correlation'] = corr_scores
    except Exception as e:
        print(f"Correlation scoring failed: {e}")
    
    # Enhanced rank aggregation with weighted voting
    if scores:
        feature_ranks = {}
        weights = {'f_regression': 0.4, 'mutual_info': 0.4, 'correlation': 0.2}  # Prioritize advanced methods
        
        for feature in valid_features:
            weighted_score = 0
            total_weight = 0
            
            for method, score_dict in scores.items():
                if feature in score_dict and method in weights:
                    weighted_score += score_dict[feature] * weights[method]
                    total_weight += weights[method]
            
            feature_ranks[feature] = weighted_score / total_weight if total_weight > 0 else 0
        
        # Select top k features based on weighted scores
        selected = sorted(feature_ranks.items(), key=lambda x: x[1], reverse=True)[:k]
        return [f[0] for f in selected]
    
    # Fallback to simple variance-based selection
    print("Using variance-based feature selection as fallback")
    try:
        variances = X_valid.var()
        top_var_features = variances.nlargest(k).index.tolist()
        return top_var_features
    except:
        return valid_features[:k]

def create_polynomial_features(df: pd.DataFrame, df_test: Optional[pd.DataFrame], top_n: int = 5) -> Tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    df = df.copy()
    df_test = df_test.copy() if df_test is not None else None
    
    # Select top numeric features by variance (simple heuristic)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) == 0:
        return df, df_test
    
    # Get top features by variance (proxy for informativeness)
    try:
        variances = df[numeric_cols].var()
        top_features = variances.nlargest(min(top_n, len(numeric_cols))).index.tolist()
        poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
        
        df_poly = df[top_features].fillna(0)
        poly_features = poly.fit_transform(df_poly)
        
        feature_names = poly.get_feature_names_out(top_features)
        
        interaction_names = [name for name in feature_names if ' ' in name]  # Has interaction
        interaction_indices = [i for i, name in enumerate(feature_names) if ' ' in name]
        
        if len(interaction_indices) > 0:
            poly_df = pd.DataFrame(
                poly_features[:, interaction_indices], 
                columns=[f"poly_{name.replace(' ', '_')}" for name in interaction_names],
                index=df.index
            )
            df = pd.concat([df, poly_df], axis=1)
            
            if df_test is not None:
                df_test_poly = df_test[top_features].fillna(0)
                poly_features_test = poly.transform(df_test_poly)
                poly_df_test = pd.DataFrame(
                    poly_features_test[:, interaction_indices],
                    columns=[f"poly_{name.replace(' ', '_')}" for name in interaction_names],
                    index=df_test.index
                )
                df_test = pd.concat([df_test, poly_df_test], axis=1)
        
    except Exception as e:
        print(f"Polynomial feature creation failed: {e}")
    
    return df, df_test

def create_cluster_features(df: pd.DataFrame, df_test: Optional[pd.DataFrame], n_clusters: int = 8) -> Tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    df = df.copy()
    df_test = df_test.copy() if df_test is not None else None
    
    # Get numeric features for clustering
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) < 2:
        return df, df_test
    
    try:
        cluster_data = df[numeric_cols].fillna(df[numeric_cols].median())
        scaler = StandardScaler()
        cluster_data_scaled = scaler.fit_transform(cluster_data)
        kmeans = KMeans(n_clusters=n_clusters, random_state=SEED, n_init=10)
        clusters = kmeans.fit_predict(cluster_data_scaled)
        
        df['biocomputer_cluster'] = clusters
        
        cluster_distances = kmeans.transform(cluster_data_scaled)
        for i in range(n_clusters):
            df[f'distance_to_cluster_{i}'] = cluster_distances[:, i]
        
        df['min_cluster_distance'] = cluster_distances.min(axis=1)
        
        if df_test is not None:
            test_cluster_data = df_test[numeric_cols].fillna(df[numeric_cols].median())
            test_cluster_data_scaled = scaler.transform(test_cluster_data)
            
            test_clusters = kmeans.predict(test_cluster_data_scaled)
            df_test['biocomputer_cluster'] = test_clusters
            
            test_cluster_distances = kmeans.transform(test_cluster_data_scaled)
            for i in range(n_clusters):
                df_test[f'distance_to_cluster_{i}'] = test_cluster_distances[:, i]
            
            df_test['min_cluster_distance'] = test_cluster_distances.min(axis=1)
        
    except Exception as e:
        print(f"Cluster feature creation failed: {e}")
    
    return df, df_test

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

# -------------------- Load -------------------- #
seed_everything(SEED)

train_path = "./MiNDAT.csv"
test_path = "./MiNDAT_UNK.csv"

train = pd.read_csv(train_path)
train = clean_columns(train)

# Basic hygiene
if TARGET not in train.columns:
    raise KeyError(f"Target column '{TARGET}' not in training data.")

# Drop rows with missing target
train = train.dropna(subset=[TARGET]).reset_index(drop=True)

# Remove outliers via IQR (robust)
Q1, Q3 = train[TARGET].quantile([0.25, 0.75])
IQR = Q3 - Q1
lb, ub = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
train = train[(train[TARGET] >= lb) & (train[TARGET] <= ub)].reset_index(drop=True)

# Identify columns (keep features simple: no extra row stats or poly features)
features = [c for c in train.columns if c not in [TARGET, ID_COL]]
X_full = train[features].copy()
y_full = train[TARGET].copy()

if test is not None:
    X_test_full = test[features].copy()
else:
    X_test_full = None

# Replace infs
for df in [X_full] + ([X_test_full] if X_test_full is not None else []):
    if df is None:
        continue
    numc = df.select_dtypes(include=[np.number]).columns
    df[numc] = df[numc].replace([np.inf, -np.inf], np.nan)

# Advanced memory optimization
X_full = advanced_downcast(X_full)
if X_test_full is not None:
    X_test_full = advanced_downcast(X_test_full)

# Split types
cat_cols = X_full.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X_full.select_dtypes(include=[np.number]).columns.tolist()

# Simple imputers (per column)
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

X_full[num_cols] = num_imputer.fit_transform(X_full[num_cols])
if X_test_full is not None:
    X_test_full[num_cols] = num_imputer.transform(X_test_full[num_cols])

X_full[cat_cols] = cat_imputer.fit_transform(X_full[cat_cols])
if X_test_full is not None:
    X_test_full[cat_cols] = cat_imputer.transform(X_test_full[cat_cols])

# Cast categoricals to 'category' dtype (memory);
for c in cat_cols:
    X_full[c] = X_full[c].astype("category")
    if X_test_full is not None:
        X_test_full[c] = X_test_full[c].astype("category")

# -------------------- Advanced Feature Engineering -------------------- #
print("\nADVANCED FEATURE ENGINEERING FOR CORRUCYST FRAGMENTS")
print("-" * 60)

if USE_ADVANCED_FEATURES:
    print("Creating domain-specific biocomputer features...")
    X_fe = create_advanced_features(X_full.copy(), num_cols)
    if X_test_full is not None:
        X_test_fe = create_advanced_features(X_test_full.copy(), num_cols)
    else:
        X_test_fe = None
    print(f"✓ Feature engineering complete. Added {X_fe.shape[1] - X_full.shape[1]} new features")
else:
    X_fe, X_test_fe = X_full.copy(), (X_test_full.copy() if X_test_full is not None else None)

# Create interaction features for most important original features
if USE_INTERACTIONS:
    print("Creating interaction features for alien data patterns...")
    X_fe, X_test_fe = create_biocomputer_interactions(X_fe, X_test_fe, num_cols[:10])  # Top 10 original features

# Create temporal/sequence features (alien memory simulation context)
if USE_ADVANCED_FEATURES:
    print("Creating memory sequence simulation features...")
    X_fe, X_test_fe = create_memory_sequence_features(X_fe, X_test_fe, num_cols)

# Create network topology features (distributed biocomputer network context)
if USE_ADVANCED_FEATURES:
    print("Creating biocomputer network topology features...")
    X_fe, X_test_fe = create_network_topology_features(X_fe, X_test_fe, num_cols)

print(f"✓ All feature engineering complete. Final shape: {X_fe.shape}")

# KFold Target Encoding (leakage-safe)
if len(cat_cols) > 0:
    print("Performing enhanced target encoding...")
    X_te, X_test_te = enhanced_target_encode(
        X_fe,
        y_full,
        X_test_fe if X_test_fe is not None else X_fe.iloc[:0],
        cat_cols,
    )
else:
    X_te, X_test_te = X_fe.copy(), (X_test_fe.copy() if X_test_fe is not None else None)

X_te = X_te.drop(columns=cat_cols, errors="ignore")
if X_test_te is not None:
    X_test_te = X_test_te.drop(columns=cat_cols, errors="ignore")

# Feature selection with enhanced strategy
if USE_FEATURE_SELECTION and len(X_te.columns) > SELECT_TOP_K_FEATURES:
    print(f"Selecting top {SELECT_TOP_K_FEATURES} features from {len(X_te.columns)} candidates...")
    selected_features = enhanced_feature_selection(X_te, y_full, SELECT_TOP_K_FEATURES)
    X_te = X_te[selected_features]
    if X_test_te is not None:
        X_test_te = X_test_te[selected_features]
    print(f"✓ Selected {len(selected_features)} most informative features")

if USE_POLYNOMIAL:
    print("Creating polynomial combinations for top predictive features...")
    X_te, X_test_te = create_polynomial_features(X_te, X_test_te, top_n=5)

if USE_CLUSTERING:
    print("Creating clustering-based biocomputer network features...")
    X_te, X_test_te = create_cluster_features(X_te, X_test_te, n_clusters=8)

print(f"✓ Final feature matrix ready: {X_te.shape[1]} features for {len(X_te)} CORRUCYST fragments")

scaler = QuantileTransformer(output_distribution='normal', random_state=SEED)
X_lin = X_te.copy()
lin_num_cols = X_lin.select_dtypes(include=[np.number]).columns
X_lin[lin_num_cols] = scaler.fit_transform(X_lin[lin_num_cols])

if X_test_te is not None:
    X_test_lin = X_test_te.copy()
    lin_test_num = X_test_lin.select_dtypes(include=[np.number]).columns
    X_test_lin[lin_test_num] = scaler.transform(X_test_lin[lin_test_num])
else:
    X_test_lin = None

use_log = False
if AUTO_LOG_TARGET:
    skew_val = pd.Series(y_full).skew()
    if skew_val > 1.0:
        use_log = True
        y_work = np.log1p(y_full)
    else:
        y_work = y_full.copy()
else:
    y_work = y_full.copy()

# Stratified folds by target bins (for stability)
strat_bins = make_target_bins_improved(y_full, n_bins=8)
skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)# -------------------- Base Models -------------------- #
base_models: Dict[str, Tuple[object, str]] = {}

if HAS_LGB:
    base_models["LGBM"] = (
        LGBMRegressor(
            n_estimators=2000,  # Increased from 800
            learning_rate=0.05,  # Slightly increased to compensate
            max_depth=8,  # INcreased from 6
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.1,
            reg_lambda=1.0,
            min_child_samples=20,
            random_state=SEED,
            n_jobs=1,  # Changed from -1 to avoid threading issues
            verbose=-1,  # Added explicit verbose parameter
            objective='regression',  # Explicit objective
            boosting_type='gbdt',  # Explicit boosting type
        ),
        "tree",
    )

if HAS_XGB:
    base_models["XGB"] = (
        XGBRegressor(
            n_estimators=1200,
            learning_rate=0.03,
            max_depth=8,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.0,
            reg_lambda=1.0,
            random_state=SEED,
            n_jobs=-1,
            tree_method="hist",
        ),
        "tree",
    )

if HAS_CAT:
    base_models["CAT"] = (
        CatBoostRegressor(
            depth=8,
            learning_rate=0.03,
            n_estimators=1500,
            subsample=0.8,
            random_state=SEED,
            loss_function="RMSE",
            verbose=False,
        ),
        "tree",
    )

# Strong classical ensembles
base_models["ET"] = (
    ExtraTreesRegressor(
        n_estimators=1000, #increased from 800
        max_depth=None,
        max_features="sqrt",
        random_state=SEED,
        n_jobs=-1,
    ),
    "tree",
)

base_models["RF"] = (
    RandomForestRegressor(
        n_estimators=1000, #increased from 800
        max_depth=None,
        max_features="sqrt",
        random_state=SEED,
        n_jobs=-1,
    ),
    "tree",
)

base_models["GBDT"] = (
    GradientBoostingRegressor(
        n_estimators=800, #increased from 600
        learning_rate=0.03,
        max_depth=3,
        random_state=SEED,
    ),
    "tree",
)

# Linear baselines on scaled features (kept for diversity)
base_models["Ridge"] = (
    Ridge(alpha=5.0, random_state=SEED),
    "linear",
)
base_models["Lasso"] = (
    Lasso(alpha=0.0005, random_state=SEED, max_iter=20000),
    "linear",
)

# -------------- OOF Training / Prediction -------------- #
models_oof = {}
models_test_pred = {}
models_fold_objs = {}
fold_scores = []

# Feature matrices: tree models use TE (numeric-only); linear models use scaled copy
X_tree = X_te
X_tree_test = X_test_te

X_linear = X_lin
X_linear_test = X_test_lin

for name, (model, mtype) in base_models.items():
    print(f"\n[Model={name}] training with {NFOLDS}-fold CV ...")
    oof = np.zeros(len(X_tree))
    test_preds_folds = []
    fold_importances = []

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_tree, strat_bins)):
        if mtype == "tree":
            Xtr, Xva = X_tree.iloc[tr_idx], X_tree.iloc[va_idx]
        else:
            Xtr, Xva = X_linear.iloc[tr_idx], X_linear.iloc[va_idx]

        ytr, yva = y_work.iloc[tr_idx], y_work.iloc[va_idx]

        # Fresh instance per fold
        m = type(model)(**model.get_params())

        # Special handling for different model types
        if name == "LGBM" and HAS_LGB:
            # LightGBM specific handling with clean feature names and reset index
            Xtr_clean = Xtr.copy().reset_index(drop=True)
            Xva_clean = Xva.copy().reset_index(drop=True)
            ytr_clean = ytr.reset_index(drop=True)
            yva_clean = yva.reset_index(drop=True)
            
            # Ensure all data is numeric and clean feature names
            Xtr_clean = Xtr_clean.select_dtypes(include=[np.number])
            Xva_clean = Xva_clean.select_dtypes(include=[np.number])
            
            # Create simple sequential feature names to avoid any naming conflicts
            n_features = len(Xtr_clean.columns)
            simple_names = [f'feature_{i}' for i in range(n_features)]
            
            Xtr_clean.columns = simple_names
            Xva_clean.columns = simple_names
            
            try:
                # Use minimal parameters to avoid conflicts
                lgb_params = {
                    'n_estimators': 500,
                    'learning_rate': 0.1,
                    'max_depth': 6,
                    'num_leaves': 31,
                    'subsample': 0.8,
                    'colsample_bytree': 0.8,
                    'random_state': SEED,
                    'verbose': -1,
                    'n_jobs': 1
                }
                m = LGBMRegressor(**lgb_params)
                
                # Fit without eval_set to avoid feature name conflicts
                m.fit(Xtr_clean, ytr_clean)
                pred_va = m.predict(Xva_clean)
                
            except Exception as e:
                print(f"    Warning: LGBM fold {fold} failed: {e}")
                pred_va = np.full(len(yva_clean), ytr_clean.mean())  # Fallback prediction
                
        elif name == "CAT" and HAS_CAT:
            # CatBoost requires numeric input here (since we dropped original categorical cols).
            m.fit(Xtr.astype(np.float32), ytr, eval_set=(Xva.astype(np.float32), yva), verbose=False)
            pred_va = m.predict(Xva.astype(np.float32))
        else:
            m.fit(Xtr, ytr)
            pred_va = m.predict(Xva)

        oof[va_idx] = pred_va

        if X_tree_test is not None:
            Xte = X_tree_test if mtype == "tree" else X_linear_test
            if name == "LGBM" and HAS_LGB:
                # Apply same feature name cleaning for test predictions
                Xte_clean = Xte.copy().reset_index(drop=True)
                Xte_clean = Xte_clean.select_dtypes(include=[np.number])
                
                # Use same simple sequential names
                n_features = len(Xte_clean.columns)
                simple_names = [f'feature_{i}' for i in range(n_features)]
                Xte_clean.columns = simple_names
                
                try:
                    test_preds_folds.append(m.predict(Xte_clean))
                except Exception as e:
                    print(f"    Warning: LGBM test prediction failed: {e}")
                    test_preds_folds.append(np.full(len(Xte_clean), oof[oof != 0].mean() if len(oof[oof != 0]) > 0 else 0))
            elif name == "CAT" and HAS_CAT:
                test_preds_folds.append(m.predict(Xte.astype(np.float32)))
            else:
                test_preds_folds.append(m.predict(Xte))

        # Save feature importances when available
        if SAVE_FEATURE_IMPORTANCES and hasattr(m, "feature_importances_"):
            fi = pd.DataFrame({
                "feature": Xtr.columns,
                "importance": m.feature_importances_,
                "fold": fold,
                "model": name,
            })
            fold_importances.append(fi)

        models_fold_objs.setdefault(name, []).append(m)

    # Back-transform if log was used
    oof_final = np.expm1(oof) if use_log else oof
    rmse_score = rmse(y_full, oof_final)

    models_oof[name] = oof_final
    if test_preds_folds:
        pred_test_mean = np.mean(test_preds_folds, axis=0)
        pred_test_final = np.expm1(pred_test_mean) if use_log else pred_test_mean
        models_test_pred[name] = pred_test_final

    fold_scores.append({"model": name, "cv_rmse": rmse_score})
    print(f"[Model={name}] CV RMSE: {rmse_score:.5f}")

    if SAVE_FEATURE_IMPORTANCES and len(fold_importances) > 0:
        imp_df = pd.concat(fold_importances, ignore_index=True)
        imp_df.groupby("feature")["importance"].mean().sort_values(ascending=False).head(50).to_csv(
            f"feature_importances_{name}.csv", index=True
        )

# -------------------- Stacking (meta-model) -------------------- #
print("\n[Stacking] Training meta-model (GradientBoostingRegressor) on OOF predictions ...")
oof_df = pd.DataFrame(models_oof)

# Non-linear meta learner to capture complex relations between model preds
meta_model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=SEED,
)
meta_model.fit(oof_df, y_full)
meta_oof = meta_model.predict(oof_df)
stack_rmse = rmse(y_full, meta_oof)
print(f"[Stacking] Meta CV RMSE: {stack_rmse:.5f}")

# Weighted Blend of the best individual models (by CV)
sorted_models = sorted(fold_scores, key=lambda d: d["cv_rmse"])[: min(3, len(fold_scores))]
weights = []
for d in sorted_models:
    w = 1.0 / max(d["cv_rmse"], 1e-6)
    weights.append(w)
weights = np.array(weights) / np.sum(weights)

print("\n[Blending] Top models & weights:")
for (d, w) in zip(sorted_models, weights):
    print(f"  - {d['model']}: weight={w:.3f}, cv_rmse={d['cv_rmse']:.5f}")

# -------------------- Export OOF / Test Predictions -------------------- #
pd.DataFrame({"id": train[ID_COL], "y": y_full, **models_oof}).to_csv("oof_predictions.csv", index=False)
pd.DataFrame(fold_scores).to_csv("fold_scores.csv", index=False)

# Final test prediction (if test exists)
if X_test_full is not None and len(models_test_pred) > 0:
    # Stacking test preds
    test_stack_mat = pd.DataFrame(models_test_pred)
    stack_pred = meta_model.predict(test_stack_mat)

    # Weighted blend of top models
    blend_pred = np.zeros(len(test_stack_mat))
    for (d, w) in zip(sorted_models, weights):
        blend_pred += w * test_stack_mat[d["model"]].values

    # Final combo: average of stack & blend (often stabilizes)
    final_pred = 0.5 * stack_pred + 0.5 * blend_pred

    sub = pd.DataFrame({ID_COL: test[ID_COL], TARGET: final_pred})
    sub.to_csv("submission.csv", index=False)
    print("\nSaved submission.csv")

# Approach 6 (Tried Optimising the Best Approach by tuning parameters furhter by using Optuna...)

In [ ]:
import os
import re
import warnings
from typing import Dict, List, Tuple, Optional
import numpy as np
import pandas as pd
from scipy import stats

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

from sklearn.model_selection import StratifiedKFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
HAS_XGB = True
from lightgbm import LGBMRegressor
HAS_LGB = True
from catboost import CatBoostRegressor
HAS_CAT = True

warnings.filterwarnings("ignore")

SEED = 42
NFOLDS = 7
NREPEATS = 2
TARGET = "CORRUCYSTIC_DENSITY"
ID_COL = "LOCAL_IDENTIFIER"
AUTO_LOG_TARGET = True
SMOOTHING_M = 25.0
USE_FEATURE_SELECTION = True
SELECT_TOP_K_FEATURES = 100
SAVE_FEATURE_IMPORTANCES = True

# Optuna Configuration
USE_OPTUNA = True
OPTUNA_N_TRIALS = 50  # Number of optimization trials per model
OPTUNA_TIMEOUT = 300  # 5 minutes timeout per model optimization
OPTUNA_N_JOBS = 1  # Parallel jobs for Optuna (keep at 1 to avoid conflicts)
OPTUNA_CV_FOLDS = 3  # Reduced CV folds for faster optimization
OUTLIER_METHOD = "isolation_forest"  
OUTLIER_CONTAMINATION = 0.05

def seed_everything(seed: int = SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def read_first_existing(paths: List[str]) -> Optional[str]:
    for p in paths:
        if os.path.exists(p):
            return p
    return None

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"[^A-Za-z0-9_]", "_", str(c)).strip("_") for c in df.columns]
    df.columns = [re.sub(r"_+", "_", c) for c in df.columns]
    return df

def advanced_downcast(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.select_dtypes(include=[np.number]).columns:
        col = df[c]
        c_min, c_max = col.min(), col.max()
        
        if pd.api.types.is_integer_dtype(col):
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[c] = col.astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[c] = col.astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[c] = col.astype(np.int32)
        else:
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[c] = col.astype(np.float32)
    return df

def detect_outliers_advanced(df: pd.DataFrame, target_col: str, method: str = "isolation_forest", contamination: float = 0.05) -> pd.Series:
    from sklearn.ensemble import IsolationForest
    from sklearn.neighbors import LocalOutlierFactor
    
    mask = pd.Series(True, index=df.index)
    
    if method == "iqr":
        Q1, Q3 = df[target_col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lb, ub = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        mask = (df[target_col] >= lb) & (df[target_col] <= ub)
    
    elif method == "zscore":
        z_scores = np.abs(stats.zscore(df[target_col]))
        mask = z_scores < 3
    
    elif method == "isolation_forest":
        iso_forest = IsolationForest(contamination=contamination, random_state=SEED)
        outliers = iso_forest.fit_predict(df[[target_col]])
        mask = outliers == 1
    
    elif method == "local_outlier":
        lof = LocalOutlierFactor(contamination=contamination)
        outliers = lof.fit_predict(df[[target_col]])
        mask = outliers == 1
    
    return mask

def make_target_bins_improved(y: pd.Series, n_bins: int = 10) -> pd.Series:
    try:
        bins = pd.qcut(y, q=n_bins, labels=False, duplicates="drop")
        if bins.nunique() < n_bins // 2:
            raise ValueError("Too few unique bins")
        return bins.astype(int)
    except:
        bins = pd.cut(y, bins=n_bins, labels=False, include_lowest=True)
        return bins.astype(int)

def enhanced_target_encode(
    X: pd.DataFrame,
    y: pd.Series,
    X_test: pd.DataFrame,
    cat_cols: List[str],
    n_splits: int = NFOLDS,
    n_repeats: int = NREPEATS,
    smoothing_m: float = SMOOTHING_M,
    seed: int = SEED,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    X = X.copy()
    X_test = X_test.copy()
    global_mean = y.mean()
    
    rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=seed)
    
    for col in cat_cols:
        oof_list = []
        test_encoded_list = []
        
        for tr_idx, val_idx in rkf.split(X):
            tr_y = y.iloc[tr_idx]
            tr_col = X.iloc[tr_idx][col].astype("object").fillna("__MISSING__")
            val_col = X.iloc[val_idx][col].astype("object").fillna("__MISSING__")
            
            # Calculate statistics with smoothing
            stats_df = tr_y.groupby(tr_col).agg(['sum', 'count', 'mean', 'std']).reset_index()
            stats_df.columns = [col, 'sum', 'count', 'mean', 'std']
            stats_df['enc'] = (stats_df['sum'] + smoothing_m * global_mean) / (stats_df['count'] + smoothing_m)
            
            # Add uncertainty-based noise
            stats_df['uncertainty'] = 1 / np.sqrt(stats_df['count'] + 1)
            noise_scale = stats_df['uncertainty'] * stats_df['std'].fillna(y.std())
            stats_df['enc'] += np.random.normal(0, noise_scale * 0.1, len(stats_df))
            
            # Create mapping dict
            encode_map = dict(zip(stats_df[col], stats_df['enc']))
            
            # Encode validation set
            oof_encoded = val_col.map(encode_map).fillna(global_mean)
            oof_list.append(pd.Series(oof_encoded.values, index=val_col.index))
            
            # Encode test set
            test_col = X_test[col].astype("object").fillna("__MISSING__")
            test_encoded = test_col.map(encode_map).fillna(global_mean)
            test_encoded_list.append(test_encoded.values)
        
        # Combine OOF predictions
        oof_combined = pd.concat(oof_list).groupby(level=0).mean()
        X[col + "__te"] = oof_combined.reindex(X.index).astype(np.float32)
        
        # Average test predictions
        X_test[col + "__te"] = np.mean(test_encoded_list, axis=0).astype(np.float32)
    
    return X, X_test

def create_advanced_features(df: pd.DataFrame, num_cols: List[str]) -> pd.DataFrame:
    df = df.copy()
    
    if len(num_cols) == 0:
        return df
    
    num_data = df[num_cols].select_dtypes(include=[np.number])
    if num_data.empty:
        return df
    
    # Basic statistical features
    df["corr_density_mean"] = num_data.mean(axis=1)
    df["corr_density_std"] = num_data.std(axis=1)
    df["corr_density_skew"] = num_data.skew(axis=1)
    df["corr_density_kurt"] = num_data.kurtosis(axis=1)
    df["corr_density_range"] = num_data.max(axis=1) - num_data.min(axis=1)
    
    # Percentile features
    df["corr_density_q25"] = num_data.quantile(0.25, axis=1)
    df["corr_density_q75"] = num_data.quantile(0.75, axis=1)
    df["corr_density_iqr"] = df["corr_density_q75"] - df["corr_density_q25"]
    
    # Count-based features
    df["corr_positive_count"] = (num_data > 0).sum(axis=1)
    df["corr_negative_count"] = (num_data < 0).sum(axis=1)
    df["corr_zero_count"] = (num_data == 0).sum(axis=1)
    df["corr_missing_count"] = num_data.isnull().sum(axis=1)
    
    # Advanced statistical measures
    df["corr_cv"] = df["corr_density_std"] / (np.abs(df["corr_density_mean"]) + 1e-8)
    df["corr_mad"] = num_data.apply(manual_mad, axis=1)  # Using manual MAD implementation
    
    # Energy and signal processing inspired features (domain-specific)
    df["corr_energy"] = (num_data ** 2).sum(axis=1)
    df["corr_rms"] = np.sqrt(df["corr_energy"] / len(num_cols))
    
    # Handle infinite and NaN values
    inf_cols = df.select_dtypes(include=[np.number]).columns
    df[inf_cols] = df[inf_cols].replace([np.inf, -np.inf], np.nan)
    
    return df

def manual_mad(series):
    median = series.median()
    return np.median(np.abs(series - median))

def advanced_feature_selection(X: pd.DataFrame, y: pd.Series, k: int = 100) -> List[str]:
    if len(X.columns) <= k:
        return X.columns.tolist()
    
    # Remove features with too many missing values or zero variance
    valid_features = []
    for col in X.columns:
        if X[col].isnull().sum() / len(X) < 0.8 and X[col].nunique() > 1:
            valid_features.append(col)
    
    X_valid = X[valid_features].fillna(X[valid_features].median())
    
    # Combine multiple selection methods
    scores = {}
    
    # F-regression scores
    try:
        f_selector = SelectKBest(score_func=f_regression, k='all')
        f_selector.fit(X_valid, y)
        f_scores = dict(zip(valid_features, f_selector.scores_))
        scores['f_regression'] = f_scores
    except:
        pass
    
    # Mutual information scores
    try:
        mi_scores = mutual_info_regression(X_valid, y, random_state=SEED)
        mi_scores_dict = dict(zip(valid_features, mi_scores))
        scores['mutual_info'] = mi_scores_dict
    except:
        pass
    
    # Combine scores using rank aggregation
    if scores:
        feature_ranks = {}
        for feature in valid_features:
            ranks = []
            for method, score_dict in scores.items():
                if feature in score_dict:
                    # Convert to rank (lower rank = better)
                    sorted_features = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
                    rank = next(i for i, (f, _) in enumerate(sorted_features) if f == feature)
                    ranks.append(rank)
            feature_ranks[feature] = np.mean(ranks) if ranks else len(valid_features)
        
        # Select top k features
        selected = sorted(feature_ranks.items(), key=lambda x: x[1])[:k]
        return [f[0] for f in selected]
    
    return valid_features[:k]

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

# -------------------- Optuna Hyperparameter Optimization -------------------- #
def optimize_lgbm(X, y, cv_folds):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.0),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
            'random_state': SEED,
            'verbose': -1,
            'n_jobs': 1
        }
        
        scores = []
        for train_idx, val_idx in cv_folds.split(X, make_target_bins_improved(y, n_bins=8)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            # Clean feature names for LightGBM
            X_train_clean = X_train.select_dtypes(include=[np.number]).reset_index(drop=True)
            X_val_clean = X_val.select_dtypes(include=[np.number]).reset_index(drop=True)
            n_features = len(X_train_clean.columns)
            simple_names = [f'feature_{i}' for i in range(n_features)]
            X_train_clean.columns = simple_names
            X_val_clean.columns = simple_names
            
            try:
                model = LGBMRegressor(**params)
                model.fit(X_train_clean, y_train.reset_index(drop=True))
                pred = model.predict(X_val_clean)
                scores.append(rmse(y_val.reset_index(drop=True), pred))
            except Exception as e:
                return float('inf')  # Return worst score if error
                
        return np.mean(scores)
    
    return objective

def optimize_xgb(X, y, cv_folds):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1500, step=50),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.0),
            'random_state': SEED,
            'n_jobs': 1,
            'tree_method': 'hist'
        }
        
        scores = []
        for train_idx, val_idx in cv_folds.split(X, make_target_bins_improved(y, n_bins=8)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            try:
                model = XGBRegressor(**params)
                model.fit(X_train, y_train)
                pred = model.predict(X_val)
                scores.append(rmse(y_val, pred))
            except Exception as e:
                return float('inf')
                
        return np.mean(scores)
    
    return objective

def optimize_catboost(X, y, cv_folds):
    def objective(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 100, 1500, step=50),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'depth': trial.suggest_int('depth', 3, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'random_state': SEED,
            'verbose': False,
            'loss_function': 'RMSE'
        }
        
        scores = []
        for train_idx, val_idx in cv_folds.split(X, make_target_bins_improved(y, n_bins=8)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            try:
                model = CatBoostRegressor(**params)
                model.fit(X_train.astype(np.float32), y_train)
                pred = model.predict(X_val.astype(np.float32))
                scores.append(rmse(y_val, pred))
            except Exception as e:
                return float('inf')
                
        return np.mean(scores)
    
    return objective

def optimize_random_forest(X, y, cv_folds):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
            'max_depth': trial.suggest_int('max_depth', 5, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'random_state': SEED,
            'n_jobs': -1
        }
        
        scores = []
        for train_idx, val_idx in cv_folds.split(X, make_target_bins_improved(y, n_bins=8)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            try:
                model = RandomForestRegressor(**params)
                model.fit(X_train, y_train)
                pred = model.predict(X_val)
                scores.append(rmse(y_val, pred))
            except Exception as e:
                return float('inf')
                
        return np.mean(scores)
    
    return objective

def run_optuna_optimization(X, y, model_name):
    print(f"\n[Optuna] Optimizing {model_name} hyperparameters...")
    
    # Create CV folds for optimization
    strat_bins = make_target_bins_improved(y, n_bins=8)
    cv_folds = StratifiedKFold(n_splits=OPTUNA_CV_FOLDS, shuffle=True, random_state=SEED)
    
    # Select objective function based on model
    if model_name == "LGBM" and HAS_LGB:
        objective_func = optimize_lgbm(X, y, cv_folds)
    elif model_name == "XGB" and HAS_XGB:
        objective_func = optimize_xgb(X, y, cv_folds)
    elif model_name == "CAT" and HAS_CAT:
        objective_func = optimize_catboost(X, y, cv_folds)
    elif model_name == "RF":
        objective_func = optimize_random_forest(X, y, cv_folds)
    else:
        return None
    
    # Create study
    study = optuna.create_study(
        direction='minimize',
        sampler=TPESampler(seed=SEED),
        pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    )
    
    # Optimize
    study.optimize(
        objective_func,
        n_trials=OPTUNA_N_TRIALS,
        timeout=OPTUNA_TIMEOUT,
        n_jobs=OPTUNA_N_JOBS,
        show_progress_bar=True
    )
    
    print(f"[Optuna] {model_name} - Best score: {study.best_value:.5f}")
    print(f"[Optuna] {model_name} - Best params: {study.best_params}")
    
    return study.best_params

# -------------------- Load -------------------- #
seed_everything(SEED)

train_path = "./MinDAT.csv"
test_path = "./MiDAT_UNK.csv"

train = pd.read_csv(train_path)
train = clean_columns(train)

test = pd.read_csv(test_path)
test = clean_columns(test)

# Basic hygiene
if TARGET not in train.columns:
    raise KeyError(f"Target column '{TARGET}' not in training data.")

# Drop rows with missing target
train = train.dropna(subset=[TARGET]).reset_index(drop=True)

# Remove outliers via IQR (robust)
Q1, Q3 = train[TARGET].quantile([0.25, 0.75])
IQR = Q3 - Q1
lb, ub = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
train = train[(train[TARGET] >= lb) & (train[TARGET] <= ub)].reset_index(drop=True)

# Identify columns (keep features simple: no extra row stats or poly features)
features = [c for c in train.columns if c not in [TARGET, ID_COL]]
X_full = train[features].copy()
y_full = train[TARGET].copy()

if test is not None:
    X_test_full = test[features].copy()
else:
    X_test_full = None

# Replace infs
for df in [X_full] + ([X_test_full] if X_test_full is not None else []):
    if df is None:
        continue
    numc = df.select_dtypes(include=[np.number]).columns
    df[numc] = df[numc].replace([np.inf, -np.inf], np.nan)

# Advanced memory optimization
X_full = advanced_downcast(X_full)
if X_test_full is not None:
    X_test_full = advanced_downcast(X_test_full)

# Split types
cat_cols = X_full.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X_full.select_dtypes(include=[np.number]).columns.tolist()

# Simple imputers (per column)
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

X_full[num_cols] = num_imputer.fit_transform(X_full[num_cols])
if X_test_full is not None:
    X_test_full[num_cols] = num_imputer.transform(X_test_full[num_cols])

X_full[cat_cols] = cat_imputer.fit_transform(X_full[cat_cols])
if X_test_full is not None:
    X_test_full[cat_cols] = cat_imputer.transform(X_test_full[cat_cols])

# Cast categoricals to 'category' dtype (memory);
for c in cat_cols:
    X_full[c] = X_full[c].astype("category")
    if X_test_full is not None:
        X_test_full[c] = X_test_full[c].astype("category")

if len(cat_cols) > 0:
    print("Performing enhanced target encoding...")
    X_te, X_test_te = enhanced_target_encode(
        X_full,
        y_full,
        X_test_full if X_test_full is not None else X_full.iloc[:0],
        cat_cols,
    )
else:
    X_te, X_test_te = X_full.copy(), (X_test_full.copy() if X_test_full is not None else None)

X_te = X_te.drop(columns=cat_cols, errors="ignore")
if X_test_te is not None:
    X_test_te = X_test_te.drop(columns=cat_cols, errors="ignore")

# Feature selection
if USE_FEATURE_SELECTION and len(X_te.columns) > SELECT_TOP_K_FEATURES:
    print(f"Selecting top {SELECT_TOP_K_FEATURES} features...")
    selected_features = advanced_feature_selection(X_te, y_full, SELECT_TOP_K_FEATURES)
    X_te = X_te[selected_features]
    if X_test_te is not None:
        X_test_te = X_test_te[selected_features]
    print(f"Selected {len(selected_features)} features")

# Enhanced scaling for linear models
scaler = QuantileTransformer(output_distribution='normal', random_state=SEED)
X_lin = X_te.copy()
lin_num_cols = X_lin.select_dtypes(include=[np.number]).columns
X_lin[lin_num_cols] = scaler.fit_transform(X_lin[lin_num_cols])

if X_test_te is not None:
    X_test_lin = X_test_te.copy()
    lin_test_num = X_test_lin.select_dtypes(include=[np.number]).columns
    X_test_lin[lin_test_num] = scaler.transform(X_test_lin[lin_test_num])
else:
    X_test_lin = None

# Optional target log transform
use_log = False
if AUTO_LOG_TARGET:
    skew_val = pd.Series(y_full).skew()
    if skew_val > 1.0:
        use_log = True
        y_work = np.log1p(y_full)
    else:
        y_work = y_full.copy()
else:
    y_work = y_full.copy()

# Stratified folds by target bins (for stability)
strat_bins = make_target_bins_improved(y_full, n_bins=8)
skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
base_models: Dict[str, Tuple[object, str]] = {}

# Run Optuna optimization for key models if enabled
optimized_params = {}
if USE_OPTUNA:
    print("\n=== OPTUNA HYPERPARAMETER OPTIMIZATION ===")
    
    # List of models to optimize
    models_to_optimize = []
    if HAS_LGB:
        models_to_optimize.append("LGBM")
    if HAS_XGB:
        models_to_optimize.append("XGB")
    if HAS_CAT:
        models_to_optimize.append("CAT")
    models_to_optimize.append("RF")
    
    # Run optimization for each model
    for model_name in models_to_optimize:
        best_params = run_optuna_optimization(X_te, y_work, model_name)
        if best_params is not None:
            optimized_params[model_name] = best_params
    
    print(f"\n[Optuna] Optimization complete. Optimized {len(optimized_params)} models.")

# Build models with optimized or default parameters
if HAS_LGB:
    if "LGBM" in optimized_params:
        lgb_params = optimized_params["LGBM"]
        lgb_params.update({
            'random_state': SEED,
            'verbose': -1,
            'n_jobs': 1,
            'objective': 'regression',
            'boosting_type': 'gbdt'
        })
    else:
        lgb_params = {
            'n_estimators': 800,
            'learning_rate': 0.05,
            'max_depth': 6,
            'num_leaves': 31,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.1,
            'reg_lambda': 1.0,
            'min_child_samples': 20,
            'random_state': SEED,
            'verbose': -1,
            'n_jobs': 1,
            'objective': 'regression',
            'boosting_type': 'gbdt'
        }
    
    base_models["LGBM"] = (LGBMRegressor(**lgb_params), "tree")

if HAS_XGB:
    if "XGB" in optimized_params:
        xgb_params = optimized_params["XGB"]
        xgb_params.update({
            'random_state': SEED,
            'n_jobs': 1,
            'tree_method': 'hist'
        })
    else:
        xgb_params = {
            'n_estimators': 1200,
            'learning_rate': 0.03,
            'max_depth': 8,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.0,
            'reg_lambda': 1.0,
            'random_state': SEED,
            'n_jobs': 1,
            'tree_method': 'hist'
        }
    
    base_models["XGB"] = (XGBRegressor(**xgb_params), "tree")

if HAS_CAT:
    if "CAT" in optimized_params:
        cat_params = optimized_params["CAT"]
        cat_params.update({
            'random_state': SEED,
            'verbose': False,
            'loss_function': 'RMSE'
        })
    else:
        cat_params = {
            'depth': 8,
            'learning_rate': 0.03,
            'iterations': 1500,
            'subsample': 0.8,
            'random_state': SEED,
            'loss_function': 'RMSE',
            'verbose': False
        }
    
    base_models["CAT"] = (CatBoostRegressor(**cat_params), "tree")

# Random Forest with optimization
if "RF" in optimized_params:
    rf_params = optimized_params["RF"]
    rf_params.update({
        'random_state': SEED,
        'n_jobs': -1
    })
else:
    rf_params = {
        'n_estimators': 800,
        'max_depth': None,
        'max_features': 'sqrt',
        'random_state': SEED,
        'n_jobs': -1
    }

base_models["RF"] = (RandomForestRegressor(**rf_params), "tree")

# Strong classical ensembles (not optimized with Optuna for simplicity)
base_models["ET"] = (
    ExtraTreesRegressor(
        n_estimators=800,
        max_depth=None,
        max_features="sqrt",
        random_state=SEED,
        n_jobs=-1,
    ),
    "tree",
)

base_models["GBDT"] = (
    GradientBoostingRegressor(
        n_estimators=600,
        learning_rate=0.03,
        max_depth=3,
        random_state=SEED,
    ),
    "tree",
)

# Linear baselines on scaled features (kept for diversity)
base_models["Ridge"] = (
    Ridge(alpha=5.0, random_state=SEED),
    "linear",
)
base_models["Lasso"] = (
    Lasso(alpha=0.0005, random_state=SEED, max_iter=20000),
    "linear",
)

# -------------- OOF Training / Prediction -------------- #
models_oof = {}
models_test_pred = {}
models_fold_objs = {}
fold_scores = []

# Feature matrices: tree models use TE (numeric-only)
X_tree = X_te
X_tree_test = X_test_te

X_linear = X_lin
X_linear_test = X_test_lin

for name, (model, mtype) in base_models.items():
    print(f"\n[Model={name}] training with {NFOLDS}-fold CV ...")
    oof = np.zeros(len(X_tree))
    test_preds_folds = []
    fold_importances = []

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_tree, strat_bins)):
        if mtype == "tree":
            Xtr, Xva = X_tree.iloc[tr_idx], X_tree.iloc[va_idx]
        else:
            Xtr, Xva = X_linear.iloc[tr_idx], X_linear.iloc[va_idx]

        ytr, yva = y_work.iloc[tr_idx], y_work.iloc[va_idx]

        # Fresh instance per fold
        m = type(model)(**model.get_params())

        # Special handling for different model types
        if name == "LGBM" and HAS_LGB:
            # LightGBM specific handling with clean feature names and reset index
            Xtr_clean = Xtr.copy().reset_index(drop=True)
            Xva_clean = Xva.copy().reset_index(drop=True)
            ytr_clean = ytr.reset_index(drop=True)
            yva_clean = yva.reset_index(drop=True)
            
            # Ensure all data is numeric and clean feature names
            Xtr_clean = Xtr_clean.select_dtypes(include=[np.number])
            Xva_clean = Xva_clean.select_dtypes(include=[np.number])
            
            # Create simple sequential feature names to avoid any naming conflicts
            n_features = len(Xtr_clean.columns)
            simple_names = [f'feature_{i}' for i in range(n_features)]
            
            Xtr_clean.columns = simple_names
            Xva_clean.columns = simple_names
            
            try:
                # Use minimal parameters to avoid conflicts
                lgb_params = {
                    'n_estimators': 500,
                    'learning_rate': 0.1,
                    'max_depth': 6,
                    'num_leaves': 31,
                    'subsample': 0.8,
                    'colsample_bytree': 0.8,
                    'random_state': SEED,
                    'verbose': -1,
                    'n_jobs': 1
                }
                m = LGBMRegressor(**lgb_params)
                
                # Fit without eval_set to avoid feature name conflicts
                m.fit(Xtr_clean, ytr_clean)
                pred_va = m.predict(Xva_clean)
                
            except Exception as e:
                print(f"    Warning: LGBM fold {fold} failed: {e}")
                pred_va = np.full(len(yva_clean), ytr_clean.mean())  # Fallback prediction
                
        elif name == "CAT" and HAS_CAT:
            # CatBoost requires numeric input here (since we dropped original categorical cols).
            m.fit(Xtr.astype(np.float32), ytr, eval_set=(Xva.astype(np.float32), yva), verbose=False)
            pred_va = m.predict(Xva.astype(np.float32))
        else:
            m.fit(Xtr, ytr)
            pred_va = m.predict(Xva)

        oof[va_idx] = pred_va

        if X_tree_test is not None:
            Xte = X_tree_test if mtype == "tree" else X_linear_test
            if name == "LGBM" and HAS_LGB:
                # Apply same feature name cleaning for test predictions
                Xte_clean = Xte.copy().reset_index(drop=True)
                Xte_clean = Xte_clean.select_dtypes(include=[np.number])
                
                # Use same simple sequential names
                n_features = len(Xte_clean.columns)
                simple_names = [f'feature_{i}' for i in range(n_features)]
                Xte_clean.columns = simple_names
                
                try:
                    test_preds_folds.append(m.predict(Xte_clean))
                except Exception as e:
                    print(f"    Warning: LGBM test prediction failed: {e}")
                    test_preds_folds.append(np.full(len(Xte_clean), oof[oof != 0].mean() if len(oof[oof != 0]) > 0 else 0))
            elif name == "CAT" and HAS_CAT:
                test_preds_folds.append(m.predict(Xte.astype(np.float32)))
            else:
                test_preds_folds.append(m.predict(Xte))

        # Save feature importances when available
        if SAVE_FEATURE_IMPORTANCES and hasattr(m, "feature_importances_"):
            fi = pd.DataFrame({
                "feature": Xtr.columns,
                "importance": m.feature_importances_,
                "fold": fold,
                "model": name,
            })
            fold_importances.append(fi)

        models_fold_objs.setdefault(name, []).append(m)

    # Back-transform if log was used
    oof_final = np.expm1(oof) if use_log else oof
    rmse_score = rmse(y_full, oof_final)

    models_oof[name] = oof_final
    if test_preds_folds:
        pred_test_mean = np.mean(test_preds_folds, axis=0)
        pred_test_final = np.expm1(pred_test_mean) if use_log else pred_test_mean
        models_test_pred[name] = pred_test_final

    fold_scores.append({"model": name, "cv_rmse": rmse_score})
    print(f"[Model={name}] CV RMSE: {rmse_score:.5f}")

    if SAVE_FEATURE_IMPORTANCES and len(fold_importances) > 0:
        imp_df = pd.concat(fold_importances, ignore_index=True)
        imp_df.groupby("feature")["importance"].mean().sort_values(ascending=False).head(50).to_csv(
            f"feature_importances_{name}.csv", index=True
        )

# -------------------- Stacking (meta-model) -------------------- #
print("\n[Stacking] Training meta-model (GradientBoostingRegressor) on OOF predictions ...")
oof_df = pd.DataFrame(models_oof)

meta_model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=SEED,
)
meta_model.fit(oof_df, y_full)
meta_oof = meta_model.predict(oof_df)
stack_rmse = rmse(y_full, meta_oof)
print(f"[Stacking] Meta CV RMSE: {stack_rmse:.5f}")

# Weighted Blend of the best individual models (by CV)
sorted_models = sorted(fold_scores, key=lambda d: d["cv_rmse"])[: min(3, len(fold_scores))]
weights = []
for d in sorted_models:
    w = 1.0 / max(d["cv_rmse"], 1e-6)
    weights.append(w)
weights = np.array(weights) / np.sum(weights)

print("\n[Blending] Top models & weights:")
for (d, w) in zip(sorted_models, weights):
    print(f"  - {d['model']}: weight={w:.3f}, cv_rmse={d['cv_rmse']:.5f}")

pd.DataFrame({"id": train[ID_COL], "y": y_full, **models_oof}).to_csv("oof_predictions.csv", index=False)
pd.DataFrame(fold_scores).to_csv("fold_scores.csv", index=False)

if X_test_full is not None and len(models_test_pred) > 0:
    # Stacking test preds
    test_stack_mat = pd.DataFrame(models_test_pred)
    stack_pred = meta_model.predict(test_stack_mat)

    # Weighted blend of top models
    blend_pred = np.zeros(len(test_stack_mat))
    for (d, w) in zip(sorted_models, weights):
        blend_pred += w * test_stack_mat[d["model"]].values

    # Final combo: average of stack & blend (often stabilizes)
    final_pred = 0.5 * stack_pred + 0.5 * blend_pred

    sub = pd.DataFrame({ID_COL: test[ID_COL], TARGET: final_pred})
    sub.to_csv("submission.csv", index=False)
    print("\nSaved submission.csv")

# Another Simpler approach explored

In [ ]:
import pandas as pd, numpy as np, warnings
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.preprocessing import LabelEncoder, QuantileTransformer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import xgboost as xgb, lightgbm as lgb
from catboost import CatBoostRegressor
import re
warnings.filterwarnings("ignore")

train = pd.read_csv("MiNDAT.csv")
test = pd.read_csv("MiNDAT_UNK.csv")

target, id_col = "CORRUCYSTIC_DENSITY", "LOCAL_IDENTIFIER"
train = train.dropna(subset=[target])

print(f"Initial dataset: {train.shape[0]} fragments")
print(f"Target distribution - Mean: {train[target].mean():.4f}, Std: {train[target].std():.4f}")
print(f"Target range: [{train[target].min():.4f}, {train[target].max():.4f}]")

# Advanced Outlier Detection - Using multiple methods for alien data
def detect_outliers_iqr(data, multiplier=1.5):
    Q1, Q3 = data.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    return (data < Q1 - multiplier*IQR) | (data > Q3 + multiplier*IQR)

def detect_outliers_zscore(data, threshold=3):
    z_scores = np.abs((data - data.mean()) / data.std())
    return z_scores > threshold

# More conservative outlier removal for alien biocomputer data
outliers_iqr = detect_outliers_iqr(train[target], multiplier=2.0)  # More conservative
outliers_zscore = detect_outliers_zscore(train[target], threshold=3.5)
extreme_outliers = outliers_iqr & outliers_zscore  # Only remove extreme outliers

print(f"Removing {extreme_outliers.sum()} extreme outliers ({extreme_outliers.sum()/len(train)*100:.2f}%)")
train = train[~extreme_outliers]

features = [c for c in train.columns if c not in [target, id_col]]
X, y = train[features], train[target]
X_test = test[features]

print(f"Feature space: {len(features)} dimensions")
print(f"Training fragments: {X.shape[0]}")
print(f"Test fragments: {X_test.shape[0]}")

# Enhanced Categorical Encoding - Important for MINDSPIKE_VERSION and other alien categories
cat_cols = X.select_dtypes(include="object").columns
print(f"Categorical features (alien classifications): {len(cat_cols)}")

# Check for high cardinality categories that might need special handling
for col in cat_cols:
    unique_train = X[col].nunique()
    unique_total = pd.concat([X[col], X_test[col]]).nunique()
    print(f"  {col}: {unique_train} train classes, {unique_total} total classes")

# Advanced categorical encoding
for col in cat_cols:
    # Combine train and test for consistent encoding
    combined_data = pd.concat([X[col], X_test[col]]).fillna("CORRUPTED_DATA")
    
    # For high cardinality features, use target encoding
    if combined_data.nunique() > 20:
        # Target encoding for high cardinality categories
        target_means = train.groupby(col)[target].mean()
        global_mean = train[target].mean()
        
        # Smooth target encoding to prevent overfitting
        counts = train.groupby(col).size()
        smoothing_factor = 10
        smoothed_means = (target_means * counts + global_mean * smoothing_factor) / (counts + smoothing_factor)
        
        X[col] = X[col].map(smoothed_means).fillna(global_mean)
        X_test[col] = X_test[col].map(smoothed_means).fillna(global_mean)
    else:
        # Standard label encoding for low cardinality
        le = LabelEncoder()
        le.fit(combined_data)
        X[col] = le.transform(X[col].fillna("CORRUPTED_DATA"))
        X_test[col] = le.transform(X_test[col].fillna("CORRUPTED_DATA"))

# Advanced Numerical Feature Processing for Alien Biocomputer Data
num_cols = X.select_dtypes(include=np.number).columns
print(f"Numerical features (sensor readings): {len(num_cols)}")

# Handle infinite values and extreme outliers
for col in num_cols:
    # Replace infinities with NaN for proper imputation
    X[col] = X[col].replace([np.inf, -np.inf], np.nan)
    X_test[col] = X_test[col].replace([np.inf, -np.inf], np.nan)
    
    # Clip extreme values (beyond 5 standard deviations) - important for alien data
    if X[col].notna().sum() > 10:  # Only if we have enough data points
        mean_val = X[col].mean()
        std_val = X[col].std()
        if std_val > 0:
            lower_bound = mean_val - 5 * std_val
            upper_bound = mean_val + 5 * std_val
            X[col] = X[col].clip(lower_bound, upper_bound)
            X_test[col] = X_test[col].clip(lower_bound, upper_bound)

# Advanced Imputation Strategy
print("Applying advanced imputation for missing alien data...")

# Use IterativeImputer for better handling of complex patterns
imputer = IterativeImputer(random_state=42, max_iter=10)
X_imputed = pd.DataFrame(
    imputer.fit_transform(X[num_cols]),
    columns=num_cols,
    index=X.index
)
X_test_imputed = pd.DataFrame(
    imputer.transform(X_test[num_cols]),
    columns=num_cols,
    index=X_test.index
)

# Replace original numerical columns with imputed ones
X[num_cols] = X_imputed
X_test[num_cols] = X_test_imputed

# Advanced Feature Engineering for Alien Biocomputer Analysis
print("Engineering features for CORRUCYST fragment analysis...")

# Select most important features for interaction terms
feature_importance_selector = SelectKBest(f_regression, k=min(15, len(num_cols)))
feature_importance_selector.fit(X[num_cols], y)
important_features = num_cols[feature_importance_selector.get_support()]

print(f"Selected {len(important_features)} most important features for engineering")

# 1. Interaction features between critical measurements
for i, f1 in enumerate(important_features[:8]):  # Limit to prevent explosion
    for j, f2 in enumerate(important_features[i+1:i+4]):  # More selective
        # Multiplicative interactions (energy resonance patterns)
        X[f'{f1}_x_{f2}'] = X[f1] * X[f2]
        X_test[f'{f1}_x_{f2}'] = X_test[f1] * X_test[f2]
        
        # Ratio features (stability ratios)
        X[f'{f1}_div_{f2}'] = X[f1] / (np.abs(X[f2]) + 1e-8)
        X_test[f'{f1}_div_{f2}'] = X_test[f1] / (np.abs(X_test[f2]) + 1e-8)

# 2. Polynomial transformations for non-linear alien patterns
for f in important_features[:6]:
    # Quadratic terms for resonance patterns
    X[f'{f}_squared'] = X[f] ** 2
    X_test[f'{f}_squared'] = X_test[f] ** 2
    
    # Cube root for stability measurements
    X[f'{f}_cbrt'] = np.sign(X[f]) * np.abs(X[f]) ** (1/3)
    X_test[f'{f}_cbrt'] = np.sign(X_test[f]) * np.abs(X_test[f]) ** (1/3)
    
    # Log transformations for exponential relationships
    X[f'{f}_log'] = np.log1p(np.abs(X[f]))
    X_test[f'{f}_log'] = np.log1p(np.abs(X_test[f]))

# 3. Statistical aggregations across feature groups
if len(important_features) > 5:
    # Group statistics for pattern recognition
    X['important_mean'] = X[important_features].mean(axis=1)
    X['important_std'] = X[important_features].std(axis=1)
    X['important_skew'] = X[important_features].skew(axis=1)
    X['important_max'] = X[important_features].max(axis=1)
    X['important_min'] = X[important_features].min(axis=1)
    
    X_test['important_mean'] = X_test[important_features].mean(axis=1)
    X_test['important_std'] = X_test[important_features].std(axis=1)
    X_test['important_skew'] = X_test[important_features].skew(axis=1)
    X_test['important_max'] = X_test[important_features].max(axis=1)
    X_test['important_min'] = X_test[important_features].min(axis=1)

# 4. Alien-specific patterns (based on domain knowledge)
# Energy coherence ratios
energy_features = [f for f in X.columns if any(keyword in f.lower() for keyword in ['energy', 'power', 'volt', 'amp'])]
if len(energy_features) >= 2:
    X['energy_coherence'] = X[energy_features].sum(axis=1) / (X[energy_features].std(axis=1) + 1e-8)
    X_test['energy_coherence'] = X_test[energy_features].sum(axis=1) / (X_test[energy_features].std(axis=1) + 1e-8)

print(f"Feature engineering complete. New feature count: {X.shape[1]}")

# Clean feature names for LightGBM / XGBoost compatibility
def clean_column(name):
    # Replace any non-alphanumeric character with underscore
    return re.sub(r'[^A-Za-z0-9_]', '_', str(name))

X.columns = [clean_column(c) for c in X.columns]
X_test.columns = [clean_column(c) for c in X_test.columns]

# Advanced Scaling Strategy for Alien Data
print("Applying robust scaling for alien biocomputer measurements...")

# Use QuantileTransformer for non-normal distributions (common in alien data)
scaler = QuantileTransformer(output_distribution='normal', random_state=42)
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# Keep original data for tree-based models and scaled for linear models
X_original = X.copy()
X_test_original = X_test.copy()
X = X_scaled
X_test = X_test_scaled

print(f"Final feature matrix: {X.shape} for training, {X_test.shape} for prediction")

# Enhanced Model Suite for Alien Data Analysis
models = {
    "RF": RandomForestRegressor(n_estimators=500, max_depth=20, random_state=42, n_jobs=-1, 
                               min_samples_split=3, min_samples_leaf=2),
    "ET": ExtraTreesRegressor(n_estimators=500, max_depth=20, random_state=42, n_jobs=-1,
                             min_samples_split=3, min_samples_leaf=2),
    "GB": GradientBoostingRegressor(n_estimators=300, learning_rate=0.03, max_depth=8, 
                                   random_state=42, subsample=0.8),
    "XGB": xgb.XGBRegressor(n_estimators=500, learning_rate=0.03, max_depth=8, 
                           random_state=42, subsample=0.8, colsample_bytree=0.8),
    "LGB": lgb.LGBMRegressor(n_estimators=500, learning_rate=0.03, max_depth=8, 
                            random_state=42, subsample=0.8, colsample_bytree=0.8, verbose=-1),
    "CatBoost": CatBoostRegressor(iterations=500, learning_rate=0.03, depth=8, 
                                 random_seed=42, verbose=False),
    "Ridge": Ridge(alpha=10.0),
    "Lasso": Lasso(alpha=1.0, max_iter=2000),
    "ElasticNet": ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=2000),
    "BayesianRidge": BayesianRidge(),
    "Huber": HuberRegressor(epsilon=1.35, max_iter=500),
}

# Tree-based models use original data, linear models use scaled data
tree_models = ["RF", "ET", "GB", "XGB", "LGB", "CatBoost"]
linear_models = ["Ridge", "Lasso", "ElasticNet", "BayesianRidge", "Huber"]

# Enhanced Cross-Validation with Multiple Metrics
kf = KFold(n_splits=7, shuffle=True, random_state=42)  # More folds for better validation
scores, trained_models = {}, {}

print("\n=== CORRUCYST Fragment Analysis Results ===")
for name, model in models.items():
    print(f"Training {name} for alien biocomputer analysis...")
    
    # Choose appropriate data
    X_train = X_original if name in tree_models else X
    X_test_model = X_test_original if name in tree_models else X_test
    
    rmses, maes, r2s, folds = [], [], [], []
    for tr, val in kf.split(X_train):
        m = type(model)(**model.get_params())
        m.fit(X_train.iloc[tr], y.iloc[tr])
        p = m.predict(X_train.iloc[val])
        
        rmses.append(mean_squared_error(y.iloc[val], p, squared=False))
        maes.append(mean_absolute_error(y.iloc[val], p))
        r2s.append(r2_score(y.iloc[val], p))
        folds.append(m)
    
    scores[name] = {
        "CV_RMSE": np.mean(rmses), "CV_RMSE_STD": np.std(rmses),
        "CV_MAE": np.mean(maes), "CV_MAE_STD": np.std(maes),
        "CV_R2": np.mean(r2s), "CV_R2_STD": np.std(r2s)
    }
    trained_models[name] = folds
    
    print(f"{name:12} - RMSE: {np.mean(rmses):.4f} (±{np.std(rmses):.4f}) | "
          f"MAE: {np.mean(maes):.4f} (±{np.std(maes):.4f}) | "
          f"R²: {np.mean(r2s):.4f} (±{np.std(r2s):.4f})")

best_model_name = min(scores, key=lambda x: scores[x]["CV_RMSE"])
print(f"\nBest performing model: {best_model_name}")

# Advanced Hyperparameter Tuning for Best Model
print(f"\nPerforming advanced hyperparameter tuning for {best_model_name}...")

hyperparameter_grids = {
    "RF": {
        "n_estimators": [300, 500, 700],
        "max_depth": [15, 20, 25],
        "min_samples_split": [2, 3, 5],
        "min_samples_leaf": [1, 2, 3]
    },
    "ET": {
        "n_estimators": [300, 500, 700],
        "max_depth": [15, 20, 25],
        "min_samples_split": [2, 3, 5],
        "min_samples_leaf": [1, 2, 3]
    },
    "GB": {
        "n_estimators": [200, 300, 400],
        "learning_rate": [0.02, 0.03, 0.05],
        "max_depth": [6, 8, 10],
        "subsample": [0.7, 0.8, 0.9]
    },
    "XGB": {
        "n_estimators": [300, 500, 700],
        "learning_rate": [0.02, 0.03, 0.05],
        "max_depth": [6, 8, 10],
        "subsample": [0.7, 0.8, 0.9]
    },
    "LGB": {
        "n_estimators": [300, 500, 700],
        "learning_rate": [0.02, 0.03, 0.05],
        "max_depth": [6, 8, 10],
        "subsample": [0.7, 0.8, 0.9]
    },
    "CatBoost": {
        "iterations": [300, 500, 700],
        "learning_rate": [0.02, 0.03, 0.05],
        "depth": [6, 8, 10]
    },
    "Ridge": {"alpha": [0.1, 1, 10, 100, 1000]},
    "Lasso": {"alpha": [0.001, 0.01, 0.1, 1, 10]},
    "ElasticNet": {"alpha": [0.1, 1, 10], "l1_ratio": [0.1, 0.5, 0.7, 0.9]},
    "BayesianRidge": {"alpha_1": [1e-6, 1e-5, 1e-4], "alpha_2": [1e-6, 1e-5, 1e-4]},
    "Huber": {"epsilon": [1.1, 1.35, 1.5, 2.0], "alpha": [0.0001, 0.001, 0.01]}
}

tuned_model = None
if best_model_name in hyperparameter_grids:
    base_model = models[best_model_name]
    X_tune = X_original if best_model_name in tree_models else X
    
    grid = GridSearchCV(
        base_model, 
        hyperparameter_grids[best_model_name], 
        cv=5,
        scoring="neg_mean_squared_error", 
        n_jobs=-1, 
        verbose=1
    )
    grid.fit(X_tune, y)
    tuned_model = grid.best_estimator_
    print(f"Best hyperparameters: {grid.best_params_}")
    print(f"Tuned model CV score: {-grid.best_score_:.4f}")

# Advanced Ensemble Strategy
print("\n=== Creating Advanced Ensemble for CORRUCYST Prediction ===")

# 1. Top performing models weighted ensemble
top_models = sorted(scores.items(), key=lambda x: x[1]["CV_RMSE"])[:5]
print("Top 5 models for ensemble:")
for name, score in top_models:
    print(f"  {name}: RMSE {score['CV_RMSE']:.4f}")

# Weight by inverse RMSE with exponential scaling
ensemble_weights = {}
total_weight = 0
for name, score in top_models:
    weight = np.exp(-score["CV_RMSE"] * 2)  # Exponential weighting
    ensemble_weights[name] = weight
    total_weight += weight

# Normalize weights
for name in ensemble_weights:
    ensemble_weights[name] /= total_weight
    print(f"  {name} weight: {ensemble_weights[name]:.3f}")

# Create ensemble predictions
ensemble_preds = np.zeros(X_test.shape[0])
for name, weight in ensemble_weights.items():
    X_pred = X_test_original if name in tree_models else X_test
    fold_preds = np.mean([m.predict(X_pred) for m in trained_models[name]], axis=0)
    ensemble_preds += weight * fold_preds

# 2. Stacking ensemble (meta-learner)
print("\nCreating stacking ensemble...")
meta_features = np.zeros((X.shape[0], len(top_models)))
meta_test_features = np.zeros((X_test.shape[0], len(top_models)))

# Generate meta-features using cross-validation
for fold_idx, (tr, val) in enumerate(kf.split(X)):
    for model_idx, (name, _) in enumerate(top_models):
        X_fold = X_original if name in tree_models else X
        X_test_fold = X_test_original if name in tree_models else X_test
        
        # Train on fold and predict validation
        model = trained_models[name][fold_idx]
        meta_features[val, model_idx] = model.predict(X_fold.iloc[val])

# Generate test meta-features
for model_idx, (name, _) in enumerate(top_models):
    X_pred = X_test_original if name in tree_models else X_test
    meta_test_features[:, model_idx] = np.mean(
        [m.predict(X_pred) for m in trained_models[name]], axis=0
    )

# Train meta-learner (simple Ridge regression)
meta_learner = Ridge(alpha=1.0)
meta_learner.fit(meta_features, y)
stacking_preds = meta_learner.predict(meta_test_features)

# Final predictions - ensemble of ensemble and stacking
if tuned_model is not None:
    X_final = X_original if best_model_name in tree_models else X_test
    tuned_preds = tuned_model.predict(X_final)
    # Weighted combination: 40% ensemble, 30% stacking, 30% tuned best model
    final_preds = 0.4 * ensemble_preds + 0.3 * stacking_preds + 0.3 * tuned_preds
else:
    # Weighted combination: 60% ensemble, 40% stacking
    final_preds = 0.6 * ensemble_preds + 0.4 * stacking_preds

# Generate submission
submission = pd.DataFrame({
    id_col: test[id_col], 
    target: final_preds
})
submission.to_csv("enhanced_corrucyst_submission.csv", index=False)

# Approach where i treid featuer engineering wiht optuna optimisation

In [ ]:
import os
import re
import warnings
from typing import Dict, List, Tuple, Optional
import numpy as np
import pandas as pd
from scipy import stats

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

from sklearn.model_selection import StratifiedKFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

warnings.filterwarnings("ignore")

SEED = 42
NFOLDS = 10  # Increased for better stability
NREPEATS = 3  # More repeats for robust estimates
TARGET = "CORRUCYSTIC_DENSITY"
ID_COL = "LOCAL_IDENTIFIER"
AUTO_LOG_TARGET = True
SMOOTHING_M = 50.0  # Higher smoothing for more conservative TE
USE_FEATURE_SELECTION = True
SELECT_TOP_K_FEATURES = 150  # More features for complex models
SAVE_FEATURE_IMPORTANCES = True

# Advanced Tuning Configuration
USE_PSEUDO_LABELING = True
PSEUDO_LABEL_THRESHOLD = 0.95  # Confidence threshold for pseudo-labels
USE_ADVANCED_FEATURES = True
USE_INTERACTION_FEATURES = True
USE_NOISE_INJECTION = True
NOISE_LEVEL = 0.01  # Small noise for regularization
USE_MULTI_LEVEL_STACKING = True
USE_ADAPTIVE_WEIGHTS = True

# Optuna Configuration (reduced for faster tuning)
USE_OPTUNA = True
OPTUNA_N_TRIALS = 30  
OPTUNA_TIMEOUT = 200
OPTUNA_N_JOBS = 1
OPTUNA_CV_FOLDS = 3
OUTLIER_METHOD = "isolation_forest"
OUTLIER_CONTAMINATION = 0.05 


# -------------------- Enhanced Utils -------------------- #
def seed_everything(seed: int = SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def read_first_existing(paths: List[str]) -> Optional[str]:
    for p in paths:
        if os.path.exists(p):
            return p
    return None

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Clean column names more thoroughly."""
    df = df.copy()
    df.columns = [re.sub(r"[^A-Za-z0-9_]", "_", str(c)).strip("_") for c in df.columns]
    # Remove consecutive underscores
    df.columns = [re.sub(r"_+", "_", c) for c in df.columns]
    return df

def advanced_downcast(df: pd.DataFrame) -> pd.DataFrame:
    """More aggressive memory optimization."""
    df = df.copy()
    for c in df.select_dtypes(include=[np.number]).columns:
        col = df[c]
        c_min, c_max = col.min(), col.max()
        
        if pd.api.types.is_integer_dtype(col):
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[c] = col.astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[c] = col.astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[c] = col.astype(np.int32)
        else:
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[c] = col.astype(np.float32)
    return df

def detect_outliers_advanced(df: pd.DataFrame, target_col: str, method: str = "isolation_forest", contamination: float = 0.05) -> pd.Series:
    """Advanced outlier detection using multiple methods."""
    from sklearn.ensemble import IsolationForest
    from sklearn.neighbors import LocalOutlierFactor
    
    mask = pd.Series(True, index=df.index)
    
    if method == "iqr":
        Q1, Q3 = df[target_col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lb, ub = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        mask = (df[target_col] >= lb) & (df[target_col] <= ub)
    
    elif method == "zscore":
        z_scores = np.abs(stats.zscore(df[target_col]))
        mask = z_scores < 3
    
    elif method == "isolation_forest":
        iso_forest = IsolationForest(contamination=contamination, random_state=SEED)
        outliers = iso_forest.fit_predict(df[[target_col]])
        mask = outliers == 1
    
    elif method == "local_outlier":
        lof = LocalOutlierFactor(contamination=contamination)
        outliers = lof.fit_predict(df[[target_col]])
        mask = outliers == 1
    
    return mask

def make_target_bins_improved(y: pd.Series, n_bins: int = 10) -> pd.Series:
    """Improved target binning with better handling of edge cases."""
    # Try quantile-based binning first
    try:
        bins = pd.qcut(y, q=n_bins, labels=False, duplicates="drop")
        if bins.nunique() < n_bins // 2:  # If too few unique bins
            raise ValueError("Too few unique bins")
        return bins.astype(int)
    except:
        # Fallback to equal-width binning
        bins = pd.cut(y, bins=n_bins, labels=False, include_lowest=True)
        return bins.astype(int)

def enhanced_target_encode(
    X: pd.DataFrame,
    y: pd.Series,
    X_test: pd.DataFrame,
    cat_cols: List[str],
    n_splits: int = NFOLDS,
    n_repeats: int = NREPEATS,
    smoothing_m: float = SMOOTHING_M,
    seed: int = SEED,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Enhanced target encoding with repeated CV and noise addition for robustness."""
    X = X.copy()
    X_test = X_test.copy()
    global_mean = y.mean()
    
    # Use repeated K-fold for more stable encoding
    rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=seed)
    
    for col in cat_cols:
        oof_list = []
        test_encoded_list = []
        
        for tr_idx, val_idx in rkf.split(X):
            tr_y = y.iloc[tr_idx]
            tr_col = X.iloc[tr_idx][col].astype("object").fillna("__MISSING__")
            val_col = X.iloc[val_idx][col].astype("object").fillna("__MISSING__")
            
            # Calculate statistics with smoothing
            stats_df = tr_y.groupby(tr_col).agg(['sum', 'count', 'mean', 'std']).reset_index()
            stats_df.columns = [col, 'sum', 'count', 'mean', 'std']
            stats_df['enc'] = (stats_df['sum'] + smoothing_m * global_mean) / (stats_df['count'] + smoothing_m)
            
            # Add uncertainty-based noise
            stats_df['uncertainty'] = 1 / np.sqrt(stats_df['count'] + 1)
            noise_scale = stats_df['uncertainty'] * stats_df['std'].fillna(y.std())
            stats_df['enc'] += np.random.normal(0, noise_scale * 0.1, len(stats_df))
            
            # Create mapping dict
            encode_map = dict(zip(stats_df[col], stats_df['enc']))
            
            # Encode validation set
            oof_encoded = val_col.map(encode_map).fillna(global_mean)
            oof_list.append(pd.Series(oof_encoded.values, index=val_col.index))
            
            # Encode test set
            test_col = X_test[col].astype("object").fillna("__MISSING__")
            test_encoded = test_col.map(encode_map).fillna(global_mean)
            test_encoded_list.append(test_encoded.values)
        
        # Combine OOF predictions
        oof_combined = pd.concat(oof_list).groupby(level=0).mean()
        X[col + "__te"] = oof_combined.reindex(X.index).astype(np.float32)
        
        # Average test predictions
        X_test[col + "__te"] = np.mean(test_encoded_list, axis=0).astype(np.float32)
    
    return X, X_test

def create_advanced_features(df: pd.DataFrame, num_cols: List[str]) -> pd.DataFrame:
    df = df.copy()
    
    if len(num_cols) == 0:
        return df
    
    # Ensure we have numeric data
    num_data = df[num_cols].select_dtypes(include=[np.number])
    if num_data.empty:
        return df
    
    # Basic statistical features
    df["corr_density_mean"] = num_data.mean(axis=1)
    df["corr_density_std"] = num_data.std(axis=1)
    df["corr_density_skew"] = num_data.skew(axis=1)
    df["corr_density_kurt"] = num_data.kurtosis(axis=1)
    df["corr_density_range"] = num_data.max(axis=1) - num_data.min(axis=1)
    
    # Percentile features
    df["corr_density_q25"] = num_data.quantile(0.25, axis=1)
    df["corr_density_q75"] = num_data.quantile(0.75, axis=1)
    df["corr_density_iqr"] = df["corr_density_q75"] - df["corr_density_q25"]
    
    # Count-based features
    df["corr_positive_count"] = (num_data > 0).sum(axis=1)
    df["corr_negative_count"] = (num_data < 0).sum(axis=1)
    df["corr_zero_count"] = (num_data == 0).sum(axis=1)
    df["corr_missing_count"] = num_data.isnull().sum(axis=1)
    
    # Advanced statistical measures
    df["corr_cv"] = df["corr_density_std"] / (np.abs(df["corr_density_mean"]) + 1e-8)
    df["corr_mad"] = num_data.apply(manual_mad, axis=1)  # Using manual MAD implementation
    
    # Energy and signal processing inspired features (domain-specific)
    df["corr_energy"] = (num_data ** 2).sum(axis=1)
    df["corr_rms"] = np.sqrt(df["corr_energy"] / len(num_cols))
    
    # Handle infinite and NaN values
    inf_cols = df.select_dtypes(include=[np.number]).columns
    df[inf_cols] = df[inf_cols].replace([np.inf, -np.inf], np.nan)
    
    return df

def manual_mad(series):
    median = series.median()
    return np.median(np.abs(series - median))

def create_interaction_features(df: pd.DataFrame, num_cols: List[str], max_interactions: int = 20) -> pd.DataFrame:
    df = df.copy()
    
    if len(num_cols) < 2:
        return df
    
    # Select top correlated features with target for interactions
    # For now, use first few numerical features
    top_cols = num_cols[:min(6, len(num_cols))]
    
    interaction_count = 0
    for i, col1 in enumerate(top_cols):
        for j, col2 in enumerate(top_cols[i+1:], i+1):
            if interaction_count >= max_interactions:
                break
            
            # Create interaction features
            df[f"{col1}_x_{col2}"] = df[col1] * df[col2]
            df[f"{col1}_div_{col2}"] = df[col1] / (df[col2] + 1e-8)
            df[f"{col1}_add_{col2}"] = df[col1] + df[col2]
            df[f"{col1}_sub_{col2}"] = df[col1] - df[col2]
            
            interaction_count += 4
            
        if interaction_count >= max_interactions:
            break
    
    return df

def create_polynomial_features(df: pd.DataFrame, num_cols: List[str], degree: int = 2, max_features: int = 10) -> pd.DataFrame:
    df = df.copy()
    
    if len(num_cols) == 0:
        return df
    
    # Select top features for polynomial expansion
    top_cols = num_cols[:min(max_features, len(num_cols))]
    
    for col in top_cols:
        if degree >= 2:
            df[f"{col}_squared"] = df[col] ** 2
        if degree >= 3:
            df[f"{col}_cubed"] = df[col] ** 3
        
        # Log and sqrt transformations
        df[f"{col}_log"] = np.log1p(np.abs(df[col]))
        df[f"{col}_sqrt"] = np.sqrt(np.abs(df[col]))
    
    return df

def create_binning_features(df: pd.DataFrame, num_cols: List[str], n_bins: int = 5) -> pd.DataFrame:
    df = df.copy()
    
    for col in num_cols[:10]:  # Limit to top 10 features
        try:
            # Equal frequency binning
            df[f"{col}_bin_freq"] = pd.qcut(df[col], q=n_bins, labels=False, duplicates='drop')
            # Equal width binning
            df[f"{col}_bin_width"] = pd.cut(df[col], bins=n_bins, labels=False)
        except:
            continue  # Skip if binning fails
    
    return df

def add_noise_features(df: pd.DataFrame, noise_level: float = 0.01, n_features: int = 5) -> pd.DataFrame:
    df = df.copy()
    
    for i in range(n_features):
        df[f"noise_{i}"] = np.random.normal(0, noise_level, len(df))
    
    return df

def create_cluster_features(df: pd.DataFrame, num_cols: List[str], n_clusters: int = 8) -> pd.DataFrame:
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler
    
    df = df.copy()
    
    if len(num_cols) < 2:
        return df
    
    # Use top numerical features for clustering
    cluster_data = df[num_cols[:10]].fillna(0)
    
    if cluster_data.shape[1] >= 2:
        scaler = StandardScaler()
        cluster_data_scaled = scaler.fit_transform(cluster_data)
        
        kmeans = KMeans(n_clusters=n_clusters, random_state=SEED, n_init=10)
        df['cluster_id'] = kmeans.fit_predict(cluster_data_scaled)
        
        # Distance to cluster centers
        distances = kmeans.transform(cluster_data_scaled)
        df['min_cluster_dist'] = distances.min(axis=1)
        df['mean_cluster_dist'] = distances.mean(axis=1)
    
    return df

def create_pseudo_labels(X_test: pd.DataFrame, models_oof: Dict, models_test_pred: Dict, threshold: float = 0.95) -> Tuple[pd.DataFrame, pd.Series]:
    """Create pseudo-labels from high-confidence predictions."""
    if len(models_test_pred) == 0:
        return pd.DataFrame(), pd.Series(dtype=float)
    
    # Calculate ensemble prediction and confidence
    predictions = np.array(list(models_test_pred.values())).T
    ensemble_pred = np.mean(predictions, axis=1)
    prediction_std = np.std(predictions, axis=1)
    
    # Use coefficient of variation as confidence measure
    confidence = 1 / (1 + prediction_std / (np.abs(ensemble_pred) + 1e-8))
    
    # Select high-confidence predictions
    high_conf_mask = confidence >= threshold
    
    if high_conf_mask.sum() > 0:
        pseudo_X = X_test[high_conf_mask].copy()
        pseudo_y = pd.Series(ensemble_pred[high_conf_mask], index=pseudo_X.index)
        return pseudo_X, pseudo_y
    
    return pd.DataFrame(), pd.Series(dtype=float)

def calculate_adaptive_weights(fold_scores: List[Dict], diversity_factor: float = 0.1) -> Dict[str, float]:
    models = list(fold_scores[0].keys())
    if 'model' in models:
        models.remove('model')
    
    # Calculate performance weights (inverse of RMSE)
    perf_weights = {}
    for model in models:
        avg_score = np.mean([score.get(model, float('inf')) for score in fold_scores if model in score])
        perf_weights[model] = 1.0 / max(avg_score, 1e-6)
    
    # Normalize performance weights
    total_perf = sum(perf_weights.values())
    perf_weights = {k: v/total_perf for k, v in perf_weights.items()}
    
    # Add diversity bonus (simplified - could use correlation-based diversity)
    final_weights = {}
    for model in models:
        diversity_bonus = diversity_factor * np.random.uniform(0.8, 1.2)  # Random diversity factor
        final_weights[model] = perf_weights[model] * (1 + diversity_bonus)
    
    # Normalize final weights
    total_final = sum(final_weights.values())
    final_weights = {k: v/total_final for k, v in final_weights.items()}
    
    return final_weights

def create_meta_features(X: pd.DataFrame, models_oof: Dict) -> pd.DataFrame:
    """Create meta-features from base model predictions."""
    meta_df = X.copy()
    
    # Add OOF predictions as features
    for model_name, oof_pred in models_oof.items():
        meta_df[f"oof_{model_name}"] = oof_pred
    
    # Add ensemble statistics
    if len(models_oof) > 1:
        oof_matrix = np.array(list(models_oof.values())).T
        meta_df["oof_mean"] = np.mean(oof_matrix, axis=1)
        meta_df["oof_std"] = np.std(oof_matrix, axis=1)
        meta_df["oof_min"] = np.min(oof_matrix, axis=1)
        meta_df["oof_max"] = np.max(oof_matrix, axis=1)
        meta_df["oof_range"] = meta_df["oof_max"] - meta_df["oof_min"]
    
    return meta_df

def advanced_feature_selection(X: pd.DataFrame, y: pd.Series, k: int = 100) -> List[str]:
    """Advanced feature selection using multiple methods."""
    if len(X.columns) <= k:
        return X.columns.tolist()
    
    # Remove features with too many missing values or zero variance
    valid_features = []
    for col in X.columns:
        if X[col].isnull().sum() / len(X) < 0.8 and X[col].nunique() > 1:
            valid_features.append(col)
    
    X_valid = X[valid_features].fillna(X[valid_features].median())
    
    # Combine multiple selection methods
    scores = {}
    
    # F-regression scores
    try:
        f_selector = SelectKBest(score_func=f_regression, k='all')
        f_selector.fit(X_valid, y)
        f_scores = dict(zip(valid_features, f_selector.scores_))
        scores['f_regression'] = f_scores
    except:
        pass
    
    # Mutual information scores
    try:
        mi_scores = mutual_info_regression(X_valid, y, random_state=SEED)
        mi_scores_dict = dict(zip(valid_features, mi_scores))
        scores['mutual_info'] = mi_scores_dict
    except:
        pass
    
    # Combine scores using rank aggregation
    if scores:
        feature_ranks = {}
        for feature in valid_features:
            ranks = []
            for method, score_dict in scores.items():
                if feature in score_dict:
                    # Convert to rank (lower rank = better)
                    sorted_features = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
                    rank = next(i for i, (f, _) in enumerate(sorted_features) if f == feature)
                    ranks.append(rank)
            feature_ranks[feature] = np.mean(ranks) if ranks else len(valid_features)
        
        # Select top k features
        selected = sorted(feature_ranks.items(), key=lambda x: x[1])[:k]
        return [f[0] for f in selected]
    
    return valid_features[:k]

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

# -------------------- Optuna Hyperparameter Optimization -------------------- #
def optimize_lgbm(X, y, cv_folds):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.0),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
            'random_state': SEED,
            'verbose': -1,
            'n_jobs': 1
        }
        
        scores = []
        for train_idx, val_idx in cv_folds.split(X, make_target_bins_improved(y, n_bins=8)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            # Clean feature names for LightGBM
            X_train_clean = X_train.select_dtypes(include=[np.number]).reset_index(drop=True)
            X_val_clean = X_val.select_dtypes(include=[np.number]).reset_index(drop=True)
            n_features = len(X_train_clean.columns)
            simple_names = [f'feature_{i}' for i in range(n_features)]
            X_train_clean.columns = simple_names
            X_val_clean.columns = simple_names
            
            try:
                model = LGBMRegressor(**params)
                model.fit(X_train_clean, y_train.reset_index(drop=True))
                pred = model.predict(X_val_clean)
                scores.append(rmse(y_val.reset_index(drop=True), pred))
            except Exception as e:
                return float('inf')  # Return worst score if error
                
        return np.mean(scores)
    
    return objective

def optimize_xgb(X, y, cv_folds):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1500, step=50),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.0),
            'random_state': SEED,
            'n_jobs': 1,
            'tree_method': 'hist'
        }
        
        scores = []
        for train_idx, val_idx in cv_folds.split(X, make_target_bins_improved(y, n_bins=8)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            try:
                model = XGBRegressor(**params)
                model.fit(X_train, y_train)
                pred = model.predict(X_val)
                scores.append(rmse(y_val, pred))
            except Exception as e:
                return float('inf')
                
        return np.mean(scores)
    
    return objective

def optimize_catboost(X, y, cv_folds):
    def objective(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 100, 1500, step=50),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'depth': trial.suggest_int('depth', 3, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'random_state': SEED,
            'verbose': False,
            'loss_function': 'RMSE'
        }
        
        scores = []
        for train_idx, val_idx in cv_folds.split(X, make_target_bins_improved(y, n_bins=8)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            try:
                model = CatBoostRegressor(**params)
                model.fit(X_train.astype(np.float32), y_train)
                pred = model.predict(X_val.astype(np.float32))
                scores.append(rmse(y_val, pred))
            except Exception as e:
                return float('inf')
                
        return np.mean(scores)
    
    return objective

def optimize_random_forest(X, y, cv_folds):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
            'max_depth': trial.suggest_int('max_depth', 5, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'random_state': SEED,
            'n_jobs': -1
        }
        
        scores = []
        for train_idx, val_idx in cv_folds.split(X, make_target_bins_improved(y, n_bins=8)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            try:
                model = RandomForestRegressor(**params)
                model.fit(X_train, y_train)
                pred = model.predict(X_val)
                scores.append(rmse(y_val, pred))
            except Exception as e:
                return float('inf')
                
        return np.mean(scores)
    
    return objective

def run_optuna_optimization(X, y, model_name):
    print(f"\n[Optuna] Optimizing {model_name} hyperparameters...")
    
    # Create CV folds for optimization
    strat_bins = make_target_bins_improved(y, n_bins=8)
    cv_folds = StratifiedKFold(n_splits=OPTUNA_CV_FOLDS, shuffle=True, random_state=SEED)
    
    # Select objective function based on model
    if model_name == "LGBM" and HAS_LGB:
        objective_func = optimize_lgbm(X, y, cv_folds)
    elif model_name == "XGB" and HAS_XGB:
        objective_func = optimize_xgb(X, y, cv_folds)
    elif model_name == "CAT" and HAS_CAT:
        objective_func = optimize_catboost(X, y, cv_folds)
    elif model_name == "RF":
        objective_func = optimize_random_forest(X, y, cv_folds)
    else:
        return None  # Skip optimization for this model
    
    # Create study
    study = optuna.create_study(
        direction='minimize',
        sampler=TPESampler(seed=SEED),
        pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    )
    
    # Optimize
    study.optimize(
        objective_func,
        n_trials=OPTUNA_N_TRIALS,
        timeout=OPTUNA_TIMEOUT,
        n_jobs=OPTUNA_N_JOBS,
        show_progress_bar=True
    )
    
    print(f"[Optuna] {model_name} - Best score: {study.best_value:.5f}")
    print(f"[Optuna] {model_name} - Best params: {study.best_params}")
    
    return study.best_params

# -------------------- Load -------------------- #
seed_everything(SEED)

train_path = "./MiNDAT.csv"
test_path = "./MiNDAT_UNK.csv"

train = pd.read_csv(train_path)
train = clean_columns(train)

test = pd.read_csv(test_path)
test = clean_columns(test)

# Basic hygiene
if TARGET not in train.columns:
    raise KeyError(f"Target column '{TARGET}' not in training data.")

# Drop rows with missing target
train = train.dropna(subset=[TARGET]).reset_index(drop=True)

# Remove outliers via IQR (robust)
Q1, Q3 = train[TARGET].quantile([0.25, 0.75])
IQR = Q3 - Q1
lb, ub = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
train = train[(train[TARGET] >= lb) & (train[TARGET] <= ub)].reset_index(drop=True)

# Identify columns (keep features simple: no extra row stats or poly features)
features = [c for c in train.columns if c not in [TARGET, ID_COL]]
X_full = train[features].copy()
y_full = train[TARGET].copy()

if test is not None:
    X_test_full = test[features].copy()
else:
    X_test_full = None

# Replace infs
for df in [X_full] + ([X_test_full] if X_test_full is not None else []):
    if df is None:
        continue
    numc = df.select_dtypes(include=[np.number]).columns
    df[numc] = df[numc].replace([np.inf, -np.inf], np.nan)

# Advanced memory optimization
X_full = advanced_downcast(X_full)
if X_test_full is not None:
    X_test_full = advanced_downcast(X_test_full)

# Split types
cat_cols = X_full.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X_full.select_dtypes(include=[np.number]).columns.tolist()

# Simple imputers (per column)
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

X_full[num_cols] = num_imputer.fit_transform(X_full[num_cols])
if X_test_full is not None:
    X_test_full[num_cols] = num_imputer.transform(X_test_full[num_cols])

X_full[cat_cols] = cat_imputer.fit_transform(X_full[cat_cols])
if X_test_full is not None:
    X_test_full[cat_cols] = cat_imputer.transform(X_test_full[cat_cols])

# Cast categoricals to 'category' dtype (memory);
for c in cat_cols:
    X_full[c] = X_full[c].astype("category")
    if X_test_full is not None:
        X_test_full[c] = X_test_full[c].astype("category")

# -------------------- Target Encoding Only (No Feature Engineering) -------------------- #
# KFold Target Encoding (leakage-safe)
if len(cat_cols) > 0:
    print("Performing enhanced target encoding...")
    X_te, X_test_te = enhanced_target_encode(
        X_full,
        y_full,
        X_test_full if X_test_full is not None else X_full.iloc[:0],
        cat_cols,
    )
else:
    X_te, X_test_te = X_full.copy(), (X_test_full.copy() if X_test_full is not None else None)

# >>> IMPORTANT: Drop original categorical columns post-TE so models get numeric-only features <<<
X_te = X_te.drop(columns=cat_cols, errors="ignore")
if X_test_te is not None:
    X_test_te = X_test_te.drop(columns=cat_cols, errors="ignore")

# -------------------- Advanced Feature Engineering -------------------- #
print("\n[Advanced Features] Creating enhanced features...")

# Get current numerical columns
current_num_cols = X_te.select_dtypes(include=[np.number]).columns.tolist()

if USE_ADVANCED_FEATURES:
    print("  Creating interaction features...")
    if USE_INTERACTION_FEATURES:
        X_te = create_interaction_features(X_te, current_num_cols[:8], max_interactions=15)
        if X_test_te is not None:
            X_test_te = create_interaction_features(X_test_te, current_num_cols[:8], max_interactions=15)
    
    print("  Creating polynomial features...")
    X_te = create_polynomial_features(X_te, current_num_cols[:6], degree=2, max_features=8)
    if X_test_te is not None:
        X_test_te = create_polynomial_features(X_test_te, current_num_cols[:6], degree=2, max_features=8)
    
    print("  Creating binning features...")
    X_te = create_binning_features(X_te, current_num_cols[:8], n_bins=5)
    if X_test_te is not None:
        X_test_te = create_binning_features(X_test_te, current_num_cols[:8], n_bins=5)
    
    print("  Creating cluster features...")
    X_te = create_cluster_features(X_te, current_num_cols, n_clusters=8)
    if X_test_te is not None:
        X_test_te = create_cluster_features(X_test_te, current_num_cols, n_clusters=8)

if USE_NOISE_INJECTION:
    print("  Adding noise features for regularization...")
    X_te = add_noise_features(X_te, noise_level=NOISE_LEVEL, n_features=3)
    if X_test_te is not None:
        X_test_te = add_noise_features(X_test_te, noise_level=NOISE_LEVEL, n_features=3)

print(f"  Advanced feature engineering complete. New shape: {X_te.shape}")

# Clean any infinite values introduced by feature engineering
for df in [X_te] + ([X_test_te] if X_test_te is not None else []):
    if df is None:
        continue
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan)
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Feature selection
if USE_FEATURE_SELECTION and len(X_te.columns) > SELECT_TOP_K_FEATURES:
    print(f"Selecting top {SELECT_TOP_K_FEATURES} features...")
    selected_features = advanced_feature_selection(X_te, y_full, SELECT_TOP_K_FEATURES)
    X_te = X_te[selected_features]
    if X_test_te is not None:
        X_test_te = X_test_te[selected_features]
    print(f"Selected {len(selected_features)} features")

# Enhanced scaling for linear models
scaler = QuantileTransformer(output_distribution='normal', random_state=SEED)
X_lin = X_te.copy()
lin_num_cols = X_lin.select_dtypes(include=[np.number]).columns
X_lin[lin_num_cols] = scaler.fit_transform(X_lin[lin_num_cols])

if X_test_te is not None:
    X_test_lin = X_test_te.copy()
    lin_test_num = X_test_lin.select_dtypes(include=[np.number]).columns
    X_test_lin[lin_test_num] = scaler.transform(X_test_lin[lin_test_num])
else:
    X_test_lin = None

# Optional target log transform
use_log = False
if AUTO_LOG_TARGET:
    skew_val = pd.Series(y_full).skew()
    if skew_val > 1.0:
        use_log = True
        y_work = np.log1p(y_full)
    else:
        y_work = y_full.copy()
else:
    y_work = y_full.copy()

# Stratified folds by target bins (for stability)
strat_bins = make_target_bins_improved(y_full, n_bins=8)
skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
# -------------------- Base Models with Optuna Optimization -------------------- #
base_models: Dict[str, Tuple[object, str]] = {}

# Run Optuna optimization for key models if enabled
optimized_params = {}
if USE_OPTUNA:
    print("\n=== OPTUNA HYPERPARAMETER OPTIMIZATION ===")
    
    # List of models to optimize
    models_to_optimize = []
    if HAS_LGB:
        models_to_optimize.append("LGBM")
    if HAS_XGB:
        models_to_optimize.append("XGB")
    if HAS_CAT:
        models_to_optimize.append("CAT")
    models_to_optimize.append("RF")  # Always available
    
    # Run optimization for each model
    for model_name in models_to_optimize:
        best_params = run_optuna_optimization(X_te, y_work, model_name)
        if best_params is not None:
            optimized_params[model_name] = best_params
    
    print(f"\n[Optuna] Optimization complete. Optimized {len(optimized_params)} models.")

# Build models with optimized or default parameters
if HAS_LGB:
    if "LGBM" in optimized_params:
        lgb_params = optimized_params["LGBM"]
        lgb_params.update({
            'random_state': SEED,
            'verbose': -1,
            'n_jobs': 1,
            'objective': 'regression',
            'boosting_type': 'gbdt'
        })
    else:
        lgb_params = {
            'n_estimators': 800,
            'learning_rate': 0.05,
            'max_depth': 6,
            'num_leaves': 31,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.1,
            'reg_lambda': 1.0,
            'min_child_samples': 20,
            'random_state': SEED,
            'verbose': -1,
            'n_jobs': 1,
            'objective': 'regression',
            'boosting_type': 'gbdt'
        }
    
    base_models["LGBM"] = (LGBMRegressor(**lgb_params), "tree")

if HAS_XGB:
    if "XGB" in optimized_params:
        xgb_params = optimized_params["XGB"]
        xgb_params.update({
            'random_state': SEED,
            'n_jobs': 1,
            'tree_method': 'hist'
        })
    else:
        xgb_params = {
            'n_estimators': 1200,
            'learning_rate': 0.03,
            'max_depth': 8,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.0,
            'reg_lambda': 1.0,
            'random_state': SEED,
            'n_jobs': 1,
            'tree_method': 'hist'
        }
    
    base_models["XGB"] = (XGBRegressor(**xgb_params), "tree")

if HAS_CAT:
    if "CAT" in optimized_params:
        cat_params = optimized_params["CAT"]
        cat_params.update({
            'random_state': SEED,
            'verbose': False,
            'loss_function': 'RMSE'
        })
    else:
        cat_params = {
            'depth': 8,
            'learning_rate': 0.03,
            'iterations': 1500,
            'subsample': 0.8,
            'random_state': SEED,
            'loss_function': 'RMSE',
            'verbose': False
        }
    
    base_models["CAT"] = (CatBoostRegressor(**cat_params), "tree")

# Random Forest with optimization
if "RF" in optimized_params:
    rf_params = optimized_params["RF"]
    rf_params.update({
        'random_state': SEED,
        'n_jobs': -1
    })
else:
    rf_params = {
        'n_estimators': 800,
        'max_depth': None,
        'max_features': 'sqrt',
        'random_state': SEED,
        'n_jobs': -1
    }

base_models["RF"] = (RandomForestRegressor(**rf_params), "tree")

# Strong classical ensembles (not optimized with Optuna for simplicity and to prevent overfitting)
base_models["ET"] = (
    ExtraTreesRegressor(
        n_estimators=800,
        max_depth=None,
        max_features="sqrt",
        random_state=SEED,
        n_jobs=-1,
    ),
    "tree",
)

base_models["GBDT"] = (
    GradientBoostingRegressor(
        n_estimators=600,
        learning_rate=0.03,
        max_depth=3,
        random_state=SEED,
    ),
    "tree",
)

base_models["Ridge"] = (
    Ridge(alpha=5.0, random_state=SEED),
    "linear",
)
base_models["Lasso"] = (
    Lasso(alpha=0.0005, random_state=SEED, max_iter=20000),
    "linear",
)

# -------------- OOF Training / Prediction -------------- #
models_oof = {}
models_test_pred = {}
models_fold_objs = {}
fold_scores = []

X_tree = X_te
X_tree_test = X_test_te

X_linear = X_lin
X_linear_test = X_test_lin

for name, (model, mtype) in base_models.items():
    print(f"\n[Model={name}] training with {NFOLDS}-fold CV ...")
    oof = np.zeros(len(X_tree))
    test_preds_folds = []
    fold_importances = []

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_tree, strat_bins)):
        if mtype == "tree":
            Xtr, Xva = X_tree.iloc[tr_idx], X_tree.iloc[va_idx]
        else:
            Xtr, Xva = X_linear.iloc[tr_idx], X_linear.iloc[va_idx]

        ytr, yva = y_work.iloc[tr_idx], y_work.iloc[va_idx]

        # Fresh instance per fold
        m = type(model)(**model.get_params())

        # Special handling for different model types
        if name == "LGBM" and HAS_LGB:
            # LightGBM specific handling with clean feature names and reset index
            Xtr_clean = Xtr.copy().reset_index(drop=True)
            Xva_clean = Xva.copy().reset_index(drop=True)
            ytr_clean = ytr.reset_index(drop=True)
            yva_clean = yva.reset_index(drop=True)
            
            # Ensure all data is numeric and clean feature names
            Xtr_clean = Xtr_clean.select_dtypes(include=[np.number])
            Xva_clean = Xva_clean.select_dtypes(include=[np.number])
            
            # Create simple sequential feature names to avoid any naming conflicts
            n_features = len(Xtr_clean.columns)
            simple_names = [f'feature_{i}' for i in range(n_features)]
            
            Xtr_clean.columns = simple_names
            Xva_clean.columns = simple_names
            
            try:
                # Use minimal parameters to avoid conflicts
                lgb_params = {
                    'n_estimators': 500,
                    'learning_rate': 0.1,
                    'max_depth': 6,
                    'num_leaves': 31,
                    'subsample': 0.8,
                    'colsample_bytree': 0.8,
                    'random_state': SEED,
                    'verbose': -1,
                    'n_jobs': 1
                }
                m = LGBMRegressor(**lgb_params)
                
                # Fit without eval_set to avoid feature name conflicts
                m.fit(Xtr_clean, ytr_clean)
                pred_va = m.predict(Xva_clean)
                
            except Exception as e:
                print(f"    Warning: LGBM fold {fold} failed: {e}")
                pred_va = np.full(len(yva_clean), ytr_clean.mean())  # Fallback prediction
                
        elif name == "CAT" and HAS_CAT:
            # CatBoost requires numeric input here (since we dropped original categorical cols).
            m.fit(Xtr.astype(np.float32), ytr, eval_set=(Xva.astype(np.float32), yva), verbose=False)
            pred_va = m.predict(Xva.astype(np.float32))
        else:
            m.fit(Xtr, ytr)
            pred_va = m.predict(Xva)

        oof[va_idx] = pred_va

        if X_tree_test is not None:
            Xte = X_tree_test if mtype == "tree" else X_linear_test
            if name == "LGBM" and HAS_LGB:
                # Apply same feature name cleaning for test predictions
                Xte_clean = Xte.copy().reset_index(drop=True)
                Xte_clean = Xte_clean.select_dtypes(include=[np.number])
                
                # Use same simple sequential names
                n_features = len(Xte_clean.columns)
                simple_names = [f'feature_{i}' for i in range(n_features)]
                Xte_clean.columns = simple_names
                
                try:
                    test_preds_folds.append(m.predict(Xte_clean))
                except Exception as e:
                    print(f"    Warning: LGBM test prediction failed: {e}")
                    test_preds_folds.append(np.full(len(Xte_clean), oof[oof != 0].mean() if len(oof[oof != 0]) > 0 else 0))
            elif name == "CAT" and HAS_CAT:
                test_preds_folds.append(m.predict(Xte.astype(np.float32)))
            else:
                test_preds_folds.append(m.predict(Xte))

        # Save feature importances when available
        if SAVE_FEATURE_IMPORTANCES and hasattr(m, "feature_importances_"):
            fi = pd.DataFrame({
                "feature": Xtr.columns,
                "importance": m.feature_importances_,
                "fold": fold,
                "model": name,
            })
            fold_importances.append(fi)

        models_fold_objs.setdefault(name, []).append(m)

    # Back-transform if log was used
    oof_final = np.expm1(oof) if use_log else oof
    rmse_score = rmse(y_full, oof_final)

    models_oof[name] = oof_final
    if test_preds_folds:
        pred_test_mean = np.mean(test_preds_folds, axis=0)
        pred_test_final = np.expm1(pred_test_mean) if use_log else pred_test_mean
        models_test_pred[name] = pred_test_final

    fold_scores.append({"model": name, "cv_rmse": rmse_score})
    print(f"[Model={name}] CV RMSE: {rmse_score:.5f}")

    if SAVE_FEATURE_IMPORTANCES and len(fold_importances) > 0:
        imp_df = pd.concat(fold_importances, ignore_index=True)
        imp_df.groupby("feature")["importance"].mean().sort_values(ascending=False).head(50).to_csv(
            f"feature_importances_{name}.csv", index=True
        )

# -------------------- Advanced Multi-Level Stacking -------------------- #
print("\n=== ADVANCED MULTI-LEVEL STACKING ===")

# Pseudo-labeling for semi-supervised learning
if USE_PSEUDO_LABELING and X_test_te is not None:
    print("\n[Pseudo-Labeling] Creating high-confidence pseudo-labels...")
    pseudo_X, pseudo_y = create_pseudo_labels(X_test_te, models_oof, models_test_pred, PSEUDO_LABEL_THRESHOLD)
    
    if len(pseudo_y) > 0:
        print(f"  Generated {len(pseudo_y)} pseudo-labels with confidence >= {PSEUDO_LABEL_THRESHOLD}")
        
        # Add pseudo-labels to training data
        X_extended = pd.concat([X_te, pseudo_X], ignore_index=True)
        y_extended = pd.concat([y_full, pseudo_y], ignore_index=True)
        
        # Retrain top 3 models with pseudo-labels
        top_models = sorted(fold_scores, key=lambda d: d["cv_rmse"])[:3]
        for model_info in top_models:
            model_name = model_info["model"]
            if model_name in base_models:
                print(f"  Retraining {model_name} with pseudo-labels...")
                model, mtype = base_models[model_name]
                
                # Create fresh model instance
                m = type(model)(**model.get_params())
                
                if mtype == "tree":
                    X_train_data = X_extended
                else:
                    # Scale extended data for linear models
                    scaler_extended = QuantileTransformer(output_distribution='normal', random_state=SEED)
                    X_train_data = X_extended.copy()
                    numeric_cols = X_train_data.select_dtypes(include=[np.number]).columns
                    X_train_data[numeric_cols] = scaler_extended.fit_transform(X_train_data[numeric_cols])
                
                # Special handling for different model types
                if model_name == "LGBM" and HAS_LGB:
                    X_clean = X_train_data.select_dtypes(include=[np.number]).reset_index(drop=True)
                    n_features = len(X_clean.columns)
                    simple_names = [f'feature_{i}' for i in range(n_features)]
                    X_clean.columns = simple_names
                    try:
                        m.fit(X_clean, y_extended.reset_index(drop=True))
                        # Update test predictions
                        X_test_clean = X_test_te.select_dtypes(include=[np.number]).reset_index(drop=True)
                        X_test_clean.columns = simple_names
                        models_test_pred[model_name] = m.predict(X_test_clean)
                    except:
                        print(f"    Warning: {model_name} pseudo-label retraining failed")
                elif model_name == "CAT" and HAS_CAT:
                    m.fit(X_train_data.astype(np.float32), y_extended)
                    models_test_pred[model_name] = m.predict(X_test_te.astype(np.float32))
                else:
                    m.fit(X_train_data, y_extended)
                    test_data = X_test_te if mtype == "tree" else X_test_lin
                    if test_data is not None:
                        models_test_pred[model_name] = m.predict(test_data)
    else:
        print("  No high-confidence pseudo-labels generated")

# Level 1: Base model predictions (already computed)
oof_df = pd.DataFrame(models_oof)
print(f"\n[Level 1] Base models OOF shape: {oof_df.shape}")

# Level 2: Meta-features and diverse meta-learners
if USE_MULTI_LEVEL_STACKING:
    print("\n[Level 2] Training diverse meta-learners...")
    
    # Create enhanced meta-features
    meta_features = create_meta_features(X_te, models_oof)
    
    # Multiple meta-learners for diversity
    meta_models = {
        'GBDT_meta': GradientBoostingRegressor(
            n_estimators=300, learning_rate=0.05, max_depth=3, 
            subsample=0.8, random_state=SEED
        ),
        'RF_meta': RandomForestRegressor(
            n_estimators=200, max_depth=8, random_state=SEED, n_jobs=-1
        ),
        'Ridge_meta': Ridge(alpha=10.0, random_state=SEED),
    }
    
    meta_oof_preds = {}
    meta_test_preds = {}
    
    for meta_name, meta_model in meta_models.items():
        print(f"  Training {meta_name}...")
        
        # Train on OOF predictions
        meta_model.fit(oof_df, y_full)
        meta_oof = meta_model.predict(oof_df)
        meta_oof_preds[meta_name] = meta_oof
        
        # Predict on test set
        if X_test_te is not None and len(models_test_pred) > 0:
            test_meta_df = pd.DataFrame(models_test_pred)
            meta_test_preds[meta_name] = meta_model.predict(test_meta_df)
        
        rmse_score = rmse(y_full, meta_oof)
        print(f"    {meta_name} CV RMSE: {rmse_score:.5f}")
    
    # Level 3: Final ensemble of meta-learners
    print("\n[Level 3] Final ensemble...")
    
    if USE_ADAPTIVE_WEIGHTS:
        # Calculate adaptive weights based on performance
        model_scores = [{"model": name, "cv_rmse": rmse(y_full, pred)} 
                       for name, pred in meta_oof_preds.items()]
        adaptive_weights = calculate_adaptive_weights([{m["model"]: m["cv_rmse"] for m in model_scores}])
        
        print("  Adaptive weights:")
        for model, weight in adaptive_weights.items():
            print(f"    {model}: {weight:.3f}")
        
        # Weighted ensemble
        final_oof = np.zeros(len(y_full))
        final_test = np.zeros(len(X_test_te) if X_test_te is not None else 0)
        
        for meta_name, weight in adaptive_weights.items():
            if meta_name in meta_oof_preds:
                final_oof += weight * meta_oof_preds[meta_name]
                if meta_name in meta_test_preds:
                    final_test += weight * meta_test_preds[meta_name]
    else:
        # Simple average ensemble
        final_oof = np.mean(list(meta_oof_preds.values()), axis=0)
        if meta_test_preds:
            final_test = np.mean(list(meta_test_preds.values()), axis=0)
    
    final_rmse = rmse(y_full, final_oof)
    print(f"  Final ensemble CV RMSE: {final_rmse:.5f}")
    
    # Store final predictions
    meta_oof = final_oof
    stack_rmse = final_rmse
    
else:
    # Original simple stacking
    print("\n[Simple Stacking] Training single meta-model...")
    meta_model = GradientBoostingRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=3, 
        subsample=0.8, random_state=SEED
    )
    meta_model.fit(oof_df, y_full)
    meta_oof = meta_model.predict(oof_df)
    stack_rmse = rmse(y_full, meta_oof)
    print(f"  Meta CV RMSE: {stack_rmse:.5f}")

# Weighted Blend of the best individual models (for comparison)
sorted_models = sorted(fold_scores, key=lambda d: d["cv_rmse"])[: min(3, len(fold_scores))]
weights = []
for d in sorted_models:
    w = 1.0 / max(d["cv_rmse"], 1e-6)
    weights.append(w)
weights = np.array(weights) / np.sum(weights)

print("\n[Traditional Blending] Top models & weights:")
for (d, w) in zip(sorted_models, weights):
    print(f"  - {d['model']}: weight={w:.3f}, cv_rmse={d['cv_rmse']:.5f}")

# -------------------- Export OOF / Test Predictions -------------------- #
pd.DataFrame({"id": train[ID_COL], "y": y_full, **models_oof}).to_csv("oof_predictions.csv", index=False)
pd.DataFrame(fold_scores).to_csv("fold_scores.csv", index=False)

# Final test prediction (if test exists)
if X_test_full is not None and len(models_test_pred) > 0:
    if USE_MULTI_LEVEL_STACKING and 'final_test' in locals():
        # Use advanced multi-level ensemble prediction
        final_pred = final_test
        print(f"\nUsing advanced multi-level ensemble prediction")
    else:
        # Fallback to traditional stacking
        test_stack_mat = pd.DataFrame(models_test_pred)
        
        if 'meta_model' in locals():
            stack_pred = meta_model.predict(test_stack_mat)
        else:
            stack_pred = test_stack_mat.mean(axis=1).values
        
        # Weighted blend of top models
        blend_pred = np.zeros(len(test_stack_mat))
        for (d, w) in zip(sorted_models, weights):
            if d["model"] in test_stack_mat.columns:
                blend_pred += w * test_stack_mat[d["model"]].values
        
        # Final combo: average of stack & blend (often stabilizes)
        final_pred = 0.6 * stack_pred + 0.4 * blend_pred  # Slightly favor stacking
        print(f"\nUsing traditional stacking + blending")

    sub = pd.DataFrame({ID_COL: test[ID_COL], TARGET: final_pred})
    sub.to_csv("submission_tuned.csv", index=False)
    print("Saved submission_tuned.csv")

    print("\n=== ULTRA-TUNED PIPELINE SUMMARY ===")
    print(f"Advanced Stack CV RMSE: {stack_rmse:.5f}")
    print(f"Feature Engineering: {X_te.shape[1]} total features")
    print(f"Pseudo-labeling: {'Enabled' if USE_PSEUDO_LABELING else 'Disabled'}")
    print(f"Multi-level Stacking: {'Enabled' if USE_MULTI_LEVEL_STACKING else 'Disabled'}")
    print(f"Adaptive Weights: {'Enabled' if USE_ADAPTIVE_WEIGHTS else 'Disabled'}")
    
    print("\nIndividual Model Performance:")
    for d in sorted(fold_scores, key=lambda d: d["cv_rmse"]):
        print(f"{d['model']:<8} | CV RMSE: {d['cv_rmse']:.5f}")
    
    print(f"\nSubmission Details:")
    print(f"Shape: {sub.shape}")
    print(f"Target stats: mean={final_pred.mean():.4f} | std={final_pred.std():.4f}")
    print(f"Range: [{final_pred.min():.4f}, {final_pred.max():.4f}]")
    
    # Save additional analysis
    analysis = {
        'advanced_stack_rmse': stack_rmse,
        'total_features': X_te.shape[1],
        'pseudo_labeling': USE_PSEUDO_LABELING,
        'multi_level_stacking': USE_MULTI_LEVEL_STACKING,
        'adaptive_weights': USE_ADAPTIVE_WEIGHTS,
        'individual_scores': {d['model']: d['cv_rmse'] for d in fold_scores}
    }

# Approach using NNs (which didn't work out...)

In [ ]:
import os
import re
import warnings
from typing import List, Tuple, Optional
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
HAS_XGB = True

warnings.filterwarnings("ignore")

SEED = 42
NFOLDS = 5
TARGET = "CORRUCYSTIC_DENSITY"
ID_COL = "LOCAL_IDENTIFIER"
USE_LOG_TARGET = False
SMOOTHING_M = 10.0

# Neural Network Configuration
USE_MPS = torch.backends.mps.is_available() and torch.backends.mps.is_built()
DEVICE = "mps" if USE_MPS else "cuda" if torch.cuda.is_available() else "cpu"
NN_EPOCHS = 100
NN_BATCH_SIZE = 256
NN_LEARNING_RATE = 0.001
NN_PATIENCE = 15

CREATE_ADVANCED_FEATURES = True
N_CLUSTERS = 5
N_PCA_COMPONENTS = 8

print(f"Using device: {DEVICE}")
if USE_MPS:
    print("✅ MPS (Apple Silicon GPU) acceleration enabled!")

def seed_everything(seed: int = SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def read_first_existing(paths: List[str]) -> Optional[str]:
    for p in paths:
        if os.path.exists(p):
            return p
    return None

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"[^A-Za-z0-9_]", "_", str(c)).strip("_") for c in df.columns]
    df.columns = [re.sub(r"_+", "_", c) for c in df.columns]
    df.columns = [f"col_{i}" if c == "" else c for i, c in enumerate(df.columns)]
    return df

def safe_imputation(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # Handle numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        # Replace infinite values first
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        
        # Simple median imputation
        if df[col].isnull().any():
            median_val = df[col].median()
            if pd.isna(median_val):
                median_val = 0.0
            df[col] = df[col].fillna(median_val)
    
    # Handle categorical columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    for col in categorical_cols:
        mode_val = df[col].mode()
        if len(mode_val) > 0:
            df[col] = df[col].fillna(mode_val.iloc[0])
        else:
            df[col] = df[col].fillna("UNKNOWN")
    
    return df

def create_simple_features(df: pd.DataFrame, target_col: str = None) -> pd.DataFrame:
    df = df.copy()
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if target_col and target_col in numeric_cols:
        numeric_cols.remove(target_col)
    
    if len(numeric_cols) == 0:
        return df
    
    print("Creating simple features...")
    
    # Basic statistics (robust)
    df['feat_mean'] = df[numeric_cols].mean(axis=1)
    df['feat_std'] = df[numeric_cols].std(axis=1).fillna(0)
    df['feat_median'] = df[numeric_cols].median(axis=1)
    df['feat_min'] = df[numeric_cols].min(axis=1)
    df['feat_max'] = df[numeric_cols].max(axis=1)
    df['feat_range'] = df['feat_max'] - df['feat_min']
    
    # Count features
    df['feat_positive'] = (df[numeric_cols] > 0).sum(axis=1)
    df['feat_negative'] = (df[numeric_cols] < 0).sum(axis=1)
    df['feat_zero'] = (df[numeric_cols] == 0).sum(axis=1)
    
    # Simple ratios
    df['feat_pos_ratio'] = df['feat_positive'] / len(numeric_cols)
    df['feat_neg_ratio'] = df['feat_negative'] / len(numeric_cols)
    
    # Handle any remaining infinite/nan values
    new_cols = ['feat_mean', 'feat_std', 'feat_median', 'feat_min', 'feat_max', 
                'feat_range', 'feat_positive', 'feat_negative', 'feat_zero',
                'feat_pos_ratio', 'feat_neg_ratio']
    
    for col in new_cols:
        if col in df.columns:
            df[col] = df[col].replace([np.inf, -np.inf], np.nan)
            df[col] = df[col].fillna(df[col].median())
    
    return df

def enhanced_target_encode(
    X: pd.DataFrame,
    y: pd.Series,
    X_test: pd.DataFrame,
    cat_cols: List[str],
    n_splits: int = NFOLDS,
    smoothing_m: float = SMOOTHING_M,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    if len(cat_cols) == 0:
        return X.copy(), X_test.copy()
    
    X = X.copy()
    X_test = X_test.copy()
    global_mean = y.mean()
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    for col in cat_cols:
        print(f"Target encoding {col}...")
        oof_encoded = np.full(len(X), global_mean)
        test_encoded_list = []
        
        for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
            tr_y = y.iloc[tr_idx]
            tr_col = X.iloc[tr_idx][col].astype(str).fillna("MISSING")
            val_col = X.iloc[val_idx][col].astype(str).fillna("MISSING")
            
            # Calculate target mean for each category
            target_mean = tr_y.groupby(tr_col).mean()
            target_count = tr_y.groupby(tr_col).count()
            
            # Apply smoothing
            smoothed_mean = (target_mean * target_count + global_mean * smoothing_m) / (target_count + smoothing_m)
            
            # Encode validation set
            oof_encoded[val_idx] = val_col.map(smoothed_mean).fillna(global_mean)
            
            # Encode test set
            test_col = X_test[col].astype(str).fillna("MISSING")
            test_encoded = test_col.map(smoothed_mean).fillna(global_mean)
            test_encoded_list.append(test_encoded)
        
        # Final encoding
        X[f'{col}_te'] = oof_encoded
        X_test[f'{col}_te'] = np.mean(test_encoded_list, axis=0)
    
    return X, X_test

class SimpleTabularNN(nn.Module):
    def __init__(self, input_dim: int, hidden_dims: List[int] = [256, 128, 64]):
        super(SimpleTabularNN, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)
        
        # Initialize weights
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                nn.init.constant_(module.bias, 0)
    
    def forward(self, x):
        return self.network(x).squeeze()

def train_simple_nn(X_train, y_train, X_val, y_val, epochs=NN_EPOCHS):
    input_dim = X_train.shape[1]
    model = SimpleTabularNN(input_dim, [256, 128, 64]).to(DEVICE)
    
    # Convert to tensors
    X_train_tensor = torch.FloatTensor(X_train).to(DEVICE)
    y_train_tensor = torch.FloatTensor(y_train).to(DEVICE)
    X_val_tensor = torch.FloatTensor(X_val).to(DEVICE)
    y_val_tensor = torch.FloatTensor(y_val).to(DEVICE)
    
    # Create data loaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=NN_BATCH_SIZE, shuffle=True)
    
    # Optimizer and loss
    optimizer = optim.Adam(model.parameters(), lr=NN_LEARNING_RATE)
    criterion = nn.MSELoss()
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor).item()
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= NN_PATIENCE:
                break
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch}: Train Loss={train_loss/len(train_loader):.6f}, Val Loss={val_loss:.6f}")
    
    return model

def predict_simple_nn(model, X):
    model.eval()
    X_tensor = torch.FloatTensor(X).to(DEVICE)
    
    with torch.no_grad():
        predictions = model(X_tensor).cpu().numpy()
    
    return predictions

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

seed_everything(SEED)
print("🚀 Enhanced Robust Pipeline Starting...")

# Load data
train_path = "./MiNDAT.csv"
test_path = "./MiNDAT_UNK.csv"

train = pd.read_csv(train_path)
train = clean_columns(train)

test = pd.read_csv(test_path)
test = clean_columns(test)

# Data cleaning
if TARGET not in train.columns:
    raise KeyError(f"Target column '{TARGET}' not found!")

# Remove rows with missing target
train = train.dropna(subset=[TARGET]).reset_index(drop=True)

# Simple outlier removal using IQR
Q1, Q3 = train[TARGET].quantile([0.25, 0.75])
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
train = train[(train[TARGET] >= lower_bound) & (train[TARGET] <= upper_bound)].reset_index(drop=True)

# Prepare features
features = [c for c in train.columns if c not in [TARGET, ID_COL]]
X_full = train[features].copy()
y_full = train[TARGET].copy()

if test is not None:
    X_test_full = test[features].copy()
else:
    X_test_full = X_full.iloc[:0].copy()

# Safe imputation
print("🔧 Performing safe imputation...")
X_full = safe_imputation(X_full)
if test is not None:
    X_test_full = safe_imputation(X_test_full)

# Identify column types
cat_cols = X_full.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X_full.select_dtypes(include=[np.number]).columns.tolist()

# Target encoding
if len(cat_cols) > 0:
    print("Performing target encoding...")
    X_full, X_test_full = enhanced_target_encode(X_full, y_full, X_test_full, cat_cols)
    # Remove original categorical columns
    X_full = X_full.drop(columns=cat_cols)
    X_test_full = X_test_full.drop(columns=cat_cols)

# Feature engineering
if CREATE_ADVANCED_FEATURES:
    print("Creating features...")
    X_full = create_simple_features(X_full, TARGET)
    if test is not None:
        X_test_full = create_simple_features(X_test_full, TARGET)

print(f"🚀 Final feature count: {X_full.shape[1]}")

# Final safety check and scaling
X_full = safe_imputation(X_full)
X_test_full = safe_imputation(X_test_full)

# Feature scaling
print("📏 Scaling features...")
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_full)
X_test_scaled = scaler.transform(X_test_full)

# Cross-validation
print(f"\n🔄 Starting {NFOLDS}-fold cross-validation...")
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

# Initialize results storage
models = {}
oof_predictions = {}
test_predictions = {}
cv_scores = {}

# ==================== Neural Network Training ==================== #
print("\n🧠 Training Neural Networks...")

nn_oof = np.zeros(len(X_full))
nn_test_preds = []
nn_fold_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\n📊 Neural Net Fold {fold + 1}/{NFOLDS}")
    
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_full.iloc[train_idx].values, y_full.iloc[val_idx].values
    
    # Train model
    model = train_simple_nn(X_train, y_train, X_val, y_val)
    
    # Predictions
    val_pred = predict_simple_nn(model, X_val)
    nn_oof[val_idx] = val_pred
    
    # Test predictions
    if test is not None:
        test_pred = predict_simple_nn(model, X_test_scaled)
        nn_test_preds.append(test_pred)
    
    # Score
    fold_rmse = rmse(y_val, val_pred)
    nn_fold_scores.append(fold_rmse)
    print(f"🎯 Fold {fold + 1} Neural Net RMSE: {fold_rmse:.6f}")

nn_cv_score = rmse(y_full, nn_oof)
print(f"\n🧠 Neural Network CV RMSE: {nn_cv_score:.6f}")

# Store neural network results
oof_predictions['NeuralNet'] = nn_oof
cv_scores['NeuralNet'] = nn_cv_score

if test is not None and len(nn_test_preds) > 0:
    test_predictions['NeuralNet'] = np.mean(nn_test_preds, axis=0)

# ==================== Classical ML Models ==================== #
print("\n🔧 Training Classical ML Models...")

# Prepare data for tree models (unscaled)
X_tree = X_full.values
X_tree_test = X_test_full.values

# 1. Random Forest
print("🌲 Training Random Forest...")
rf_oof = np.zeros(len(X_full))
rf_test_preds = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_tree)):
    X_train, X_val = X_tree[train_idx], X_tree[val_idx]
    y_train, y_val = y_full.iloc[train_idx], y_full.iloc[val_idx]
    
    rf = RandomForestRegressor(
        n_estimators=200,
        max_depth=10,
        random_state=SEED,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    
    val_pred = rf.predict(X_val)
    rf_oof[val_idx] = val_pred
    
    if test is not None:
        rf_test_preds.append(rf.predict(X_tree_test))

rf_cv_score = rmse(y_full, rf_oof)
print(f"🌲 Random Forest CV RMSE: {rf_cv_score:.6f}")

oof_predictions['RandomForest'] = rf_oof
cv_scores['RandomForest'] = rf_cv_score
if test is not None:
    test_predictions['RandomForest'] = np.mean(rf_test_preds, axis=0)

# 2. XGBoost (if available)
if HAS_XGB:
    print("🚀 Training XGBoost...")
    xgb_oof = np.zeros(len(X_full))
    xgb_test_preds = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_tree)):
        X_train, X_val = X_tree[train_idx], X_tree[val_idx]
        y_train, y_val = y_full.iloc[train_idx], y_full.iloc[val_idx]
        
        xgb = XGBRegressor(
            n_estimators=200,
            max_depth=6,
            learning_rate=0.1,
            random_state=SEED,
            n_jobs=-1
        )
        xgb.fit(X_train, y_train)
        
        val_pred = xgb.predict(X_val)
        xgb_oof[val_idx] = val_pred
        
        if test is not None:
            xgb_test_preds.append(xgb.predict(X_tree_test))
    
    xgb_cv_score = rmse(y_full, xgb_oof)
    print(f"🚀 XGBoost CV RMSE: {xgb_cv_score:.6f}")
    
    oof_predictions['XGBoost'] = xgb_oof
    cv_scores['XGBoost'] = xgb_cv_score
    if test is not None:
        test_predictions['XGBoost'] = np.mean(xgb_test_preds, axis=0)

# 3. Ridge Regression
print("📈 Training Ridge Regression...")
ridge_oof = np.zeros(len(X_full))
ridge_test_preds = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_full.iloc[train_idx], y_full.iloc[val_idx]
    
    ridge = Ridge(alpha=1.0, random_state=SEED)
    ridge.fit(X_train, y_train)
    
    val_pred = ridge.predict(X_val)
    ridge_oof[val_idx] = val_pred
    
    if test is not None:
        ridge_test_preds.append(ridge.predict(X_test_scaled))

ridge_cv_score = rmse(y_full, ridge_oof)
print(f"📈 Ridge CV RMSE: {ridge_cv_score:.6f}")

oof_predictions['Ridge'] = ridge_oof
cv_scores['Ridge'] = ridge_cv_score
if test is not None:
    test_predictions['Ridge'] = np.mean(ridge_test_preds, axis=0)

# ==================== Ensemble ==================== #
print("\n🎯 Creating Ensemble...")

# Simple average ensemble
if len(oof_predictions) > 1:
    ensemble_oof = np.mean(list(oof_predictions.values()), axis=0)
    ensemble_cv_score = rmse(y_full, ensemble_oof)
    print(f"🎯 Ensemble CV RMSE: {ensemble_cv_score:.6f}")
    
    if test is not None and len(test_predictions) > 0:
        ensemble_test = np.mean(list(test_predictions.values()), axis=0)
        test_predictions['Ensemble'] = ensemble_test
    
    oof_predictions['Ensemble'] = ensemble_oof
    cv_scores['Ensemble'] = ensemble_cv_score

# ==================== Results Summary ==================== #
print("\n📊 Final Results Summary:")
print("=" * 50)
for model_name, score in sorted(cv_scores.items(), key=lambda x: x[1]):
    print(f"{model_name:<15}: {score:.6f} RMSE")

# OOF predictions
oof_df = pd.DataFrame({
    'id': train[ID_COL],
    'y_true': y_full,
    **oof_predictions
})
oof_df.to_csv('enhanced_oof_predictions.csv', index=False)

# Test predictions
if test is not None and len(test_predictions) > 0:
    # Use best model for final submission
    best_model = min(cv_scores.items(), key=lambda x: x[1])[0]
    print(f"🏆 Best model: {best_model} (RMSE: {cv_scores[best_model]:.6f})")
    
    submission = pd.DataFrame({
        ID_COL: test[ID_COL],
        TARGET: test_predictions[best_model]
    })
    
    # Safety check for submission
    submission[TARGET] = submission[TARGET].clip(
        train[TARGET].quantile(0.01),
        train[TARGET].quantile(0.99)
    )
    
    submission.to_csv('enhanced_submission.csv', index=False)

# After this there was another pipeline where i tried optimisign the neural netwrok with Optuna and added a better Meta Learner plus a more robust stacking with both simpler models and advanced models like LGBM, Catboost... however the code for that is something i am not able to find... 🥲